In [ ]:
# ====Klook 크롤링 시스템=========================================================
# 🚀 그룹 1: 통일된 함수명 - (hashlib 리팩토링 완료)
# - 도시 정보를 UNIFIED_CITY_INFO로 통합하여 단일 소스로 관리
# - hashlib 기반 초고속 중복 방지 시스템 추가
#cd "/mnt/c/Users/redsk/OneDrive/デスクトップ/mikael_project/test_folder"
# =============================================================================

import pandas as pd
import warnings, os, time, shutil, urllib, random
import threading 
_csv_loading_lock = threading.Lock() ## 🔒 자물쇠 걸기
warnings.filterwarnings(action='ignore')
import platform
import re                        # 가격/평점 정제용 정규식
import json                      # 메타데이터 JSON 저장용
import hashlib                   # 🆕 초고속 URL 중복 방지 시스템용
from datetime import datetime    # 타임스탬프용
from PIL import Image
from typing import List, Dict, Tuple, Optional
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException, WebDriverException

import chromedriver_autoinstaller
import undetected_chromedriver as uc
from user_agents import parse
import selenium

print(f"🔧 Selenium 버전: {selenium.__version__}")

# ⭐⭐⭐ 중요 설정: 여기서 수정하세요! ⭐⭐⭐
CONFIG = {
    "WAIT_TIMEOUT": 10,
    "RETRY_COUNT": 3,
    "POPUP_WAIT": 5,
    "SAVE_IMAGES": True,
    

    # 🆕 hashlib 시스템 설정
    "USE_HASH_SYSTEM": True,       # hashlib 시스템 사용 여부
    "HASH_LENGTH": 12,             # 해시 길이 (기본 12자리)
    "KEEP_CSV_SYSTEM": True,       # 기존 CSV 시스템 병행 유지

    # 🆕 V2 3-tier URL 시스템 설정
    "USE_V2_URL_SYSTEM": True,     # V2 3-tier URL 시스템 사용
    "V2_URL_COLLECTED": "url_collected",    # 수집 대기 URL 폴더
    "V2_URL_DONE": "url_done",             # 완료된 URL 폴더
    "V2_URL_PROGRESS": "url_progress",     # 진행 상황 폴더

    # 🆕 페이지 최적화 설정 추가
    "SMART_WAIT_MAX": 8,          # smart_wait_for_page_load 최대 대기
    "NEW_TAB_ENABLED": False,      # 새 탭 크롤링 활성화
    "PAGE_LOAD_TIMEOUT": 6,       # 페이지 로드 타임아웃

    "SHORT_MIN_DELAY": 0.2,    # 타이핑 간격 (0.2초 ~ 0.5초)
    "SHORT_MAX_DELAY": 0.5,

    "MEDIUM_MIN_DELAY": 7,     # 페이지 로드 등 일반 대기 (7초 ~ 15초)
    "MEDIUM_MAX_DELAY": 15,

    "LONG_MIN_DELAY": 20,      # 가끔씩 쉬는 시간 (20초 ~ 40초)
    "LONG_MAX_DELAY": 40,
    
    "MAX_PRODUCTS_PER_CITY": 1,     #⭐⭐⭐⭐⭐⭐⭐⭐⭐#
    
    # 🆕 Gemini 지적사항 해결: USER_AGENT 추가
    "USER_AGENT": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
}

# 🏙️ 검색할 도시들 (여기서 변경!)
CITIES_TO_SEARCH = ["서울"]

# =============================================================================
# 📍 [최종 수정본] 단일 정보 소스 및 리팩토링된 함수
# =============================================================================

UNIFIED_CITY_INFO = {
    # -------------------------------
    # 한국
    # -------------------------------
    "서울": {"대륙": "아시아", "국가": "대한민국", "코드": "SEL", "영문명": "seoul"},
    "부산": {"대륙": "아시아", "국가": "대한민국", "코드": "PUS", "영문명": "busan"},
    "제주": {"대륙": "아시아", "국가": "대한민국", "코드": "CJU", "영문명": "jeju"},
    "대구": {"대륙": "아시아", "국가": "대한민국", "코드": "TAE", "영문명": "daegu"},
    "광주": {"대륙": "아시아", "국가": "대한민국", "코드": "KWJ", "영문명": "gwangju"},
    "여수": {"대륙": "아시아", "국가": "대한민국", "코드": "RSU", "영문명": "yeosu"},
    "인천": {"대륙": "아시아", "국가": "대한민국", "코드": "ICN", "영문명": "incheon"},
    "속초": {"대륙": "아시아", "국가": "대한민국", "코드": "SOK", "영문명": "sokcho"},
    "강릉": {"대륙": "아시아", "국가": "대한민국", "코드": "GAN", "영문명": "gangneung"},
    "김포": {"대륙": "아시아", "국가": "대한민국", "코드": "GMP", "영문명": "gimpo"},

    # -------------------------------
    # 동남아시아
    # -------------------------------
    # 태국
    "방콕": {"대륙": "아시아", "국가": "태국", "코드": "BKK", "영문명": "bangkok"},
    "파타야": {"대륙": "아시아", "국가": "태국", "코드": "BKK", "영문명": "pattaya"},
    "아유타야": {"대륙": "아시아", "국가": "태국", "코드": "BKK", "영문명": "ayutthaya"},
    "치앙마이": {"대륙": "아시아", "국가": "태국", "코드": "CNX", "영문명": "chiang mai"},
    "빠이": {"대륙": "아시아", "국가": "태국", "코드": "CNX", "영문명": "pai"},
    "치앙라이": {"대륙": "아시아", "국가": "태국", "코드": "CEI", "영문명": "chiang rai"},
    "푸켓": {"대륙": "아시아", "국가": "태국", "코드": "HKT", "영문명": "phuket"},
    "피피": {"대륙": "아시아", "국가": "태국", "코드": "KBV", "영문명": "phi phi"},
    "크라비": {"대륙": "아시아", "국가": "태국", "코드": "KBV", "영문명": "krabi"},
    "후아힌": {"대륙": "아시아", "국가": "태국", "코드": "HHQ", "영문명": "hua hin"},
    "코사무이": {"대륙": "아시아", "국가": "태국", "코드": "USM", "영문명": "koh samui"},
    "코팡안": {"대륙": "아시아", "국가": "태국", "코드": "USM", "영문명": "koh phangan"},  # 코사무이 경유

    # 싱가포르
    "싱가포르": {"대륙": "아시아", "국가": "싱가포르", "코드": "SIN", "영문명": "singapore"},

    # 말레이시아
    "쿠알라룸푸르": {"대륙": "아시아", "국가": "말레이시아", "코드": "KUL", "영문명": "kuala lumpur"},
    "코타키나발루": {"대륙": "아시아", "국가": "말레이시아", "코드": "BKI", "영문명": "kota kinabalu"},
    "페낭": {"대륙": "아시아", "국가": "말레이시아", "코드": "PEN", "영문명": "penang"},
    "랑카위": {"대륙": "아시아", "국가": "말레이시아", "코드": "LGK", "영문명": "langkawi"},
    "조호르바루": {"대륙": "아시아", "국가": "말레이시아", "코드": "JHB", "영문명": "johor bahru"},

    # 필리핀
    "세부": {"대륙": "아시아", "국가": "필리핀", "코드": "CEB", "영문명": "cebu"},
    "보홀": {"대륙": "아시아", "국가": "필리핀", "코드": "TAG", "영문명": "bohol"},
    "마닐라": {"대륙": "아시아", "국가": "필리핀", "코드": "MNL", "영문명": "manila"},
    "보라카이": {"대륙": "아시아", "국가": "필리핀", "코드": "MPH", "영문명": "boracay"},
    "팔라완": {"대륙": "아시아", "국가": "필리핀", "코드": "PPS", "영문명": "palawan"},
    "다바오": {"대륙": "아시아", "국가": "필리핀", "코드": "DVO", "영문명": "davao"},

    # 베트남
    "다낭": {"대륙": "아시아", "국가": "베트남", "코드": "DAD", "영문명": "da nang"},
    "호이안": {"대륙": "아시아", "국가": "베트남", "코드": "DAD", "영문명": "hoi an"},
    "후에": {"대륙": "아시아", "국가": "베트남", "코드": "HUI", "영문명": "hue"},
    "호치민": {"대륙": "아시아", "국가": "베트남", "코드": "SGN", "영문명": "ho chi minh city"},
    "무이네": {"대륙": "아시아", "국가": "베트남", "코드": "SGN", "영문명": "mui ne"},
    "푸꾸옥": {"대륙": "아시아", "국가": "베트남", "코드": "PQC", "영문명": "phu quoc"},
    "나트랑": {"대륙": "아시아", "국가": "베트남", "코드": "CXR", "영문명": "nha trang"},
    "하노이": {"대륙": "아시아", "국가": "베트남", "코드": "HAN", "영문명": "hanoi"},
    "달랏": {"대륙": "아시아", "국가": "베트남", "코드": "DLI", "영문명": "da lat"},
    "사파": {"대륙": "아시아", "국가": "베트남", "코드": "HAN", "영문명": "sapa"},  # 하노이 경유
    "껀터": {"대륙": "아시아", "국가": "베트남", "코드": "VCA", "영문명": "can tho"},

    # 캄보디아
    "프놈펜": {"대륙": "아시아", "국가": "캄보디아", "코드": "PNH", "영문명": "phnom penh"},
    "시엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP", "영문명": "siem reap"},
    "씨엠립": {"대륙": "아시아", "국가": "캄보디아", "코드": "REP", "영문명": "siem reap"},

    # 라오스
    "비엔티안": {"대륙": "아시아", "국가": "라오스", "코드": "VTE", "영문명": "vientiane"},
    "방비엥": {"대륙": "아시아", "국가": "라오스", "코드": "VTE", "영문명": "vang vieng"},
    "루앙프라방": {"대륙": "아시아", "국가": "라오스", "코드": "LPQ", "영문명": "luang prabang"},

    # 인도네시아
    "발리": {"대륙": "아시아", "국가": "인도네시아", "코드": "DPS", "영문명": "bali"},

    # 몰디브
    "말레": {"대륙": "아시아", "국가": "몰디브", "코드": "MLE", "영문명": "male"},

    # 스리랑카
    "콜롬보": {"대륙": "아시아", "국가": "스리랑카", "코드": "CMB", "영문명": "colombo"},

    # 우즈베키스탄
    "타슈켄트": {"대륙": "아시아", "국가": "우즈베키스탄", "코드": "TAS", "영문명": "tashkent"},
    "사마르칸트": {"대륙": "아시아", "국가": "우즈베키스탄", "코드": "SKD", "영문명": "samarkand"},
    "부하라": {"대륙": "아시아", "국가": "우즈베키스탄", "코드": "BHK", "영문명": "bukhara"},
}

# -------------------------------
# 일본 + 중국/대만
# -------------------------------
UNIFIED_CITY_INFO.update({
    # 일본 (기존 + 추가)
    "도쿄": {"대륙": "아시아", "국가": "일본", "코드": "NRT", "영문명": "tokyo"},
    "오사카": {"대륙": "아시아", "국가": "일본", "코드": "KIX", "영문명": "osaka"},
    "교토": {"대륙": "아시아", "국가": "일본", "코드": "KIX", "영문명": "kyoto"},  # KIX 이용
    "나고야": {"대륙": "아시아", "국가": "일본", "코드": "NGO", "영문명": "nagoya"},
    "후쿠오카": {"대륙": "아시아", "국가": "일본", "코드": "FUK", "영문명": "fukuoka"},
    "벳푸": {"대륙": "아시아", "국가": "일본", "코드": "OIT", "영문명": "beppu"},  # OIT 이용
    "오이타": {"대륙": "아시아", "국가": "일본", "코드": "OIT", "영문명": "oita"},
    "구마모토": {"대륙": "아시아", "국가": "일본", "코드": "KMJ", "영문명": "kumamoto"},
    "오키나와": {"대륙": "아시아", "국가": "일본", "코드": "OKA", "영문명": "okinawa"},
    "미야코지마": {"대륙": "아시아", "국가": "일본", "코드": "MMY", "영문명": "miyakojima"},
    "삿포로": {"대륙": "아시아", "국가": "일본", "코드": "CTS", "영문명": "sapporo"},
    # 일본 추가
    "나가사키": {"대륙": "아시아", "국가": "일본", "코드": "NGS", "영문명": "nagasaki"},
    "사가": {"대륙": "아시아", "국가": "일본", "코드": "HSG", "영문명": "saga"},
    "미야자키": {"대륙": "아시아", "국가": "일본", "코드": "KMI", "영문명": "miyazaki"},
    "가고시마": {"대륙": "아시아", "국가": "일본", "코드": "KOJ", "영문명": "kagoshima"},
    "요코하마": {"대륙": "아시아", "국가": "일본", "코드": "HND", "영문명": "yokohama"},  # 도쿄 하네다 이용
    "나라": {"대륙": "아시아", "국가": "일본", "코드": "KIX", "영문명": "nara"},      # 간사이 이용
    "히로시마": {"대륙": "아시아", "국가": "일본", "코드": "HIJ", "영문명": "hiroshima"},
    "하코다테": {"대륙": "아시아", "국가": "일본", "코드": "HKD", "영문명": "hakodate"},

    # 중국/대만
    "타이베이": {"대륙": "아시아", "국가": "대만", "코드": "TPE", "영문명": "taipei"},
    "가오슝": {"대륙": "아시아", "국가": "대만", "코드": "KHH", "영문명": "kaohsiung"},
    "타이중": {"대륙": "아시아", "국가": "대만", "코드": "RMQ", "영문명": "taichung"},
    "상하이": {"대륙": "아시아", "국가": "중국", "코드": "PVG", "영문명": "shanghai"},
    "베이징": {"대륙": "아시아", "국가": "중국", "코드": "PEK", "영문명": "beijing"},
    "싼야": {"대륙": "아시아", "국가": "중국", "코드": "SYX", "영문명": "sanya"},
    "홍콩": {"대륙": "아시아", "국가": "홍콩", "코드": "HKG", "영문명": "hong kong"},
    "마카오": {"대륙": "아시아", "국가": "마카오", "코드": "MFM", "영문명": "macau"},
})

# -------------------------------
# 유럽 (이탈리아 확장 + 추천지 포함)
# -------------------------------
UNIFIED_CITY_INFO.update({
    # 프랑스
    "파리": {"대륙": "유럽", "국가": "프랑스", "코드": "CDG", "영문명": "paris"},
    "니스": {"대륙": "유럽", "국가": "프랑스", "코드": "NCE", "영문명": "nice"},
    "리옹": {"대륙": "유럽", "국가": "프랑스", "코드": "LYS", "영문명": "lyon"},

    # 영국·아일랜드
    "런던": {"대륙": "유럽", "국가": "영국", "코드": "LHR", "영문명": "london"},
    "더블린": {"대륙": "유럽", "국가": "아일랜드", "코드": "DUB", "영문명": "dublin"},

    # 이탈리아 (기존 + 아말피 해안)
    "로마": {"대륙": "유럽", "국가": "이탈리아", "코드": "FCO", "영문명": "rome"},
    "피렌체": {"대륙": "유럽", "국가": "이탈리아", "코드": "FLR", "영문명": "florence"},
    "베네치아": {"대륙": "유럽", "국가": "이탈리아", "코드": "VCE", "영문명": "venice"},
    "밀라노": {"대륙": "유럽", "국가": "이탈리아", "코드": "MXP", "영문명": "milan"},
    "나폴리": {"대륙": "유럽", "국가": "이탈리아", "코드": "NAP", "영문명": "naples"},
    "아말피": {"대륙": "유럽", "국가": "이탈리아", "코드": "NAP", "영문명": "amalfi"},
    "포시타노": {"대륙": "유럽", "국가": "이탈리아", "코드": "NAP", "영문명": "positano"},
    "라벨로": {"대륙": "유럽", "국가": "이탈리아", "코드": "NAP", "영문명": "ravello"},
    "소렌토": {"대륙": "유럽", "국가": "이탈리아", "코드": "NAP", "영문명": "sorrento"},
    "카프리": {"대륙": "유럽", "국가": "이탈리아", "코드": "NAP", "영문명": "capri"},
    "팔레르모": {"대륙": "유럽", "국가": "이탈리아", "코드": "PMO", "영문명": "palermo"},
    "카타니아": {"대륙": "유럽", "국가": "이탈리아", "코드": "CTA", "영문명": "catania"},
    "토리노": {"대륙": "유럽", "국가": "이탈리아", "코드": "TRN", "영문명": "turin"},
    "제노바": {"대륙": "유럽", "국가": "이탈리아", "코드": "GOA", "영문명": "genoa"},
    "베로나": {"대륙": "유럽", "국가": "이탈리아", "코드": "VRN", "영문명": "verona"},
    "피사": {"대륙": "유럽", "국가": "이탈리아", "코드": "PSA", "영문명": "pisa"},
    "시에나": {"대륙": "유럽", "국가": "이탈리아", "코드": "FLR", "영문명": "siena"},  # FLR 이용

    # 스페인
    "바르셀로나": {"대륙": "유럽", "국가": "스페인", "코드": "BCN", "영문명": "barcelona"},
    "마드리드": {"대륙": "유럽", "국가": "스페인", "코드": "MAD", "영문명": "madrid"},
    "세비야": {"대륙": "유럽", "국가": "스페인", "코드": "SVQ", "영문명": "seville"},
    "그라나다": {"대륙": "유럽", "국가": "스페인", "코드": "GRX", "영문명": "granada"},
    "이비자": {"대륙": "유럽", "국가": "스페인", "코드": "IBZ", "영문명": "ibiza"},
    "발렌시아": {"대륙": "유럽", "국가": "스페인", "코드": "VLC", "영문명": "valencia"},
    "말라가": {"대륙": "유럽", "국가": "스페인", "코드": "AGP", "영문명": "malaga"},

    # 포르투갈
    "리스본": {"대륙": "유럽", "국가": "포르투갈", "코드": "LIS", "영문명": "lisbon"},
    "포르투": {"대륙": "유럽", "국가": "포르투갈", "코드": "OPO", "영문명": "porto"},

    # 중부·동유럽
    "프라하": {"대륙": "유럽", "국가": "체코", "코드": "PRG", "영문명": "prague"},
    "비엔나": {"대륙": "유럽", "국가": "오스트리아", "코드": "VIE", "영문명": "vienna"},
    "부다페스트": {"대륙": "유럽", "국가": "헝가리", "코드": "BUD", "영문명": "budapest"},
    "바르샤바": {"대륙": "유럽", "국가": "폴란드", "코드": "WAW", "영문명": "warsaw"},
    "크라쿠프": {"대륙": "유럽", "국가": "폴란드", "코드": "KRK", "영문명": "krakow"},

    # 스위스
    "취리히": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH", "영문명": "zurich"},
    "인터라켄": {"대륙": "유럽", "국가": "스위스", "코드": "ZRH", "영문명": "interlaken"},  # ZRH 이용
    "제네바": {"대륙": "유럽", "국가": "스위스", "코드": "GVA", "영문명": "geneva"},

    # 베네룩스·북유럽·발칸
    "암스테르담": {"대륙": "유럽", "국가": "네덜란드", "코드": "AMS", "영문명": "amsterdam"},
    "브뤼셀": {"대륙": "유럽", "국가": "벨기에", "코드": "BRU", "영문명": "brussels"},
    "브뤼헤": {"대륙": "유럽", "국가": "벨기에", "코드": "BRU", "영문명": "bruges"},  # 브뤼셀 경유
    "코펜하겐": {"대륙": "유럽", "국가": "덴마크", "코드": "CPH", "영문명": "copenhagen"},
    "스톡홀름": {"대륙": "유럽", "국가": "스웨덴", "코드": "ARN", "영문명": "stockholm"},
    "오슬로": {"대륙": "유럽", "국가": "노르웨이", "코드": "OSL", "영문명": "oslo"},
    "헬싱키": {"대륙": "유럽", "국가": "핀란드", "코드": "HEL", "영문명": "helsinki"},
    "자그레브": {"대륙": "유럽", "국가": "크로아티아", "코드": "ZAG", "영문명": "zagreb"},
    "두브로브니크": {"대륙": "유럽", "국가": "크로아티아", "코드": "DBV", "영문명": "dubrovnik"},
    "스플리트": {"대륙": "유럽", "국가": "크로아티아", "코드": "SPU", "영문명": "split"},

    # 독일
    "뮌헨": {"대륙": "유럽", "국가": "독일", "코드": "MUC", "영문명": "munich"},
    "베를린": {"대륙": "유럽", "국가": "독일", "코드": "BER", "영문명": "berlin"},
    "프랑크푸르트": {"대륙": "유럽", "국가": "독일", "코드": "FRA", "영문명": "frankfurt"},

    # 그리스·터키
    "아테네": {"대륙": "유럽", "국가": "그리스", "코드": "ATH", "영문명": "athens"},
    "산토리니": {"대륙": "유럽", "국가": "그리스", "코드": "JTR", "영문명": "santorini"},
    "이스탄불": {"대륙": "유럽", "국가": "터키", "코드": "IST", "영문명": "istanbul"},
})

# -------------------------------
# 북미, 오세아니아
# -------------------------------
UNIFIED_CITY_INFO.update({
    "뉴욕": {"대륙": "북미", "국가": "미국", "코드": "JFK", "영문명": "new york city"},
    "로스앤젤레스": {"대륙": "북미", "국가": "미국", "코드": "LAX", "영문명": "los angeles"},
    "시카고": {"대륙": "북미", "국가": "미국", "코드": "ORD", "영문명": "chicago"},
    "하와이": {"대륙": "북미", "국가": "미국", "코드": "HNL", "영문명": "hawaii"},
    "샌프란시스코": {"대륙": "북미", "국가": "미국", "코드": "SFO", "영문명": "san francisco"},
    "라스베이거스": {"대륙": "북미", "국가": "미국", "코드": "LAS", "영문명": "las vegas"},
    "워싱턴 D.C.": {"대륙": "북미", "국가": "미국", "코드": "IAD", "영문명": "washington, d.c."},
    "보스턴": {"대륙": "북미", "국가": "미국", "코드": "BOS", "영문명": "boston"},
    "시애틀": {"대륙": "북미", "국가": "미국", "코드": "SEA", "영문명": "seattle"},

    # 미국 추가
    "마이애미": {"대륙": "북미", "국가": "미국", "코드": "MIA", "영문명": "miami"},
    "올랜도": {"대륙": "북미", "국가": "미국", "코드": "MCO", "영문명": "orlando"},
    "뉴올리언스": {"대륙": "북미", "국가": "미국", "코드": "MSY", "영문명": "new orleans"},

    # 캐나다
    "밴쿠버": {"대륙": "북미", "국가": "캐나다", "코드": "YVR", "영문명": "vancouver"},
    "토론토": {"대륙": "북미", "국가": "캐나다", "코드": "YYZ", "영문명": "toronto"},
    "몬트리올": {"대륙": "북미", "국가": "캐나다", "코드": "YUL", "영문명": "montreal"},
    "캘거리": {"대륙": "북미", "국가": "캐나다", "코드": "YYC", "영문명": "calgary"},

    # 멕시코
    "칸쿤": {"대륙": "북미", "국가": "멕시코", "코드": "CUN", "영문명": "cancun"},
    "멕시코시티": {"대륙": "북미", "국가": "멕시코", "코드": "MEX", "영문명": "mexico city"},
    "시드니": {"대륙": "오세아니아", "국가": "호주", "코드": "SYD", "영문명": "sydney"},

    # 오세아니아
    "멜버른": {"대륙": "오세아니아", "국가": "호주", "코드": "MEL", "영문명": "melbourne"},
    "브리즈번": {"대륙": "오세아니아", "국가": "호주", "코드": "BNE", "영문명": "brisbane"},
    "퍼스": {"대륙": "오세아니아", "국가": "호주", "코드": "PER", "영문명": "perth"},
    "케언즈": {"대륙": "오세아니아", "국가": "호주", "코드": "CNS", "영문명": "cairns"},

    "오클랜드": {"대륙": "오세아니아", "국가": "뉴질랜드", "코드": "AKL", "영문명": "auckland"},
    "퀸스타운": {"대륙": "오세아니아", "국가": "뉴질랜드", "코드": "ZQN", "영문명": "queenstown"},

    # 미크로네시아 (한국 인기 휴양지)
    "괌": {"대륙": "오세아니아", "국가": "괌", "코드": "GUM", "영문명": "guam"},
    "사이판": {"대륙": "오세아니아", "국가": "북마리아나 제도", "코드": "SPN", "영문명": "saipan"},
})


# -------------------------------
# 중동, 아프리카
# -------------------------------
UNIFIED_CITY_INFO.update({
    "두바이": {"대륙": "아시아", "국가": "아랍에미리트", "코드": "DXB", "영문명": "dubai"},
    "아부다비": {"대륙": "아시아", "국가": "아랍에미리트", "코드": "AUH", "영문명": "abu dhabi"},
    "도하": {"대륙": "아시아", "국가": "카타르", "코드": "DOH", "영문명": "doha"},
    "텔아비브": {"대륙": "아시아", "국가": "이스라엘", "코드": "TLV", "영문명": "tel aviv"},
    "예루살렘": {"대륙": "아시아", "국가": "이스라엘", "코드": "TLV", "영문명": "jerusalem"},  # TLV 이용
    "카이로": {"대륙": "아프리카", "국가": "이집트", "코드": "CAI", "영문명": "cairo"},
    "마라케시": {"대륙": "아프리카", "국가": "모로코", "코드": "RAK", "영문명": "marrakech"},
    "카사블랑카": {"대륙": "아프리카", "국가": "모로코", "코드": "CMN", "영문명": "casablanca"},
    "케이프타운": {"대륙": "아프리카", "국가": "남아프리카공화국", "코드": "CPT", "영문명": "cape town"},
    "요하네스버그": {"대륙": "아프리카", "국가": "남아프리카공화국", "코드": "JNB", "영문명": "johannesburg"},
    "나이로비": {"대륙": "아프리카", "국가": "케냐", "코드": "NBO", "영문명": "nairobi"},
})


print(f"✅ UNIFIED_CITY_INFO 최종 업데이트 완료! 총 {len(UNIFIED_CITY_INFO)}개 도시")

# =============================================================================
# 🆕 hashlib 기반 초고속 중복 방지 시스템
# =============================================================================

def get_url_hash(url):
    """URL을 고유한 짧은 해시로 변환 (0.0001초)"""
    hash_length = CONFIG.get("HASH_LENGTH", 12)
    return hashlib.md5(url.encode('utf-8')).hexdigest()[:hash_length]

def is_url_processed_fast(url, city_name):
    """해시 파일 존재 여부로 초고속 중복 체크 (0.001초)"""
    if not CONFIG.get("USE_HASH_SYSTEM", True):
        return False
        
    url_hash = get_url_hash(url)
    hash_file = os.path.join("hash_index", city_name, f"{url_hash}.done")
    return os.path.exists(hash_file)

def mark_url_processed_fast(url, city_name, product_number=None):
    """해시 파일 생성으로 완료 표시 (0.002초)"""
    if not CONFIG.get("USE_HASH_SYSTEM", True):
        return False
        
    url_hash = get_url_hash(url)
    hash_dir = os.path.join("hash_index", city_name)
    os.makedirs(hash_dir, exist_ok=True)
    
    hash_file = os.path.join(hash_dir, f"{url_hash}.done")
    with open(hash_file, 'w', encoding='utf-8') as f:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        f.write(f"URL: {url}\n")
        f.write(f"Product: {product_number}\n")
        f.write(f"Completed: {timestamp}\n")
    
    return True

def get_hash_stats(city_name):
    """해시 시스템 통계 (0.01초)"""
    hash_dir = os.path.join("hash_index", city_name)
    if not os.path.exists(hash_dir):
        return {'total': 0, 'files': [], 'latest': None}
    
    done_files = [f for f in os.listdir(hash_dir) if f.endswith('.done')]
    latest_file = None
    
    if done_files:
        latest_file = max(done_files, 
                         key=lambda x: os.path.getctime(os.path.join(hash_dir, x)))
    
    return {
        'total': len(done_files),
        'files': done_files,
        'latest': latest_file
    }

def migrate_csv_to_hash(city_name):
    """기존 CSV 데이터를 해시 시스템으로 마이그레이션"""
    print(f"🔄 {city_name} CSV → 해시 시스템 마이그레이션...")
    
    try:
        # 기존 CSV에서 URL 읽기
        csv_urls = get_completed_urls_from_csv(city_name)
        
        if not csv_urls:
            print(f"   ℹ️ 마이그레이션할 CSV 데이터가 없습니다")
            return True
        
        # 해시 폴더 생성
        hash_dir = os.path.join("hash_index", city_name)
        os.makedirs(hash_dir, exist_ok=True)
        
        # 각 URL을 해시 파일로 변환
        migrated_count = 0
        for url in csv_urls:
            if not is_url_processed_fast(url, city_name):
                mark_url_processed_fast(url, city_name, "migrated")
                migrated_count += 1
        
        print(f"   ✅ {migrated_count}개 URL 마이그레이션 완료")
        print(f"   📊 해시 시스템 통계: {get_hash_stats(city_name)}")
        return True
        
    except Exception as e:
        print(f"   ❌ 마이그레이션 실패: {e}")
        return False

def hybrid_is_processed(url, city_name):
    """하이브리드 중복 체크: 해시(빠름) + CSV(호환성)"""
    # 1. 초고속 해시 체크 먼저 (0.001초)
    if is_url_processed_fast(url, city_name):
        return True
    
    # 2. 기존 CSV 호환성 체크 (0.1초, 필요시에만)
    if CONFIG.get("KEEP_CSV_SYSTEM", True):
        csv_urls = get_completed_urls_from_csv(city_name)
        if url in csv_urls:
            # CSV에는 있지만 해시에 없으면 해시에도 추가
            mark_url_processed_fast(url, city_name, "csv_sync")
            return True
    
    return False

def ensure_url_directories_v2():
    """V2 3-tier URL 디렉토리 구조 생성"""
    if not CONFIG.get("USE_V2_URL_SYSTEM", True):
        return True
    
    try:
        directories = [
            CONFIG["V2_URL_COLLECTED"],  # url_collected/
            CONFIG["V2_URL_DONE"],       # url_done/
            CONFIG["V2_URL_PROGRESS"]    # url_progress/
        ]
        for directory in directories:
            os.makedirs(directory, exist_ok=True)
        return True
    except Exception as e:
        print(f"❌ V2 URL 디렉토리 생성 실패: {e}")
        return False
    
# =============================================================================
# 🔧 핵심 함수들 - 단일 정보 소스(UNIFIED_CITY_INFO) 사용 + hashlib 통합
# =============================================================================

# 도시별로 작업 내역을 기록할 로그 파일 경로 템플릿
URL_LOG_FILE_TEMPLATE = 'url_done/{city_code}_done.log'

def get_completed_urls_from_csv(city_name):
    """[V2 3-tier + CSV 통합 버전] 완료된 URL을 3-tier 시스템에서 로드합니다."""
    # 1. V2 시스템에서 완료된 URL 로드
    v2_urls = set()
    if CONFIG.get("USE_V2_URL_SYSTEM", True):
        try:
            city_code = get_city_code(city_name)
            done_file = os.path.join(CONFIG["V2_URL_DONE"], f"{city_code}_done.log")
            if os.path.exists(done_file):
                with open(done_file, 'r', encoding='utf-8') as f:
                    v2_urls = set(line.strip() for line in f if line.strip())
        except Exception as e:
            print(f"⚠️ V2 URL 로드 오류: {e}")
    
    # 2. 기존 CSV에서 URL 로드 (호환성)
    csv_urls = set()
    try:
        continent, country = get_city_info(city_name)
        # 도시국가 특별 처리
        if city_name in ["마카오", "홍콩", "싱가포르"]:
            csv_path = os.path.join("data", continent, f"klook_{city_name}_products.csv")
        else:
            csv_path = os.path.join("data", continent, country, city_name, f"klook_{city_name}_products.csv")
        
        if os.path.exists(csv_path):
            import pandas as pd
            df = pd.read_csv(csv_path, encoding='utf-8-sig', usecols=['URL'])
            csv_urls = set(df['URL'].dropna())
    except Exception as e:
        print(f"⚠️ CSV에서 URL을 읽는 중 오류 발생 (무시 가능): {e}")
    
    # 3. V2 + CSV 통합 결과 반환
    combined_urls = v2_urls.union(csv_urls)
    
    # 출력 홍수 방지 - 출력 메시지 제거
    # if CONFIG.get("USE_V2_URL_SYSTEM", True) and len(v2_urls) > 0:
    #     print(f"✅ V2+CSV 통합 로드: V2({len(v2_urls)}) + CSV({len(csv_urls)}) = {len(combined_urls)}개")
    
    return combined_urls
 
def load_session_state(city_name):
    """
    [hashlib 통합 버전] 이전 세션의 상태를 CSV와 해시 시스템을 교차 검증하여 완벽하게 복원합니다.
    """
    print(f"🔄 '{city_name}' 도시의 이전 작업 세션 상태를 불러옵니다...")

    # 1. V2+CSV 통합에서 완료된 URL 목록 가져오기 (가장 확실한 증거)
    completed_urls_from_csv = get_completed_urls_from_csv(city_name)
    print(f"   - 최종 결과물(CSV)에서 {len(completed_urls_from_csv)}개의 완료된 URL을 확인했습니다.")

    # 2. 해시 시스템 통계 확인
    hash_stats = get_hash_stats(city_name)
    print(f"   - 해시 시스템에서 {hash_stats['total']}개의 완료된 URL을 확인했습니다.")

    # 3. 필요시 마이그레이션 수행
    if len(completed_urls_from_csv) > hash_stats['total']:
        print(f"   - CSV → 해시 시스템 자동 마이그레이션 수행 중...")
        migrate_csv_to_hash(city_name)

    # 4. 도시별 로그 파일에서 완료된 URL 목록 가져오기 (호환성)
    city_code = get_city_code(city_name)
    log_file = URL_LOG_FILE_TEMPLATE.format(city_code=city_code)
    completed_urls_from_log = set()
    try:
        with open(log_file, 'r', encoding='utf-8') as f:
            completed_urls_from_log = set(line.strip() for line in f)
        print(f"   - 작업 노트(Log)에서 {len(completed_urls_from_log)}개의 완료된 URL을 확인했습니다.")
    except FileNotFoundError:
        print("   - 이 도시의 작업 노트(Log)가 없습니다. 새로 시작합니다.")

    # 5. 통합 마스터 목록 생성 (해시 시스템이 메인, CSV/Log는 백업)
    master_completed_urls = completed_urls_from_csv.union(completed_urls_from_log)
    print(f"   - 총 {len(master_completed_urls)}개의 고유한 URL을 완료한 것으로 최종 확인됩니다.")

    # 6. 마지막 상품 번호 불러오기 (기존 함수 사용)
    last_number = get_last_product_number(city_name)
    start_number = last_number + 1
    print(f"   - 마지막 상품 번호는 {last_number}번 입니다.")
    print(f"✅ 상태 복원 완료. 크롤링은 {start_number}번부터 시작됩니다.")
    print(f"🚀 해시 시스템 활성화: {CONFIG.get('USE_HASH_SYSTEM', True)}")

    return start_number, master_completed_urls

def save_url_to_log(city_name, url):
    """[V2 3-tier + hashlib 통합 버전] 수집 완료한 URL을 3-tier 시스템에 기록합니다."""
    # 1. V2 3-tier 시스템에 기록 (새로운 방식)
    if CONFIG.get("USE_V2_URL_SYSTEM", True):
        ensure_url_directories_v2()
        city_code = get_city_code(city_name)
        done_file = os.path.join(CONFIG["V2_URL_DONE"], f"{city_code}_done.log")
        with open(done_file, 'a', encoding='utf-8') as f:
            f.write(url + '\n')
    
    # 2. 해시 시스템에 기록 (초고속)
    if CONFIG.get("USE_HASH_SYSTEM", True):
        mark_url_processed_fast(url, city_name)
    
    # 3. 기존 로그 시스템도 유지 (호환성)
    if CONFIG.get("KEEP_CSV_SYSTEM", True):
        os.makedirs("url_cache/completed", exist_ok=True)
        log_file = os.path.join("url_cache", "completed", f"{city_name}_done.log")
        with open(log_file, 'a', encoding='utf-8') as f:
            f.write(url + '\n')

def get_city_code(city_name):
    """도시명으로 공항 코드 반환 (UNIFIED_CITY_INFO 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        code = info.get("코드", city_name[:3].upper())
        return code
    return city_name[:3].upper()

def get_city_info(city_name):
    """통합된 도시 정보 가져오기 (사전 정의된 값만 사용)"""
    info = UNIFIED_CITY_INFO.get(city_name)
    if info:
        return info["대륙"], info["국가"]
    else:
        # 정의되지 않은 도시에 대한 기본값
        return "기타", "기타"

def get_last_product_number(city_name):
    """기존 CSV에서 마지막 번호 확인 (번호 연속성 확보)"""
    try:
        continent, country = get_city_info(city_name)
        
        # 🎯 ========== 도시국가 특별 처리 추가 ==========
        if city_name in ["마카오", "홍콩", "싱가포르"]:
            # 도시국가: 대륙 폴더 바로 아래에서 찾기
            csv_path = os.path.join("data", continent, f"klook_{city_name}_products.csv")
        else:
            # 일반 도시: 기존 구조
            csv_path = os.path.join("data", continent, country, city_name, f"klook_{city_name}_products.csv")
        # ================================================
        
        if os.path.exists(csv_path):
            df = pd.read_csv(csv_path, encoding='utf-8-sig')
            if not df.empty and '번호' in df.columns:
                last_number = df['번호'].max()
                print(f"📊 기존 CSV 마지막 번호: {last_number}")
                return last_number
        
        print(f"📄 기존 CSV 파일 없음 - 0부터 시작")
        return 0   # 파일이 없으면 0 반환 (다음은 1부터 시작)
        
    except Exception as e:
        print(f"⚠️ 마지막 번호 확인 실패: {e}")
        return 0   # 0 반환하여 다음이 1부터 시작

def get_product_name(driver, url_type="Product"):
    """✅ 상품명 수집 (KLOOK 최적화)"""
    print(f"  📊 {url_type} 상품명 수집 중...")

    title_selectors = [
        (By.CSS_SELECTOR, "#activity_title > h1 > span"),    # KLOOK 최우선 (100% 확인됨)
        (By.CSS_SELECTOR, "#activity_title .vam"),           # KLOOK 백업
        (By.CSS_SELECTOR, "#activity_title h1"),             # KLOOK 백업2
        (By.CSS_SELECTOR, "h1"),                             # 범용 백업
    ]

    for selector_type, selector_value in title_selectors:
        try:
            title_element = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            found_name = title_element.text
            return found_name
        except TimeoutException:
            continue
    
    raise NoSuchElementException("상품명을 찾을 수 없습니다")

def get_price(driver, logger=None):
    """✅ 가격 수집 - KLOOK 최적화 버전 (로깅 강화)"""
    log = logger if logger else print
    log("  💰 가격 정보 수집 중...")

    price_selectors = [
        (By.CSS_SELECTOR, "#banner_atlas .salling-price span"),     # 판매가 (최우선)
        (By.CSS_SELECTOR, "#banner_atlas .market-price b"),         # 정가
        (By.CSS_SELECTOR, "#banner_atlas .price-box span"),         # 범용 백업
        (By.CSS_SELECTOR, "span[data-v-7d296880]"),                 # data-v 속성
        (By.CSS_SELECTOR, ".price"),
        (By.CSS_SELECTOR, "[class*='price']"),
        (By.XPATH, "//span[contains(text(), '₩') and string-length(text()) < 30]"),
        (By.XPATH, "//span[contains(text(), '원') and contains(text(), ',') and string-length(text()) < 30]"),
    ]

    invalid_keywords = [
        '쿠폰', '받기', '다운', '할인', '적립', '포인트',
        '최소', '인원', '명', '최대', '선택', '옵션',
        '예약', '신청', '문의', '상담', '확인', '명부터',
        '시간', '일정', '코스', '투어', '여행',
        '언어', '가이드', '포함', '불포함', '이상',
        '취소', '환불', '변경', '안내', '모집'
    ]

    price_patterns = [
        r'₩\s*\d{1,3}(?:,\d{3})+',        # ₩ 35,400
        r'\d{1,3}(?:,\d{3})+원[~-]?',     # 10,000원~
        r'\d+,\d+원[~-]?',                # 간단한 천단위
        r'\d{4,}원[~-]?',                 # 10000원~
    ]

    for selector_type, selector_value in price_selectors:
        try:
            log(f"  🔎 셀렉터 시도: {selector_type} | {selector_value}")
            price_elements = driver.find_elements(selector_type, selector_value)
            if not price_elements:
                log("    ↪ 요소 없음")
                continue

            for idx, price_element in enumerate(price_elements, start=1):
                try:
                    found_price = price_element.text.strip()
                except StaleElementReferenceException as e:
                    log(f"    ⚠️ 요소#{idx} StaleElementReference: {e}")
                    continue
                except WebDriverException as e:
                    log(f"    ⚠️ 요소#{idx} WebDriverException: {e}")
                    continue

                if not found_price:
                    log(f"    ↪ 요소#{idx} 빈 텍스트 -> 스킵")
                    continue

                if any(keyword in found_price for keyword in invalid_keywords):
                    log(f"    ↪ 요소#{idx} 금지 키워드 포함('{found_price}') -> 스킵")
                    continue

                if len(found_price) > 30:
                    log(f"    ↪ 요소#{idx} 길이 초과('{found_price}') -> 스킵")
                    continue

                is_valid_price = any(re.search(pattern, found_price) for pattern in price_patterns)
                if is_valid_price and ('원' in found_price or '₩' in found_price):
                    log(f"    ✅ 유효한 가격 발견: '{found_price}'")
                    return found_price
                else:
                    log(f"    ↪ 요소#{idx} 패턴 불일치('{found_price}') -> 스킵")

        except (NoSuchElementException, TimeoutException) as e:
            log(f"  ❌ 셀렉터 실패: {type(e).__name__} | {selector_value} | {e}")
            continue
        except WebDriverException as e:
            log(f"  ❌ 드라이버 예외: {type(e).__name__} | {e}")
            continue
        except Exception as e:
            log(f"  ❌ 알 수 없는 예외: {type(e).__name__} | {e}")
            continue

    log("    ❌ 가격 정보를 찾을 수 없습니다")
    return "정보 없음"

def clean_price(price_text):
    """✅ 가격 정제 (모든 사이트 통일: 77,900원 형태)"""
    if not price_text or price_text == "정보 없음":
        return "정보 없음"
    
    # 모든 가격 패턴 (₩, 원, $ 등 모두 지원)
    price_patterns = [
        r'₩\s*(\d{1,3}(?:,\d{3})*)',           # ₩ 77,900
        r'\$\s*(\d{1,3}(?:,\d{3})*)',          # $ 100
        r'(\d{1,3}(?:,\d{3})*)\s*원[~-]?',     # 77,900원
        r'(\d{1,3}(?:,\d{3})*)'                # 77900 (숫자만)
    ]
    
    for pattern in price_patterns:
        match = re.search(pattern, price_text)
        if match:
            # 모든 경우에 "77,900원" 형태로 통일
            return f"{match.group(1)}원"
    
    return price_text

def clean_rating(rating_text):
    """✅ 평점 정제 (기존: extract_clean_rating → 새로운: clean_rating) (공용 함수)"""
    if not rating_text or rating_text == "정보 없음":
        return "정보 없음"
    
    rating_pattern = r'(\d+\.?\d*)'
    match = re.search(rating_pattern, rating_text)
    
    if match:
        try:
            return float(match.group(1))
        except ValueError:
            return rating_text
    else:
        return rating_text
    
  # =============================================================================
  # 🚀 V2 3-tier URL 시스템 자동 초기화
  # =============================================================================

# V2 시스템이 활성화되어 있으면 디렉토리 자동 생성
if CONFIG.get("USE_V2_URL_SYSTEM", True):
    ensure_url_directories_v2()
    print(f"🚀 V2 3-tier URL 시스템 초기화 완료!")
    print(f"   📂 수집 대기: {CONFIG['V2_URL_COLLECTED']}/")
    print(f"   📂 완료됨: {CONFIG['V2_URL_DONE']}/")
    print(f"   📂 진행상황: {CONFIG['V2_URL_PROGRESS']}/")

print("✅ 그룹 1 완료: 기본 설정 및 hashlib 통합 시스템 정의 완료!")
print(f"🚀 hashlib 시스템 상태: {'활성화' if CONFIG.get('USE_HASH_SYSTEM', True) else '비활성화'}")
print(f"🔄 CSV 호환성: {'유지' if CONFIG.get('KEEP_CSV_SYSTEM', True) else '비활성화'}")
print(f"📊 해시 길이: {CONFIG.get('HASH_LENGTH', 12)}자리")

In [ ]:
# =============================================================================
# 🚀 그룹 2: 이미지 처리 및 데이터 저장 함수들
# - 이미지 다운로드, 데이터 저장, 결과 처리 관련 함수들
# =============================================================================

def download_image(driver, product_name, city_name, product_number):
    """✅ 메인+썸네일 이미지 다운로드 - KLOOK 2개 이미지 시스템 (계층 구조 저장)"""
    if not CONFIG["SAVE_IMAGES"]:
        return {
            'main_image': {
                'status': '이미지 저장 비활성화',
                'filename': '',
                'path': '',
                'relative_path': '',
                'size': 0
            },
            'thumbnail_image': {
                'status': '이미지 저장 비활성화',
                'filename': '',
                'path': '',
                'relative_path': '',
                'size': 0
            }
        }

    print(f"  🖼️ KLOOK 메인+썸네일 이미지 다운로드 중...")

    # KLOOK 전용 이미지 셀렉터들
    main_image_selectors = [
        (By.CSS_SELECTOR, "#banner_atlas .activity-banner-image-container_left img"),  # KLOOK 메인 이미지
        (By.CSS_SELECTOR, ".activity-banner-image-container_left img"),                 # 백업 1
        (By.CSS_SELECTOR, ".main-image img"),                                           # 일반 백업
        (By.CSS_SELECTOR, ".hero-image img"),                                           # 일반 백업
        (By.CSS_SELECTOR, ".product-image img"),                                        # 일반 백업
    ]

    thumbnail_image_selectors = [
        (By.CSS_SELECTOR, "#banner_atlas .activity-banner-image-container_right img"),  # KLOOK 썸네일 이미지
        (By.CSS_SELECTOR, ".activity-banner-image-container_right img"),                # 백업 1
        (By.CSS_SELECTOR, ".product-gallery img:nth-child(2)"),                        # 일반 백업
        (By.CSS_SELECTOR, ".gallery img:nth-child(2)"),                                # 일반 백업
        (By.CSS_SELECTOR, ".slider img:nth-child(2)"),                                 # 일반 백업
    ]

    # 파일명 준비
    city_code = get_city_code(city_name)
    safe_number = max(1, product_number)
    main_filename = f"{city_code}_{safe_number:04d}.jpg"
    thumb_filename = f"{city_code}_{safe_number:04d}_thumb.jpg"

    # 계층 구조 폴더 경로 설정
    continent, country = get_city_info(city_name)
    base_folder = "klook_thumb_img"

    if city_name in ["마카오", "홍콩", "싱가포르"]:
        hierarchical_folder = os.path.join(base_folder, continent, city_name)
        main_relative_path = os.path.join(continent, city_name, main_filename)
        thumb_relative_path = os.path.join(continent, city_name, thumb_filename)
    else:
        hierarchical_folder = os.path.join(base_folder, continent, country, city_name)
        main_relative_path = os.path.join(continent, country, city_name, main_filename)
        thumb_relative_path = os.path.join(continent, country, city_name, thumb_filename)

    # 폴더 생성
    os.makedirs(hierarchical_folder, exist_ok=True)
    print(f"    📁 계층 폴더 확인: {hierarchical_folder}")

    # 메인 이미지 다운로드
    main_result = download_single_image(
        driver, main_image_selectors, hierarchical_folder,
        main_filename, main_relative_path, "메인"
    )

    # 썸네일 이미지 다운로드
    thumb_result = download_single_image(
        driver, thumbnail_image_selectors, hierarchical_folder,
        thumb_filename, thumb_relative_path, "썸네일"
    )

    return {
        'main_image': main_result,
        'thumbnail_image': thumb_result
    }


def download_single_image(driver, selectors, folder_path, filename, relative_path, image_type):
    """단일 이미지 다운로드 헬퍼 함수"""
    print(f"    🔍 {image_type} 이미지 URL 찾는 중...")

    img_url = None
    for selector_type, selector_value in selectors:
        try:
            img_elements = driver.find_elements(selector_type, selector_value)
            for img_element in img_elements:
                try:
                    img_url = img_element.get_attribute('src')
                    if not img_url or not img_url.startswith('http'):
                        continue

                    # 제외할 패턴들
                    exclude_patterns = ['logo', 'icon', 'banner', 'ad', 'avatar', 'profile', 'button', 'arrow', 'star', 'small', 'mini']
                    if any(pattern in img_url.lower() for pattern in exclude_patterns):
                        continue

                    # 이미지 크기 확인
                    try:
                        size = img_element.size
                        if size and (size.get('width', 0) < 100 or size.get('height', 0) < 100):
                            continue
                    except Exception:
                        pass

                    print(f"      ✅ {image_type} URL 발견: {img_url[:50]}...")
                    break
                except Exception:
                    continue
            if img_url:
                break
        except Exception:
            continue

    if not img_url:
        print(f"      ❌ {image_type} 이미지 URL을 찾을 수 없습니다")
        return {
            'status': f'{image_type} URL 없음',
            'filename': filename,
            'path': '',
            'relative_path': '',
            'size': 0
        }

    # 이미지 다운로드 시도
    try:
        full_path = os.path.join(folder_path, filename)

        # User-Agent 추가로 다운로드 성공률 높이기
        req = urllib.request.Request(
            img_url,
            headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
        )

        with urllib.request.urlopen(req, timeout=10) as response:
            with open(full_path, 'wb') as f:
                f.write(response.read())

        file_size = os.path.getsize(full_path)

        if file_size < 1024:
            os.remove(full_path)
            print(f"      ❌ {image_type} 파일이 너무 작습니다 ({file_size} bytes)")
            return {
                'status': f'{image_type} 파일 너무 작음',
                'filename': filename,
                'path': '',
                'relative_path': '',
                'size': 0
            }

        print(f"      ✅ {image_type} 이미지 저장 완료! ({file_size:,} bytes)")
        print(f"      📍 저장 위치: {relative_path}")

        return {
            'status': f'{image_type} 다운로드 완료',
            'filename': filename,
            'path': full_path,
            'relative_path': relative_path,
            'size': file_size
        }

    except Exception as e:
        print(f"      ⚠️ {image_type} 이미지 다운로드 실패: {type(e).__name__}: {e}")
        return {
            'status': f'{image_type} 다운로드 실패: {type(e).__name__}',
            'filename': filename,
            'path': '',
            'relative_path': '',
            'size': 0
        }


def save_results(products_data):
    """✅ 데이터 저장 (도시ID는 이미 포함됨)"""
    print("💾 하이브리드 구조로 데이터 저장 중...")
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    
    city_name = products_data[0]['도시'] if products_data else 'unknown'
    
    # DataFrame 생성 (도시ID는 이미 crawl_single_product_optimized에서 생성됨)
    df = pd.DataFrame(products_data)
    
    csv_path = f"klook_{city_name}_products_{len(products_data)}개_{timestamp}.csv"
    df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    
    print(f"📁 개별 CSV 저장 완료: {csv_path}")
    
    # 도시ID 존재 확인 및 출력
    if '도시ID' in df.columns and not df['도시ID'].empty:
        first_id = df['도시ID'].iloc[0]
        last_id = df['도시ID'].iloc[-1]
        print(f"🆔 도시ID 확인: {first_id} ~ {last_id}")
    else:
        print("⚠️ 도시ID 컬럼이 없습니다 - crawl_single_product_optimized 확인 필요")
    
    city_code = get_city_code(city_name)
    metadata = {
        "klook": {
            "last_update": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
            "product_count": len(products_data),
            "status": "success",
            "csv_path": csv_path,
            "city": city_name,
            "city_id_pattern": f"{city_code}_X"
        }
    }
    
    try:
        os.makedirs('config', exist_ok=True)
        with open('config/data_metadata.json', 'w', encoding='utf-8') as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)
        print(f"📁 메타데이터 저장 완료: config/data_metadata.json")
    except Exception as e:
        print(f"⚠️ 메타데이터 저장 실패: {e}")
    
    return csv_path

def safe_csv_write(file_path, df, mode='w', header=True):
    """CSV 파일을 안전하게 작성 (Permission denied 오류 해결)"""
    max_retries = 5
    
    for attempt in range(max_retries):
        try:
            # 기존 파일이 있고 쓰기 모드인 경우 백업 생성
            if mode == 'a' and os.path.exists(file_path):
                # 파일이 잠겨있는지 확인
                try:
                    with open(file_path, 'a', encoding='utf-8-sig') as test_file:
                        test_file.write('')  # 빈 문자열 쓰기 테스트
                except PermissionError:
                    print(f"    ⚠️ 파일이 잠겨있음, {attempt+1}번째 재시도...")
                    time.sleep(2)  # 2초 대기
                    continue
            
            # CSV 파일 작성
            df.to_csv(file_path, mode=mode, header=header, index=False, encoding='utf-8-sig')
            return True
            
        except PermissionError as e:
            print(f"    ⚠️ 권한 오류 ({attempt+1}/{max_retries}): {e}")
            if attempt < max_retries - 1:
                # 잠깐 대기 후 재시도
                wait_time = (attempt + 1) * 2  # 2, 4, 6, 8초
                print(f"    ⏰ {wait_time}초 대기 후 재시도...")
                time.sleep(wait_time)
            else:
                # 최종 시도 - 백업 파일로 저장
                backup_path = file_path.replace('.csv', f'_backup_{datetime.now().strftime("%H%M%S")}.csv')
                try:
                    df.to_csv(backup_path, mode='w', header=True, index=False, encoding='utf-8-sig')
                    print(f"    💾 백업 파일로 저장: {backup_path}")
                    return True
                except Exception as backup_error:
                    print(f"    ❌ 백업 저장도 실패: {backup_error}")
                    return False
                    
        except Exception as e:
            print(f"    ❌ 예상치 못한 오류: {e}")
            return False
    
    return False

def save_batch_data(batch_results, city_name):
    """배치 데이터 저장 (도시ID + 국가별 연속번호 개선 버전)"""
    if not batch_results:
        return None
    
    try:
        # =================================================================
        # 👑 (핵심 오류 수정) city_code 변수를 함수 시작 시 정의합니다.
        # =================================================================
        city_code = get_city_code(city_name)
        # =================================================================

        continent, country = get_city_info(city_name)
        
        df = pd.DataFrame(batch_results)
        
        if '도시ID' not in df.columns or df['도시ID'].empty:
            df['도시ID'] = [f"{city_code}_{i}" for i in range(1, len(df) + 1)]
            print(f"✅ 도시ID 컬럼 추가: {city_code}_1 ~ {city_code}_{len(df)}")
        else:
            first_id = df['도시ID'].iloc[0]
            last_id = df['도시ID'].iloc[-1] 
            print(f"✅ 도시ID 확인: {first_id} ~ {last_id}")
        
        if '번호' not in df.columns:
            df['번호'] = range(1, len(df) + 1)
            print(f"✅ 번호 컬럼 추가: 1 ~ {len(df)}")
        
        if city_name in ["마카오", "홍콩", "싱가포르"]:
            data_dir = os.path.join("data", continent)
            os.makedirs(data_dir, exist_ok=True)
            city_csv = os.path.join(data_dir, f"klook_{city_name}_products.csv")
            
            if os.path.exists(city_csv):
                city_success = safe_csv_write(city_csv, df, mode='a', header=False)
            else:
                city_success = safe_csv_write(city_csv, df, mode='w', header=True)
            
            if city_success:
                print(f"✅ 도시국가 데이터 저장 완료: {city_csv}")
                return {
                    "city_csv": city_csv,
                    "country_csv": "도시국가라서 불필요",
                    "data_count": len(batch_results)
                }
            else:
                print(f"❌ 도시국가 데이터 저장 실패")
                return None

        data_dir = os.path.join("data", continent, country, city_name)
        os.makedirs(data_dir, exist_ok=True)

        city_csv = os.path.join(data_dir, f"klook_{city_name}_products.csv")
        if os.path.exists(city_csv):
            city_success = safe_csv_write(city_csv, df, mode='a', header=False)
        else:
            city_success = safe_csv_write(city_csv, df, mode='w', header=True)

        country_dir = os.path.join("data", continent, country)
        os.makedirs(country_dir, exist_ok=True)
        country_csv = os.path.join(country_dir, f"{country}_klook_products_all.csv")
        
        country_df = df.copy()
        
        if os.path.exists(country_csv):
            existing_df = pd.read_csv(country_csv, encoding='utf-8-sig')
            if not existing_df.empty and '번호' in existing_df.columns:
                last_number = existing_df['번호'].max()
                country_df['번호'] = list(range(int(last_number) + 1, int(last_number) + 1 + len(country_df)))
                print(f"🔗 국가별 연속번호: {last_number + 1} ~ {last_number + len(country_df)}")
            country_success = safe_csv_write(country_csv, country_df, mode='a', header=False)
        else:
            country_df['번호'] = range(1, len(country_df) + 1)
            print(f"🆕 국가별 신규파일: 1 ~ {len(country_df)}")
            country_success = safe_csv_write(country_csv, country_df, mode='w', header=True)

        if city_success and country_success:
            print(f"✅ 배치 데이터 저장 완료:")
            print(f"   📁 도시별: {city_csv}")
            print(f"   📁 국가별: {country_csv}")
            print(f"   🆔 도시ID 패턴: {city_code}_X")
            print(f"   🔢 국가별 번호: 연속 처리됨")
            
            return {
                "city_csv": city_csv,
                "country_csv": country_csv,
                "data_count": len(batch_results),
                "city_id_pattern": f"{city_code}_X",
                "country_numbering": "연속"
            }
        else:
            print(f"⚠️ 일부 파일 저장 실패 (도시:{city_success}, 국가:{country_success})")
            return {
                "city_csv": city_csv if city_success else "저장실패",
                "country_csv": country_csv if country_success else "저장실패",
                "data_count": len(batch_results)
            }
        
    except Exception as e:
        print(f"❌ 배치 데이터 저장 실패: {e}")
        return None

def get_categories(driver, logger=None):
    """✅ 카테고리 정보 수집 - KLOOK 브레드크럼 최적화 버전"""
    log = logger if logger else print
    log("  🏷️ 카테고리 정보 수집 중...")

    category_selectors = [
        (By.CSS_SELECTOR, "#breadCrumb .klk-breadcrumb-item-inner"),        # KLOOK 최우선
        (By.CSS_SELECTOR, "#breadCrumb a"),                                 # 범용 백업
        (By.CSS_SELECTOR, ".klk-breadcrumb-item-inner"),                    # 클래스 기반 백업
        (By.XPATH, "//nav//a[contains(@class, 'breadcrumb')]"),             # 범용 백업
    ]

    for selector_type, selector_value in category_selectors:
        try:
            log(f"  🔎 카테고리 셀렉터 시도: {selector_value}")
            category_elements = driver.find_elements(selector_type, selector_value)

            if category_elements:
                categories = []
                for element in category_elements:
                    category_text = element.text.strip()
                    # "Klook Travel" 제외 및 빈 텍스트 필터링
                    if category_text and category_text != "Klook Travel" and len(category_text) > 0:
                        categories.append(category_text)

                if categories:
                    # " > "로 구분된 카테고리 경로 생성
                    category_path = " > ".join(categories)
                    log(f"    ✅ 카테고리 경로: {category_path}")
                    return category_path

        except Exception as e:
            log(f"  ❌ 카테고리 셀렉터 실패: {selector_value} | {e}")
            continue

    log("    ❌ 카테고리 정보를 찾을 수 없습니다")
    return "정보 없음"


from selenium.webdriver.common.by import By
import time

def get_highlights(driver, logger=None):
    """✅ 상품 하이라이트/요약 정보 수집 - KLOOK 토글 모달 최적화 버전"""
    log = logger if logger else print
    log("  ✨ 상품 하이라이트 정보 수집 중...")

    try:
        # 1단계: 펼치기 버튼 찾기
        expand_selectors = [
            (By.CSS_SELECTOR, "#highlight .experience-view-more_text"),
            (By.CSS_SELECTOR, "#highlight .experience-view-more"),
            (By.CSS_SELECTOR, ".experience-view-more_text"),
        ]

        expand_button = None
        for selector_type, selector_value in expand_selectors:
            try:
                expand_button = driver.find_element(selector_type, selector_value)
                if expand_button and expand_button.is_displayed():
                    log(f"  🔎 펼치기 버튼 발견: {selector_value}")
                    break
            except:
                continue

        if not expand_button:
            log("  ℹ️ 펼치기 버튼을 찾을 수 없습니다")
            return "정보 없음"

        # 2단계: 펼치기 버튼 클릭
        driver.execute_script("arguments[0].click();", expand_button)
        time.sleep(1)  # 모달 로딩 대기

        # 3단계: 모달에서 상세 내용 수집
        content_selectors = [
            (By.CSS_SELECTOR, ".klk-modal-body .klk-markdown"),
            (By.CSS_SELECTOR, ".klk-modal-body ul"),
            (By.CSS_SELECTOR, ".klk-modal-body"),
        ]

        for selector_type, selector_value in content_selectors:
            try:
                content_element = driver.find_element(selector_type, selector_value)
                highlight_text = content_element.text.strip()

                if highlight_text and len(highlight_text) > 10:
                    log(f"    ✅ 하이라이트 내용 수집: {len(highlight_text)}자")

                    # 4단계: 모달 닫기
                    close_button = driver.find_element(By.CSS_SELECTOR, ".klk-modal-wrapper i")
                    driver.execute_script("arguments[0].click();", close_button)
                    time.sleep(0.5)

                    return highlight_text[:500] + "..." if len(highlight_text) > 500 else highlight_text

            except Exception:
                continue

        log("    ❌ 상세 내용을 찾을 수 없습니다")
        return "정보 없음"

    except Exception as e:
        log(f"    ❌ 하이라이트 수집 실패: {e}")
        return "정보 없음"


def get_rating(driver, logger=None):
    """✅ 평점 수집 - KLOOK 최적화 버전"""
    log = logger if logger else print
    log("  ⭐ 평점 정보 수집 중...")

    rating_selectors = [
        (By.CSS_SELECTOR, "#score_participants .review-rating__avg"),    # 최우선 (텍스트 평점)
        (By.CSS_SELECTOR, "#score_participants .score-slot-box"),        # 별+점수 형태
        (By.CSS_SELECTOR, "#score_participants .review-rating"),         # 전체 영역 백업
        (By.CSS_SELECTOR, ".review-rating__avg"),                        # 범용 백업
        (By.CSS_SELECTOR, ".score-slot-box"),                            # 범용 백업
    ]

    for selector_type, selector_value in rating_selectors:
        try:
            log(f"  🔎 평점 셀렉터 시도: {selector_value}")
            rating_element = driver.find_element(selector_type, selector_value)
            found_rating = rating_element.text.strip()

            if found_rating and any(char.isdigit() for char in found_rating):
                # 평점 숫자만 추출 (예: "4.8/5" → "4.8")
                rating_match = re.search(r'(\d+\.?\d*)', found_rating)
                if rating_match:
                    rating_value = rating_match.group(1)
                    log(f"    ✅ 유효한 평점 발견: '{rating_value}'")
                    return rating_value

        except Exception as e:
            log(f"  ❌ 평점 셀렉터 실패: {selector_value} | {e}")
            continue

    log("    ❌ 평점 정보를 찾을 수 없습니다")
    return "정보 없음"

def get_review_count(driver, logger=None):
    """✅ 리뷰수 수집 - KLOOK 최적화 버전"""
    log = logger if logger else print
    log("  📝 리뷰수 정보 수집 중...")

    review_count_selectors = [
        (By.CSS_SELECTOR, "#score_participants .review-count span"),      # 최우선 (일반 형태)
        (By.CSS_SELECTOR, "#score_participants .text-underline"),         # 언더라인 형태
        (By.CSS_SELECTOR, "#score_participants .review-box span"),        # 백업
        (By.XPATH, "//span[contains(text(), '이용후기') and contains(text(), '건')]"),  # 텍스트 기반
    ]

    for selector_type, selector_value in review_count_selectors:
        try:
            log(f"  🔎 리뷰수 셀렉터 시도: {selector_value}")
            review_element = driver.find_element(selector_type, selector_value)
            review_text = review_element.text.strip()

            if review_text and '이용후기' in review_text and '건' in review_text:
                # 평점 정보가 중복 포함된 경우 제거 (8번째 케이스 대응)
                if '<div' in review_text or '/5' in review_text:
                    # "이용후기 14.3K건" 부분만 추출
                    clean_review = re.search(r'이용후기\s*[\d.K+]+건', review_text)
                    if clean_review:
                        review_text = clean_review.group(0)

                log(f"    ✅ 유효한 리뷰수 발견: '{review_text}'")
                return review_text

        except Exception as e:
            log(f"  ❌ 리뷰수 셀렉터 실패: {selector_value} | {e}")
            continue

    log("    ❌ 리뷰수 정보를 찾을 수 없습니다")
    return "정보 없음"


def get_language(driver, logger=None):
    """✅ 언어 정보 수집 - KLOOK 최적화 버전"""
    log = logger if logger else print
    log("  🌐 언어 정보 수집 중...")

    language_selectors = [
        (By.CSS_SELECTOR, "#activity_attribute_tags .js-tag-content-node"),        # KLOOK 최우선
        (By.CSS_SELECTOR, "#activity_attribute_tags span.js-tag-content-node"),    # 구체적 버전
        (By.CSS_SELECTOR, "#activity_attribute_tags [data-v-4b7be482]"),           # data 속성 기반
        (By.CSS_SELECTOR, "#activity_attribute_tags span span"),                   # 범용 백업
        # 기존 백업 셀렉터들도 유지
        (By.XPATH, "//dt[contains(text(), '언어')]/following-sibling::dd"),
        (By.XPATH, "//span[contains(@class, 'language')]"),
        (By.XPATH, "//span[contains(text(), '한국어') and string-length(text()) < 50]"),
        (By.XPATH, "//span[contains(text(), '영어') and string-length(text()) < 50]"),
    ]

    for selector_type, selector_value in language_selectors:
        try:
            log(f"  🔎 언어 셀렉터 시도: {selector_value}")
            language_element = driver.find_element(selector_type, selector_value)
            language_text = language_element.text.strip()

            if language_text and len(language_text) > 0:
                # 언어 정보 검증
                language_keywords = ['어', '语', 'English', 'Chinese', 'Korean', 'Japanese', 'Thai']
                if any(keyword in language_text for keyword in language_keywords):
                    log(f"    ✅ 언어 정보 발견: '{language_text}'")
                    return language_text

        except Exception as e:
            log(f"  ❌ 언어 셀렉터 실패: {selector_value} | {e}")
            continue

    log("    ❌ 언어 정보를 찾을 수 없습니다")
    return "정보 없음"


print("✅ 그룹 2 완료: 이미지 처리 및 데이터 저장 함수들 정의 완료!")

In [ ]:
# =============================================================================
# 🔄 그룹 3: 간소화된 상태 관리 시스템 (hashlib 통합)
# - hashlib 시스템과 통합된 단순한 상태 관리
# - 복잡한 기존 시스템을 hashlib로 대체
# =============================================================================

def ensure_config_directory():
    """config 디렉토리 안정성 확보 (단순화된 버전)"""
    config_dir = os.path.join(os.getcwd(), "config")
    os.makedirs(config_dir, exist_ok=True)
    return config_dir

def load_crawler_state():
    """크롤링 상태 로드 (hashlib 통합 버전)"""
    config_dir = ensure_config_directory()
    state_file = os.path.join(config_dir, "crawler_meta.json")

    # 기본 상태
    default_state = {
        "total_collected_count": 0,
        "last_crawled_page": 1,
        "current_session_start": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "hash_system_enabled": CONFIG.get("USE_HASH_SYSTEM", True)
    }

    # 상태 파일 로드
    if os.path.exists(state_file):
        try:
            with open(state_file, 'r', encoding='utf-8') as f:
                state = json.load(f)
            print(f"✅ 상태 파일 로드: {state['total_collected_count']}개 수집완료")
        except Exception as e:
            print(f"⚠️ 상태 파일 로드 실패: {e}, 기본값 사용")
            state = default_state
    else:
        state = default_state
        print("🆕 새로운 크롤링 세션 시작")

    # 🆕 hashlib 시스템이 활성화된 경우 복잡한 URL 로드 과정 생략
    if CONFIG.get("USE_HASH_SYSTEM", True):
        print("🚀 hashlib 시스템 활성화 - 빠른 상태 로드")
        completed_urls = set()  # hashlib 시스템에서 자동 처리
    else:
        # 기존 호환성을 위한 URL 목록 로드
        completed_urls = get_completed_urls_from_csv(CITIES_TO_SEARCH[0]) if CITIES_TO_SEARCH else set()
        print(f"✅ 완료된 URL {len(completed_urls)}개 로드")

    return state, completed_urls

def save_crawler_state(state, new_url=None):
    """크롤링 상태 저장 (hashlib 통합 버전)"""
    config_dir = ensure_config_directory()
    state_file = os.path.join(config_dir, "crawler_meta.json")

    # 상태 업데이트
    state["last_updated"] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    state["hash_system_enabled"] = CONFIG.get("USE_HASH_SYSTEM", True)

    try:
        # 상태 파일 저장
        with open(state_file, 'w', encoding='utf-8') as f:
            json.dump(state, f, ensure_ascii=False, indent=2)

        # 🆕 hashlib 시스템 활성화 시 URL 기록은 save_url_to_log에서 처리
        # 별도 처리 불필요

        return True
    except Exception as e:
        print(f"❌ 상태 저장 실패: {e}")
        return False

def collect_product_urls_from_page(driver, use_infinite_scroll=False):
    """
    🎯 [스위치 기반 적응형] URL 수집 - 기본모드/스크롤모드 선택 가능
    """
    if use_infinite_scroll:
        print("📊 [스크롤모드] 무한스크롤 + 자연스러운 스크롤로 URL 수집...")
        return collect_with_infinite_scroll(driver)
    else:
        print("📊 [기본모드] 현재 화면 단일 스캔으로 URL 수집...")
        return collect_with_single_scan(driver)

def collect_with_single_scan(driver):
    """기본모드: 현재 화면만 1회 스캔 (기존 방식)"""
    print("   🔍 단일 스캔 모드로 URL 수집 중...")

    # 랜덤 스크롤 횟수 (3~5회)
    scroll_count = random.randint(3, 5)
    print(f"      🎯 자연스러운 스크롤 패턴으로 페이지 탐색 중... ({scroll_count}회)")

    time.sleep(random.uniform(1, 2))  # 페이지 로드 후 잠시 머물기

    for i in range(scroll_count):
        smart_scroll_selector(driver)  # 조용히 실행
        time.sleep(random.uniform(0.8, 2.0))  # 스크롤 간 대기

    thinking_time = random.uniform(2, 4)
    print(f"      🤔 {thinking_time:.1f}초 선택 고민 중...")
    time.sleep(thinking_time)

    # 🔧 KLOOK 전용 셀렉터로 수정
    all_selectors = [
        "a[href*='/activity/']",  # KLOOK 상품 페이지 패턴
        ".product-item a",
        ".experience-card a",
        ".product-gallery a",
        ".search-result-list a",  # KLOOK 검색 결과
        ".product-card a"  # KLOOK 상품 카드
    ]

    collected_urls = []

    # URL 수집 (기존 코드)
    for selector in all_selectors:
        try:
            from selenium.webdriver.common.by import By
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    url = element.get_attribute('href')
                    if url:
                        collected_urls.append(url)
                except Exception:
                    continue
        except Exception:
            continue

    # 중복 제거 및 유효성 검증 (KLOOK 패턴으로 수정)
    seen = set()
    unique_and_valid_urls = []
    for url in collected_urls:
        if url not in seen:
            seen.add(url)
            import re
            # KLOOK activity URL 패턴 검증
            if re.search(r'/activity/\d+', url):
                unique_and_valid_urls.append(url)

    print(f"   ✅ 단일 스캔 완료: {len(unique_and_valid_urls)}개 URL 수집")

    # URL 타입별 통계 (KLOOK 패턴)
    activity_count = sum(1 for url in unique_and_valid_urls if '/activity/' in url)

    print(f"      🎯 KLOOK Activities: {activity_count}개")

    return unique_and_valid_urls

def collect_with_infinite_scroll(driver):
    """스크롤모드: 무한스크롤 + 자연스러운 스크롤 패턴"""
    print("   🌊 무한스크롤 시작...")

    all_urls = []
    scroll_attempts = 0
    max_scrolls = 5

    while scroll_attempts < max_scrolls:
        # 1. 현재 화면의 URL 수집
        current_urls = collect_basic_urls_from_current_view(driver)
        new_urls = [url for url in current_urls if url not in all_urls]
        all_urls.extend(new_urls)

        if new_urls:
            print(f"      📊 스크롤 {scroll_attempts+1}: {len(new_urls)}개 신규 URL 발견")

        # 2. 🎯 자연스러운 스크롤 패턴 적용 (봇 회피 핵심!)
        smart_scroll_selector(driver)

        # 3. 스크롤 후 로딩 대기
        time.sleep(random.uniform(3, 5))

        # 4. 더 이상 새로운 URL이 없으면 종료
        if not new_urls and scroll_attempts > 1:
            print(f"      🏁 새로운 상품 없음 - 스크롤 종료")
            break

        scroll_attempts += 1

    # 중복 제거 및 유효성 검증 (KLOOK 패턴으로 수정)
    seen = set()
    unique_and_valid_urls = []
    for url in all_urls:
        if url not in seen:
            seen.add(url)
            # KLOOK activity URL 패턴 검증
            import re
            if re.search(r'/activity/\d+', url):
                unique_and_valid_urls.append(url)

    print(f"   ✅ 무한스크롤 완료: {len(unique_and_valid_urls)}개 고유 URL 수집")

    # URL 타입별 통계 (KLOOK 패턴)
    activity_count = sum(1 for url in unique_and_valid_urls if '/activity/' in url)

    print(f"      🎯 KLOOK Activities: {activity_count}개")

    return unique_and_valid_urls

def collect_basic_urls_from_current_view(driver):
    """현재 화면에서만 URL 수집 (스크롤 없음) - 헬퍼 함수"""
    # 🔧 KLOOK 전용 셀렉터로 수정
    all_selectors = [
        "a[href*='/activity/']",  # KLOOK 상품 페이지 패턴
        ".product-item a",
        ".experience-card a",
        ".product-gallery a",
        ".search-result-list a",  # KLOOK 검색 결과
        ".product-card a"  # KLOOK 상품 카드
    ]

    collected_urls = []
    for selector in all_selectors:
        try:
            from selenium.webdriver.common.by import By
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            for element in elements:
                try:
                    url = element.get_attribute('href')
                    if url:
                        collected_urls.append(url)
                except Exception:
                    continue
        except Exception:
            continue

    return collected_urls

# =============================================================================
# 🔗 간소화된 URL 관리 시스템 (hashlib 통합)
# =============================================================================

def collect_urls_with_csv_safety(driver, city_name, use_infinite_scroll=False):
    """KLOOK sitemap 기반 URL 수집 시스템"""
    print(f"🗺️ KLOOK sitemap 기반 URL 수집 시작... (도시: {city_name})")

    try:
        # KLOOK sitemap에서 URL 수집
        collected_urls = fetch_klook_urls_from_sitemap(city_name)

        if not collected_urls:
            print(f"❌ {city_name}에 대한 KLOOK 상품 URL을 찾을 수 없습니다")
            return []

        # 🚀 hashlib 시스템으로 중복 필터링 (초고속)
        if CONFIG.get("USE_HASH_SYSTEM", True):
            new_urls = []
            for url in collected_urls:
                if not hybrid_is_processed(url, city_name):  # 하이브리드 체크 사용
                    new_urls.append(url)

            print(f"🚀 sitemap 기반 중복 필터링:")
            print(f"   📊 sitemap URL: {len(collected_urls)}개")
            print(f"   🆕 새로운 URL: {len(new_urls)}개")

            return new_urls
        else:
            # 기존 CSV 기반 방식 (호환성)
            return filter_new_urls_from_csv(collected_urls, city_name)

    except Exception as e:
        print(f"❌ KLOOK sitemap URL 수집 실패: {e}")
        return []


def fetch_klook_urls_from_sitemap(city_name):
    """KLOOK sitemap에서 특정 도시의 URL 추출"""
    import requests
    import xml.etree.ElementTree as ET

    print(f"🗺️ KLOOK sitemap에서 '{city_name}' 상품 검색 중...")

    try:
        # 1. KLOOK 마스터 sitemap 접근
        master_url = "https://www.klook.com/ko/sitemap-master-index.xml"
        response = requests.get(master_url, timeout=15)

        if response.status_code != 200:
            print(f"❌ sitemap 접근 실패: HTTP {response.status_code}")
            return []

        root = ET.fromstring(response.content)
        print(f"✅ 마스터 sitemap 접근 성공")

        # 2. experiences-activity sitemap 찾기 (KLOOK 주요 상품)
        target_sitemaps = []
        for sitemap in root.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}sitemap'):
            loc_elem = sitemap.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
            if loc_elem is not None:
                sitemap_url = loc_elem.text
                if 'experiences-activity' in sitemap_url:
                    target_sitemaps.append(sitemap_url)

        print(f"🎯 대상 sitemap {len(target_sitemaps)}개 발견")

        # 3. 도시 키워드 매핑 (그룹 1 함수 활용)
        city_code = get_city_code(city_name)
        continent, country = get_city_info(city_name)
        keywords = [
            city_name.lower(),    # "서울"
            'seoul',              # 정확한 영문명 추가
            country.lower(),      # "대한민국"
            'korea'               # 정확한 영문 국가명 추가
            # city_code 제거 - 너무 모호함
        ]
        print(f"🔍 검색 키워드: {keywords} (from {continent}/{country}/{city_code})")

        # 4. sitemap에서 도시별 URL 추출
        city_urls = []
        for sitemap_url in target_sitemaps[:2]:  # 처음 2개 sitemap만 처리 (속도 최적화)
            try:
                print(f"   📄 sitemap 분석 중: {sitemap_url.split('/')[-1]}")
                sub_response = requests.get(sitemap_url, timeout=15)
                sub_root = ET.fromstring(sub_response.content)

                previous_count = len(city_urls)  # 이전 개수 저장
                
                for url in sub_root.findall('.//{http://www.sitemaps.org/schemas/sitemap/0.9}url'):
                    loc = url.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc')
                    if loc is not None:
                        url_text = loc.text.lower()
                        # 키워드 매칭 및 activity URL 필터링
                        if any(keyword in url_text for keyword in keywords) and '/activity/' in url_text:
                            city_urls.append(loc.text)

                new_count = len(city_urls) - previous_count
                print(f"   📊 {new_count}개 URL 추출됨")

            except Exception as sub_e:
                print(f"   ⚠️ sitemap 처리 실패: {sub_e}")
                continue

        # 5. 중복 제거 및 결과 반환
        unique_urls = list(set(city_urls))
        print(f"🎉 총 {len(unique_urls)}개의 고유한 {city_name} 상품 URL 발견!")

        # 샘플 URL 출력
        if unique_urls:
            print(f"📋 샘플 URL:")
            for i, url in enumerate(unique_urls[:3], 1):
                print(f"   {i}. {url}")

        return unique_urls

    except Exception as e:
        print(f"❌ sitemap 처리 중 오류: {e}")
        return []

def save_collected_urls(city_name, urls_list):
    """
    [V2 3-tier 통합 버전] URL 저장 시스템
    수집된 URL 목록을 JSON 파일로 저장합니다.
    """
    # V2 시스템 우선 사용
    if CONFIG.get("USE_V2_URL_SYSTEM", True):
        try:
            # V2 시스템을 위한 디렉토리 구조를 확인하고 생성합니다.
            ensure_url_directories_v2()
            # 도시 이름으로 도시 코드를 얻습니다.
            city_code = get_city_code(city_name)
            # 저장할 파일 경로를 정의합니다.
            collect_file = os.path.join(CONFIG["V2_URL_COLLECTED"], f"{city_code}_collect.json")

            # 저장할 데이터를 딕셔너리 형태로 구성합니다.
            cache_data = {
                "city": city_name,
                "city_code": city_code,
                "urls": urls_list,
                "collected_time": datetime.now().isoformat(),
                "total_count": len(urls_list),
                "v2_system": True
            }

            # 파일에 데이터를 JSON 형식으로 저장합니다.
            with open(collect_file, 'w', encoding='utf-8') as f:
                json.dump(cache_data, f, ensure_ascii=False, indent=2)

            print(f"✅ V2 URL 캐시 저장: {len(urls_list)}개 ({collect_file})")
            return True
        except Exception as e:
            # V2 시스템에 문제가 발생하면 오류를 출력하고 False를 반환합니다.
            print(f"❌ V2 URL 저장 실패, 기존 방식 사용: {e}")

    # 기존 시스템 (fallback)
    try:
        # 기존 시스템을 위한 디렉토리를 생성합니다.
        os.makedirs("url_cache/collected", exist_ok=True)
        # 저장할 파일 경로를 정의합니다.
        cache_file = os.path.join("url_cache", "collected", f"{city_name}_urls.json")

        # 저장할 데이터를 딕셔너리 형태로 구성합니다.
        cache_data = {
            "city": city_name,
            "urls": urls_list,
            "collected_time": datetime.now().isoformat(),
            "total_count": len(urls_list),
            "hashlib_enabled": CONFIG.get("USE_HASH_SYSTEM", True)
        }

        # 파일에 데이터를 JSON 형식으로 저장합니다.
        with open(cache_file, 'w', encoding='utf-8') as f:
            json.dump(cache_data, f, ensure_ascii=False, indent=2)

        print(f"✅ 기존 URL 캐시 저장: {len(urls_list)}개 ({cache_file})")
        return True

    except Exception as e:
        # 기존 시스템에 문제가 발생하면 오류를 출력하고 False를 반환합니다.
        print(f"❌ URL 캐시 저장 실패: {e}")
        return False

def load_collected_urls(city_name):
    """
    [V2 3-tier 통합 버전] URL 로드 시스템
    JSON 파일에 저장된 URL 목록을 로드하고, 미완료된 URL만 반환합니다.
    """
    # V2 시스템 우선 사용
    if CONFIG.get("USE_V2_URL_SYSTEM", True):
        try:
            city_code = get_city_code(city_name)
            collect_file = os.path.join(CONFIG["V2_URL_COLLECTED"], f"{city_code}_collect.json")

            if os.path.exists(collect_file):
                with open(collect_file, 'r', encoding='utf-8') as f:
                    cache_data = json.load(f)

                cached_urls = cache_data.get('urls', [])
                collected_time = cache_data.get('collected_time', '')

                print(f"✅ V2 URL 캐시 로드: {len(cached_urls)}개 ({collected_time})")

                # 완료된 URL 필터링 (V2 + hashlib 하이브리드)
                pending_urls = []
                for url in cached_urls:
                    if not hybrid_is_processed(url, city_name):
                        pending_urls.append(url)

                if pending_urls:
                    print(f"🎯 V2 미완료 URL: {len(pending_urls)}개")
                    return pending_urls
                else:
                    print(f"✅ V2 모든 URL 완료됨")
                    return None
        except Exception as e:
            print(f"⚠️ V2 URL 로드 실패, 기존 방식 시도: {e}")

    # 기존 시스템 (fallback)
    try:
        cache_file = os.path.join("url_cache", "collected", f"{city_name}_urls.json")

        if not os.path.exists(cache_file):
            print(f"ℹ️ URL 캐시 없음: 신규 검색 필요")
            return None

        with open(cache_file, 'r', encoding='utf-8') as f:
            cache_data = json.load(f)

        cached_urls = cache_data.get('urls', [])
        collected_time = cache_data.get('collected_time', '')

        print(f"✅ 기존 URL 캐시 로드: {len(cached_urls)}개 ({collected_time})")

        # 완료된 URL 필터링
        pending_urls = []
        for url in cached_urls:
            if not hybrid_is_processed(url, city_name):
                pending_urls.append(url)

        return pending_urls if pending_urls else None

    except Exception as e:
        print(f"❌ URL 캐시 로드 실패: {e}")
        return None

# =============================================================================
# 🧹 단순화된 유틸리티 함수들 (선택적 사용)
# =============================================================================

def filter_new_urls_from_csv(all_urls, city_name):
    """CSV 기반으로 새로운 URL만 필터링 (hashlib 비활성화 시에만 사용)"""
    if CONFIG.get("USE_HASH_SYSTEM", True):
        print("🚀 hashlib 시스템 활성화 - CSV 필터링 대신 해시 체크 사용")
        return [url for url in all_urls if not hybrid_is_processed(url, city_name)]

    # 기존 CSV 방식
    crawled_urls = get_completed_urls_from_csv(city_name)
    new_urls = [url for url in all_urls if url not in crawled_urls]

    print(f"🔍 CSV 기반 URL 필터링:")
    print(f"   📊 전체 URL: {len(all_urls)}개")
    print(f"   ✅ 이미 완료: {len(crawled_urls)}개")
    print(f"   🆕 새로운 URL: {len(new_urls)}개")

    return new_urls

# 🚀 hashlib 시스템을 위한 별칭 함수들
collect_all_24_urls = collect_product_urls_from_page  # 별칭
filter_new_urls = filter_new_urls_from_csv  # 별칭

print("✅ 그룹 3 완료: KLOOK 전용 URL 패턴 + hashlib 통합 간소화된 상태 관리 시스템!")
print(f"🚀 hashlib 최적화: {'활성화' if CONFIG.get('USE_HASH_SYSTEM', True) else '비활성화'}")
print("🧹 KLOOK /activity/ 패턴으로 완전 변경 완료")

In [ ]:
# =============================================================================
# 🚀 그룹 4: 확장성 개선 시스템 1
# - 도시 관리, 페이지네이션 분석, 시스템 초기화
# =============================================================================

def create_city_codes_file():
    """도시 코드를 JSON 파일로 저장 (UNIFIED_CITY_INFO 기반)"""
    enhanced_city_data = {
        "version": "3.0",
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "cities": {},
        "total_cities": len(UNIFIED_CITY_INFO)
    }

    for city_name, info in UNIFIED_CITY_INFO.items():
        enhanced_city_data["cities"][city_name] = {
            "code": info.get("코드", "N/A"),
            "continent": info.get("대륙", "기타"),
            "country": info.get("국가", "기타")
        }

    try:
        # 🆕 그룹 1 함수 활용하여 config 폴더 생성
        config_dir = ensure_config_directory()
        config_file = os.path.join(config_dir, "city_codes.json")

        with open(config_file, 'w', encoding='utf-8') as f:
            json.dump(enhanced_city_data, f, ensure_ascii=False, indent=2)
        print(f"✅ {config_file} 파일 생성 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        return True
    except Exception as e:
        print(f"❌ 파일 생성 실패: {e}")
        return False

def load_city_codes_from_file():
    """JSON 파일에서 도시 코드 로드 (UNIFIED_CITY_INFO와 동기화)"""
    if not os.path.exists('config/city_codes.json'):
        print("📝 config/city_codes.json 파일이 없어서 새로 생성합니다...")
        create_city_codes_file()
    
    try:
        with open('config/city_codes.json', 'r', encoding='utf-8') as f:
            city_data = json.load(f)
        
        loaded_cities = city_data.get("cities", {})
        
        for city, info in loaded_cities.items():
            if city not in UNIFIED_CITY_INFO:
                 UNIFIED_CITY_INFO[city] = {
                     "대륙": info.get("continent"),
                     "국가": info.get("country"),
                     "코드": info.get("code")
                 }
        
        print(f"✅ config/city_codes.json 로드 및 동기화 완료! ({len(UNIFIED_CITY_INFO)}개 도시)")
        print(f"📅 마지막 업데이트: {city_data.get('last_updated', '알 수 없음')}")
        
    except Exception as e:
        print(f"⚠️ 파일 로드 실패: {e}")
        print("💡 코드의 UNIFIED_CITY_INFO를 사용합니다.")

def show_supported_cities():
    """지원하는 도시 목록 표시 (UNIFIED_CITY_INFO 기반)"""
    print("\n🌍 지원하는 도시 목록:")
    print("="*50)
    
    cities_by_continent = {}
    for city, info in UNIFIED_CITY_INFO.items():
        continent = info.get("대륙", "기타")
        if continent not in cities_by_continent:
            cities_by_continent[continent] = []
        cities_by_continent[continent].append(city)

    for continent, cities in sorted(cities_by_continent.items()):
        print(f"\n📍 {continent}:")
        for city in sorted(cities):
            code = UNIFIED_CITY_INFO[city].get("코드", "N/A")
            print(f"   {city} → {code}")
    
    print(f"\n📊 총 {len(UNIFIED_CITY_INFO)}개 도시 지원")
    print("="*50)

def validate_city(city_name):
    """도시명 유효성 검사 (UNIFIED_CITY_INFO 기반)"""
    if not city_name or len(city_name.strip()) == 0:
        return False, "도시명이 비어있습니다."
    
    if city_name in UNIFIED_CITY_INFO:
        code = UNIFIED_CITY_INFO[city_name].get("코드", "N/A")
        return True, f"지원하는 도시입니다. ({code})"
    
    similar_cities = [c for c in UNIFIED_CITY_INFO if city_name.lower() in c.lower() or c.lower() in city_name.lower()]
    
    if similar_cities:
        return False, f"지원하지 않는 도시입니다. 비슷한 도시: {', '.join(similar_cities)}"
    else:
        return False, f"지원하지 않는 도시입니다. show_supported_cities()로 지원 도시 목록을 확인해주세요."
        

def update_config_for_scalability():
    """확장성을 위한 CONFIG 업데이트"""
    global CONFIG
    
    scalability_config = {
        "AUTO_LOAD_CITIES": True,
        "AUTO_SAVE_NEW_CITIES": True,
        "ENABLE_MULTI_CITY": False,
        "CITY_PROCESSING_ORDER": "sequential",
        "BACKUP_OLD_DATA": True,
        "DATA_RETENTION_DAYS": 30,
        "ENABLE_CITY_VALIDATION": True,
        "ENABLE_DUPLICATE_CHECK": True,
    }
    
    CONFIG.update(scalability_config)
    print("⚙️ CONFIG 확장성 설정 업데이트 완료!")

def analyze_pagination(driver):
    """페이지네이션 정보 분석 - 총 페이지 수, 상품 수 파악"""
    print(f"  🔍 페이지네이션 정보 분석 중...")
    
    try:
        # 페이지 로딩 완료 대기
        time.sleep(3)
        
        # 총 상품 수 찾기
        total_products = 0
        total_selectors = [
            "//span[contains(text(), '총') and contains(text(), '개')]",
            "//span[contains(text(), '전체') and contains(text(), '개')]", 
            "//div[contains(@class, 'total') or contains(@class, 'count')]//span",
            "//span[contains(text(), '결과')]",
        ]
        
        for selector in total_selectors:
            try:
                elements = driver.find_elements(By.XPATH, selector)
                for element in elements:
                    text = element.text.strip()
                    if '개' in text and any(char.isdigit() for char in text):
                        # 숫자 추출
                        import re
                        numbers = re.findall(r'\d+', text)
                        if numbers:
                            total_products = int(numbers[0])
                            print(f"    ✅ 총 상품 수 발견: {total_products}개")
                            break
                if total_products > 0:
                    break
            except:
                continue
        
        # 페이지네이션 정보 찾기
        total_pages = 1
        has_next_button = False
        
        # 다음 페이지 버튼 찾기
        next_button_selectors = [
            "//button[contains(@aria-label, '다음')]",
            "//button[contains(text(), '다음')]",
            "//a[contains(@aria-label, '다음')]", 
            "//a[contains(text(), '다음')]",
            "//button[contains(@class, 'next')]",
            "//a[contains(@class, 'next')]",
            ".pagination .next",
            ".pager .next"
        ]
        
        for selector in next_button_selectors:
            try:
                if selector.startswith('//'):
                    elements = driver.find_elements(By.XPATH, selector)
                else:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    
                for element in elements:
                    if element.is_enabled() and element.is_displayed():
                        has_next_button = True
                        print(f"    ✅ '다음 페이지' 버튼 발견!")
                        break
                if has_next_button:
                    break
            except:
                continue
        
        # 페이지 번호 찾기 (총 페이지 수 추정)
        page_number_selectors = [
            "//button[contains(@class, 'page') or contains(@class, 'pagination')]//span",
            "//a[contains(@class, 'page') or contains(@class, 'pagination')]//span",
            ".pagination button span",
            ".pager a span"
        ]
        
        max_page = 1
        for selector in page_number_selectors:
            try:
                if selector.startswith('//'):
                    elements = driver.find_elements(By.XPATH, selector)
                else:
                    elements = driver.find_elements(By.CSS_SELECTOR, selector)
                    
                for element in elements:
                    text = element.text.strip()
                    if text.isdigit():
                        page_num = int(text)
                        max_page = max(max_page, page_num)
            except:
                continue
                
        total_pages = max_page

        # 495개 상품이 1페이지 문제 해결 로직
        if total_products > 100 and total_pages == 1:
            estimated_pages = (total_products + 23) // 24  # 논리적 계산 (24개씩)
            if has_next_button:
                total_pages = estimated_pages  # 추정값 적용
                print(f"    🔧 페이지 수 보정: {total_products}개 상품 → {total_pages}페이지로 수정")
                print(f"    📊 예상 페이지당 상품 수: ~{total_products // total_pages}개")
            else:
                print(f"    ⚠️ 다음 버튼 없음: 실제로 1페이지일 수 있음 (상품 {total_products}개)")
                # 실제로 1페이지인지 확인하기 위한 추가 로직 필요할 수 있음
                
                # 페이지당 상품 수 추정 (현재 페이지 기준)
                products_per_page = 24  # 기본값
                if total_products > 0 and total_pages > 0:
                    products_per_page = min(24, total_products // total_pages + (1 if total_products % total_pages > 0 else 0))
                
                return {
                    'total_products': total_products,
                    'total_pages': total_pages, 
                    'products_per_page': products_per_page,
                    'has_next_button': has_next_button,
                    'is_pagination_available': has_next_button or total_pages > 1
                }
        
    except Exception as e:
        print(f"    ❌ 페이지네이션 분석 실패: {e}")
        return {
            'total_products': 0,
            'total_pages': 1,
            'products_per_page': 24,
            'has_next_button': False,
            'is_pagination_available': False
        }

def check_next_button(driver):
    """KLOOK 다음 페이지 버튼 작동 확인"""
    print(f"  🔍 KLOOK 다음 페이지 버튼 확인 중...")

    try:
        # KLOOK에서 disabled 버튼이 있으면 마지막 페이지
        driver.find_element(By.CSS_SELECTOR, ".klk-pagination-next-btn-disabled")
        print(f"    🏁 마지막 페이지입니다 (disabled 버튼 발견)")
        return False
    except NoSuchElementException:
        # disabled 버튼이 없으면 다음 페이지 있음
        try:
            # 활성화된 다음페이지 버튼 확인
            next_button = driver.find_element(By.CSS_SELECTOR, ".klk-pagination-next-btn:not(.klk-pagination-next-btn-disabled)")
            if next_button.is_displayed():
                print(f"    ✅ 다음 페이지 버튼이 작동 가능합니다!")
                return True
        except Exception:
            pass

    print(f"    ❌ 다음 페이지 버튼을 찾을 수 없습니다.")
    return False


def generate_crawling_plan(pagination_info, city_name):
    """크롤링 계획 생성 및 보고"""
    print(f"\n📋 크롤링 계획 수립 중...")
    
    plan = {
        'city': city_name,
        'total_products': pagination_info['total_products'],
        'total_pages': pagination_info['total_pages'],
        'products_per_page': pagination_info['products_per_page'],
        'pagination_available': pagination_info['is_pagination_available'],
        'estimated_time_minutes': 0,
        'recommended_batch_size': CONFIG['MAX_PRODUCTS_PER_CITY'],
        'strategy': '단일 페이지'
    }
    
    # 예상 소요 시간 계산 (상품당 약 30초 추정)
    products_to_crawl = min(pagination_info['total_products'], CONFIG['MAX_PRODUCTS_PER_CITY'])
    plan['estimated_time_minutes'] = products_to_crawl * 0.5  # 상품당 30초
    
    # 전략 결정
    if pagination_info['is_pagination_available'] and pagination_info['total_pages'] > 1:
        plan['strategy'] = '다중 페이지 순회'
        if pagination_info['total_products'] > CONFIG['MAX_PRODUCTS_PER_CITY']:
            plan['strategy'] += f" (최대 {CONFIG['MAX_PRODUCTS_PER_CITY']}개 제한)"
    
    return plan

def report_reconnaissance_results(plan):
    """정찰 결과 보고"""
    print(f"\n🔍 === 정찰 완료 보고서 ===")
    print(f"📍 도시: {plan['city']}")
    print(f"📊 발견된 총 상품 수: {plan['total_products']}개")
    print(f"📄 총 페이지 수: {plan['total_pages']}페이지")
    print(f"📋 페이지당 상품 수: {plan['products_per_page']}개")
    print(f"🔄 페이지네이션 가능: {'✅ 예' if plan['pagination_available'] else '❌ 아니오'}")
    print(f"⏱️ 예상 소요 시간: {plan['estimated_time_minutes']:.1f}분")
    print(f"🎯 크롤링 전략: {plan['strategy']}")
    print(f"📦 실제 수집 예정: {min(plan['total_products'], plan['recommended_batch_size'])}개")
    print(f"=" * 50)
    
    if plan['pagination_available']:
        print(f"🚀 페이지네이션을 활용한 전체 크롤링이 가능합니다!")
        return True
    else:
        print(f"⚠️ 페이지네이션이 제한적입니다. 현재 페이지만 크롤링 가능합니다.")
        return False

def initialize_file_system():
    """파일 시스템 초기화 및 설정 (리팩토링된 버전)"""
    print("🔧 그룹 4: 확장성 개선 시스템 초기화...")
    
    update_config_for_scalability()
    
    if CONFIG.get("AUTO_LOAD_CITIES", True):
        load_city_codes_from_file()
    
    print("✅ 그룹 4 시스템 초기화 완료!")
    return True

# 자동 초기화 실행
try:
    initialize_file_system()
    print("   - create_city_codes_file(): 도시 코드 JSON 파일 생성")
    print("   - show_supported_cities(): 지원 도시 목록 표시")
    print("   - validate_city(): 도시명 유효성 검사")
    print("   - analyze_pagination(): 페이지네이션 정보 분석")
    print("   - check_next_button(): 다음 페이지 버튼 확인")
    print("   - generate_crawling_plan(): 크롤링 계획 수립")
    print("   - report_reconnaissance_results(): 정찰 결과 보고")
    print("🎯 페이지네이션 자동화를 위한 준비 완료!")
    
except Exception as e:
    print(f"❌ 그룹 4 초기화 실패: {e}")
    print("💡 기존 방식으로 계속 사용 가능합니다.")

In [ ]:
# =============================================================================
# 🛠️ 그룹 5: 브라우저 제어 및 유틸리티 함수들
# - 드라이버 설정, 페이지 네비게이션, 유틸리티 기능들
# =============================================================================

def make_user_agent(ua, is_mobile):
    """User Agent 생성 함수"""
    user_agent = parse(ua)
    model = user_agent.device.model
    platform = user_agent.os.family
    platform_version = user_agent.os.version_string + ".0.0"
    version = user_agent.browser.version[0]
    ua_full_version = user_agent.browser.version_string
    architecture = "x86"
    print(platform)
    if is_mobile:
        platform_info = "Linux armv8l"
        architecture= ""
    else:
        platform_info = "Win32"
        model = ""
    RET_USER_AGENT = {
        "appVersion" : ua.replace("Mozilla/", ""),
        "userAgent": ua,
        "platform" : f"{platform_info}",
        "acceptLanguage" : "ko-KR, kr, en-US, en",
        "userAgentMetadata":{
            "brands" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersionList" : [
                {"brand":"Google Chrome", "version":f"{version}"},
                {"brand":"Chromium", "version":f"{version}"},
                {"brand":" Not A;Brand", "version":"99"}
            ],
            "fullVersion":f"{ua_full_version}",
            "platform" :platform,
            "platformVersion":platform_version,
            "architecture":architecture,
            "model" : model,
            "mobile":is_mobile
        }
    }
    return RET_USER_AGENT

def generate_random_geolocation():
    """랜덤 지리적 위치 생성"""
    ltop_lat = 37.75415601640249
    ltop_long = 126.86767642302573
    rbottom_lat = 37.593829172663945
    rbottom_long = 127.15276051439332

    targetLat = random.uniform(rbottom_lat, ltop_lat)
    targetLong = random.uniform(ltop_long,rbottom_long)
    return {"latitude":targetLat, "longitude" : targetLong, "accuracy":100}

def setup_driver():
    """크롬 드라이버 설정 및 실행"""
    chromedriver_autoinstaller.install()
    
    options = uc.ChromeOptions()
    
    UA = CONFIG["USER_AGENT"]
    options.add_argument(f"--user-agent={UA}")
    
    rand_user_folder = random.randrange(1,100)
    raw_path = os.path.abspath("cookies")
    try:
        # shutil.rmtree(raw_path) # 폴더 삭제 방지를 위해 이 줄을 주석 처리
        pass
    except:
        pass
    os.makedirs(raw_path, exist_ok=True)
    user_cookie_name = f"{raw_path}/{rand_user_folder}"
    if os.path.exists(user_cookie_name) == False:
        os.makedirs(user_cookie_name, exist_ok=True)
    
    try:
        driver = uc.Chrome(user_data_dir=user_cookie_name, options=options)
        print("✅ 크롬 드라이버 실행 성공!")
        print(platform.system())
    except Exception as e:
        print('\n',"-"*50,'\n',"-"*50,'\n')
        print("# 키홈 메세지 : 혹시 여기서 에러 발생시 [아래 블로그 참고 -> 재부팅 -> 다시 코드실행] 해보시길 바랍니다! \n (구글크롬 버젼 업그레이드 문제)")
        raise RuntimeError
        
    UA_Data = make_user_agent(UA,False)
    driver.execute_cdp_cmd("Network.setUserAgentOverride",UA_Data)
    
    GEO_DATA = generate_random_geolocation()
    driver.execute_cdp_cmd("Emulation.setGeolocationOverride", GEO_DATA)
    driver.execute_cdp_cmd("Emulation.setUserAgentOverride", UA_Data)
    driver.execute_cdp_cmd("Emulation.setNavigatorOverrides",{"platform":"Linux armv8l"})
    
    return driver

def go_to_main_page(driver):
    """KLOOK 메인 페이지로 이동"""
    driver.get("https://www.klook.com/ko/search/result/?query=%EC%84%9C%EC%9A%B8")
    time.sleep(random.uniform(CONFIG["MEDIUM_MIN_DELAY"], CONFIG["MEDIUM_MAX_DELAY"]))
    # 🆕 메인 페이지 자연스러운 탐색
    smart_scroll_selector(driver)
    return True

def find_and_fill_search(driver, city_name):
    """검색창 찾기 및 인간적인 타이핑 적용"""
    print(f"  🔍 '{city_name}' 검색창 찾는 중...")
    search_selectors = [
        (By.CSS_SELECTOR, "#js-header-search-box input"),          # KLOOK 최우선
        (By.CSS_SELECTOR, "input[name='klkHeadSearch']"),          # KLOOK name 속성 기반
        (By.CSS_SELECTOR, ".search-box_input"),                   # KLOOK 클래스 기반
        (By.XPATH, "//input[@placeholder='어디로 놀러 가세요?']"),    # KLOOK placeholder 기반
    ]

    search_input = None
    for selector_type, selector_value in search_selectors:
        try:
            search_input = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.presence_of_element_located((selector_type, selector_value))
            )
            print(f"  ✅ 검색창을 찾았습니다!")
            break
        except TimeoutException:
            continue

    if not search_input:
        raise NoSuchElementException("검색창을 찾을 수 없습니다")

    # [수정] 사람처럼 한 글자씩 타이핑하는 로직
    search_input.clear()
    for char in city_name:
        search_input.send_keys(char)
        # 각 글자 사이에 아주 짧은 무작위 딜레이 추가
        time.sleep(random.uniform(CONFIG["SHORT_MIN_DELAY"], CONFIG["SHORT_MAX_DELAY"]))
    
    # 단어 입력 후 잠시 생각하는 것처럼 대기
    time.sleep(random.uniform(1, 2))
    print(f"  📝 '{city_name}' 키워드 입력 완료")
    return True


def click_search_button(driver):
    """검색 버튼 클릭"""
    print(f"  🔎 검색 버튼 찾는 중...")
    search_button_selectors = [
        (By.CSS_SELECTOR, "#js-header-search-box button"),        # KLOOK 최우선
        (By.CSS_SELECTOR, "#js-header-search-box > button"),     # KLOOK 구체적 버전
        (By.XPATH, "//div[@id='js-header-search-box']//button"),  # KLOOK xpath 백업
    ]
    search_clicked = False
    for selector_type, selector_value in search_button_selectors:
        try:
            search_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            search_button.click()
            print(f"  ✅ 검색 버튼 클릭 성공!")
            search_clicked = True
            time.sleep(random.uniform(CONFIG["MEDIUM_MIN_DELAY"], CONFIG["MEDIUM_MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not search_clicked:
        raise NoSuchElementException("검색 버튼을 찾을 수 없습니다")
    return True

def handle_popup(driver):
    """팝업 처리"""
    popup_selectors = [
        (By.CSS_SELECTOR, ".popup-close"),
        (By.CSS_SELECTOR, ".modal-close"),
        (By.XPATH, "//button[contains(@aria-label, '닫기')]"),
        (By.XPATH, "//button[contains(text(), '닫기')]"),
        (By.XPATH, "/html/body/div[15]/div[2]/button")
    ]

    popup_closed = False
    for selector_type, selector_value in popup_selectors:
        try:
            popup_button = WebDriverWait(driver, CONFIG["POPUP_WAIT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            popup_button.click()
            print(f"  ✅ 팝업창을 닫았습니다.")
            popup_closed = True
            time.sleep(random.uniform(1, 4))
            break
        except TimeoutException:
            continue

    if not popup_closed:
        print(f"  ℹ️ 팝업창이 없거나 이미 닫혀있습니다.")
    return True

def click_view_all(driver):
    """전체 상품 보기 버튼 클릭 (안정성 강화)"""
    print(f"  📋 전체 상품 보기 버튼 찾는 중...")
    
    view_all_selectors = [
        (By.XPATH, "//button[contains(text(), '전체')]"),
        (By.XPATH, "//span[contains(text(), '전체')]//parent::button"),
        (By.CSS_SELECTOR, "button[aria-label*='전체']"),
        (By.XPATH, "/html/body/div[4]/div[2]/div/div/div/span[21]/button")
    ]

    view_all_clicked = False
    for selector_type, selector_value in view_all_selectors:
        try:
            view_all_button = WebDriverWait(driver, CONFIG["WAIT_TIMEOUT"]).until(
                EC.element_to_be_clickable((selector_type, selector_value))
            )
            driver.execute_script("arguments[0].click();", view_all_button)
            
            print(f"  ✅ 전체 상품 보기 클릭 성공!")
            view_all_clicked = True
            time.sleep(random.uniform(CONFIG["MEDIUM_MIN_DELAY"], CONFIG["MEDIUM_MAX_DELAY"]))
            break
        except TimeoutException:
            continue

    if not view_all_clicked:
        print(f"  ⚠️ 전체 상품 보기 버튼을 찾을 수 없습니다. 현재 상품으로 진행...")
        
    return True

def safe_browser_restart():
    """안전한 브라우저 재시작 with 3번 재시도"""
    global driver
    
    for attempt in range(3):  # 3번 시도
        try:
            print(f"🔄 브라우저 재시작 시도 {attempt+1}/3...")
            
            # 1단계: 안전한 종료
            if 'driver' in globals() and driver:
                driver.quit()
                driver = None
            
            # 2단계: 대기 및 정리
            wait_time = random.uniform(5, 10)
            print(f"⏰ {wait_time:.1f}초 대기 중...")
            time.sleep(wait_time)
            
            # 3단계: 새 브라우저 시작
            print("🚀 새 브라우저 시작 중...")
            driver = setup_driver()
            
            # 4단계: 동작 검증
            print("🔍 브라우저 동작 검증 중...")
            driver.get("https://www.klook.com/")
            time.sleep(2)
            
            print("✅ 브라우저 재시작 성공!")
            return True, "재시작 성공"
            
        except Exception as e:
            print(f"❌ 재시작 시도 {attempt+1} 실패: {type(e).__name__}: {e}")
            if attempt == 2:  # 마지막 시도
                print("🚨 브라우저 재시작 최종 실패!")
                return False, f"재시작 불가: {e}"
            print(f"🔄 {3-attempt-1}초 후 재시도...")
            time.sleep(3)  # 다음 시도 전 대기
    
    return False, "최종 실패"

# =============================================================================
# 🛠️ 유틸리티 함수들 (진행률 표시, 재시도 로직 등)
# =============================================================================

def print_progress(current, total, city_name, status="진행중"):
    """진행률을 시각적으로 표시하는 함수"""
    percentage = (current / total) * 100
    bar_length = 30
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    emoji = "🔍" if status == "진행중" else "✅" if status == "완료" else "❌"
    
    print(f"\n{emoji} 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"📍 현재 작업: {city_name} - {status}")
    print("-" * 50)

def print_product_progress(current, total, product_name):
    """상품별 진행률 표시 함수"""
    percentage = (current / total) * 100
    bar_length = 20
    filled_length = int(bar_length * current // total)
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    safe_name = str(product_name)[:30] + "..." if len(str(product_name)) > 30 else str(product_name)
    print(f"    🎯 상품 진행률: [{bar}] {percentage:.1f}% ({current}/{total})")
    print(f"    📦 현재 상품: {safe_name}")

def save_intermediate_results(results, city_name):
    """중간 결과를 임시 파일로 저장하는 함수"""
    if results and CONFIG.get("SAVE_INTERMEDIATE", False):
        try:
            timestamp = time.strftime('%Y%m%d_%H%M%S')
            temp_filename = f"temp_중간저장_{city_name}_{timestamp}.csv"
            pd.DataFrame(results).to_csv(temp_filename, index=False, encoding='utf-8-sig')
            print(f"  💾 중간 결과 저장: {temp_filename}")
            return temp_filename
        except Exception as e:
            print(f"  ⚠️ 중간 저장 실패: {e}")
            return None
    return None

def retry_operation(func, operation_name, max_retries=None):
    """실패한 작업을 재시도하는 함수"""
    if max_retries is None:
        max_retries = CONFIG["RETRY_COUNT"]
    
    for attempt in range(max_retries):
        try:
            return func()
        except (TimeoutException, NoSuchElementException, WebDriverException) as e:
            if attempt == max_retries - 1:
                print(f"  ❌ {operation_name} 최종 실패: {type(e).__name__}")
                raise e
            print(f"  🔄 {operation_name} 재시도 {attempt + 1}/{max_retries} (오류: {type(e).__name__})")
            time.sleep(2)
        except Exception as e:
            print(f"  ❌ {operation_name} 예상치 못한 오류: {type(e).__name__}: {e}")
            raise e

def make_safe_filename(filename):
    """파일명에 사용할 수 없는 문자 제거"""
    if not filename:
        return "기본파일명"
    
    safe_filename = str(filename)
    unsafe_chars = ['/', '\\', ':', '*', '?', '"', '<', '>', '|', '\n', '\r', '\t']
    for char in unsafe_chars:
        safe_filename = safe_filename.replace(char, '_')
    
    if len(safe_filename) > 200:
        safe_filename = safe_filename[:200]
    
    if safe_filename.startswith('.'):
        safe_filename = '_' + safe_filename[1:]
    
    return safe_filename

# =============================================================================
# ⚡ 페이지 로딩 최적화 시스템 (그룹 5 확장)
# =============================================================================

def smart_wait_for_page_load(driver, max_wait=None):
    """동적 대기시간 - 페이지 로드 완료 감지"""
    if max_wait is None:
        max_wait = CONFIG.get("SMART_WAIT_MAX", 8)
    
    start_time = time.time()
    while time.time() - start_time < max_wait:
        try:
            if driver.execute_script("return document.readyState") == "complete":
                # 페이지 로드 완료 후 최소 대기
                time.sleep(random.uniform(0.5, 1.5))
                return True
        except (WebDriverException, Exception):
            pass
        time.sleep(0.5)

    # 최대 대기시간 초과 시에도 최소 대기
    time.sleep(random.uniform(1, 2))
    return False


# =============================================================================
# 🔧 페이지 로딩 최적화 유틸리티 함수들
# =============================================================================

def wait_for_page_ready(driver, timeout=10):
    """페이지가 완전히 준비될 때까지 대기"""
    try:
        WebDriverWait(driver, timeout).until(
            lambda d: d.execute_script("return document.readyState") == "complete"
        )
        return True
    except TimeoutException:
        print(f"      ⚠️ 페이지 준비 대기 시간 초과 ({timeout}초)")
        return False
    except Exception as e:
        print(f"      ❌ 페이지 준비 확인 실패: {e}")
        return False


def adaptive_wait(base_time=2):
    """적응형 대기 시간 (시스템 부하에 따라 조정)"""
    # CONFIG에서 설정된 범위 내에서 랜덤 대기
    min_delay = CONFIG.get("SHORT_MIN_DELAY", 0.2)
    max_delay = CONFIG.get("SHORT_MAX_DELAY", 0.5)
    
    # 기본 시간에 랜덤 요소 추가
    wait_time = base_time + random.uniform(min_delay, max_delay)
    time.sleep(wait_time)
    return wait_time


def safe_tab_operation(driver, operation_func, *args, **kwargs):
    """안전한 탭 작업 수행 (에러 복구 포함)"""
    main_tab = driver.current_window_handle
    
    try:
        result = operation_func(*args, **kwargs)
        return True, result
    except Exception as e:
        print(f"      ❌ 탭 작업 실패: {e}")
        try:
            # 메인 탭으로 복귀 시도
            driver.switch_to.window(main_tab)
            return False, f"탭 작업 실패: {e}"
        except Exception as recovery_error:
            print(f"      🚨 탭 복구도 실패: {recovery_error}")
            return False, f"탭 복구 실패: {recovery_error}"
        
def human_like_scroll_patterns(driver):
    """🎭 순수 스크롤 패턴만 (기존 시스템과 중복 없음)"""
    patterns = ["smooth_reading", "comparison_scroll", "quick_scan"]
    selected = random.choice(patterns)
    
    try:
        if selected == "smooth_reading":
            for i in range(3, 6):
                scroll_amount = random.randint(250, 500)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY + distance;
                        const duration = 800 + Math.random() * 400;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({scroll_amount});
                """)
                time.sleep(random.uniform(0.5, 2.0))

        elif selected == "comparison_scroll":
            for i in range(2, 4):
                down_amount = random.randint(400, 700)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY + distance;
                        const duration = 1000 + Math.random() * 500;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({down_amount});
                """)
                time.sleep(random.uniform(0.5, 2.0))

                up_amount = random.randint(100, 300)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY - distance;
                        const duration = 600 + Math.random() * 300;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({up_amount});
                """)
                time.sleep(random.uniform(0.5, 2.0))

        elif selected == "quick_scan":
            for i in range(4, 8):
                scroll_amount = random.randint(300, 600)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY + distance;
                        const duration = 800 + Math.random() * 400;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({scroll_amount});
                """)
                time.sleep(random.uniform(0.25, 1.0))

        pass
    except Exception as e:
        print(f"  ⚠️ 스크롤 오류: {e}")


def enhanced_scroll_patterns(driver):
    """🎭 향상된 5가지 스크롤 패턴 (호환성 개선 버전)"""
    patterns = [
        "natural_reading",      # 자연스러운 읽기
        "search_and_compare",   # 검색하고 비교하기
        "rapid_overview",       # 빠른 개요 파악
        "detailed_inspection",  # 자세한 검사
        "hesitant_browsing"     # 망설이며 탐색
    ]

    selected = random.choice(patterns)
    
    try:
        if selected == "natural_reading":
            # 자연스러운 읽기 패턴 - 일정한 속도로 아래로
            for i in range(4, 7):
                scroll_amount = random.randint(200, 400)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY + distance;
                        const duration = 800 + Math.random() * 400;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({scroll_amount});
                """)

                # 읽기 시간 - 내용에 따라 다름
                reading_time = random.uniform(1.5, 3.5)
                time.sleep(reading_time)

                # 가끔 조금 위로 되돌아가기 (재확인)
                if random.random() < 0.3:
                    back_scroll = random.randint(50, 150)
                    driver.execute_script(f"""
                        function smoothScroll(distance) {{
                            const startY = window.pageYOffset;
                            const targetY = startY - distance;
                            const duration = 400 + Math.random() * 200;
                            let start = null;
                            function step(timestamp) {{
                                if (!start) start = timestamp;
                                const progress = Math.min((timestamp - start) / duration, 1);
                                const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                                const currentY = startY + (targetY - startY) * easing;
                                window.scrollTo(0, currentY);
                                if (progress < 1) requestAnimationFrame(step);
                            }}
                            requestAnimationFrame(step);
                        }}
                        smoothScroll({back_scroll});
                    """)
                    time.sleep(random.uniform(0.5, 1.0))

        elif selected == "search_and_compare":
            # 특정 항목을 찾고 비교하는 패턴
            for i in range(3, 6):
                # 빠르게 스크롤해서 훑어보기
                fast_scroll = random.randint(500, 800)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY + distance;
                        const duration = 800 + Math.random() * 400;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const easing = 1 - Math.pow(1 - progress, 4);
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({fast_scroll});
                """)
                time.sleep(random.uniform(0.8, 1.5))

                # 관심 있는 부분에서 멈춰서 자세히 보기
                if random.random() < 0.6:
                    # 위로 조금 되돌아가서 다시 보기
                    back_amount = random.randint(200, 400)
                    driver.execute_script(f"""
                        function smoothScroll(distance) {{
                            const startY = window.pageYOffset;
                            const targetY = startY - distance;
                            const duration = 600 + Math.random() * 300;
                            let start = null;
                            function step(timestamp) {{
                                if (!start) start = timestamp;
                                const progress = Math.min((timestamp - start) / duration, 1);
                                const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                                const currentY = startY + (targetY - startY) * easing;
                                window.scrollTo(0, currentY);
                                if (progress < 1) requestAnimationFrame(step);
                            }}
                            requestAnimationFrame(step);
                        }}
                        smoothScroll({back_amount});
                    """)
                    time.sleep(random.uniform(2.0, 4.0))  # 비교 검토 시간

                    # 다시 아래로 조금
                    forward_amount = random.randint(100, 300)
                    driver.execute_script(f"""
                        function smoothScroll(distance) {{
                            const startY = window.pageYOffset;
                            const targetY = startY + distance;
                            const duration = 500 + Math.random() * 200;
                            let start = null;
                            function step(timestamp) {{
                                if (!start) start = timestamp;
                                const progress = Math.min((timestamp - start) / duration, 1);
                                const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                                const currentY = startY + (targetY - startY) * easing;
                                window.scrollTo(0, currentY);
                                if (progress < 1) requestAnimationFrame(step);
                            }}
                            requestAnimationFrame(step);
                        }}
                        smoothScroll({forward_amount});
                    """)
                    time.sleep(random.uniform(1.0, 2.0))

        elif selected == "rapid_overview":
            # 빠른 개요 파악 - 전체적으로 훑어보기
            for i in range(6, 10):
                scroll_amount = random.randint(400, 700)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY + distance;
                        const duration = 800 + Math.random() * 400;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({scroll_amount});
                """)
                time.sleep(random.uniform(0.3, 0.8))  # 짧은 정지

        elif selected == "detailed_inspection":
            # 자세한 검사 - 천천히 꼼꼼히
            for i in range(3, 5):
                small_scroll = random.randint(150, 300)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY + distance;
                        const duration = 1500 + Math.random() * 800;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const easing = -(Math.cos(Math.PI * progress) - 1) / 2;
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({small_scroll});
                """)

                # 긴 검토 시간
                inspection_time = random.uniform(3.0, 6.0)
                time.sleep(inspection_time)

                # 🔧 호환성 개선: 복잡한 마우스 이벤트 제거
                # 간단한 마우스 움직임만 시뮬레이션
                if random.random() < 0.5:  # 50%로 확률 낮춤
                    try:
                        driver.execute_script("""
                            var event = new MouseEvent('mousemove', {
                                clientX: Math.random() * 500,
                                clientY: Math.random() * 300
                            });
                            document.dispatchEvent(event);
                        """)
                        time.sleep(random.uniform(0.5, 1.0))
                    except:
                        pass  # 오류 발생 시 무시

        elif selected == "hesitant_browsing":
            # 망설이며 탐색하는 패턴
            for i in range(4, 8):
                # 조금 스크롤
                hesitant_scroll = random.randint(200, 400)
                driver.execute_script(f"""
                    function smoothScroll(distance) {{
                        const startY = window.pageYOffset;
                        const targetY = startY + distance;
                        const duration = 1000 + Math.random() * 500;
                        let start = null;
                        function step(timestamp) {{
                            if (!start) start = timestamp;
                            const progress = Math.min((timestamp - start) / duration, 1);
                            const c1 = 1.70158;
                            const c2 = c1 * 1.525;
                            const easing = progress < 0.5
                                ? (Math.pow(2 * progress, 2) * ((c2 + 1) * 2 * progress - c2)) / 2
                                : (Math.pow(2 * progress - 2, 2) * ((c2 + 1) * (progress * 2 - 2) + c2) + 2) / 2;
                            const currentY = startY + (targetY - startY) * easing;
                            window.scrollTo(0, currentY);
                            if (progress < 1) requestAnimationFrame(step);
                        }}
                        requestAnimationFrame(step);
                    }}
                    smoothScroll({hesitant_scroll});
                """)
                time.sleep(random.uniform(1.0, 2.0))

                # 50% 확률로 되돌아가기
                if random.random() < 0.5:
                    back_amount = random.randint(100, 200)
                    driver.execute_script(f"""
                        function smoothScroll(distance) {{
                            const startY = window.pageYOffset;
                            const targetY = startY - distance;
                            const duration = 700 + Math.random() * 300;
                            let start = null;
                            function step(timestamp) {{
                                if (!start) start = timestamp;
                                const progress = Math.min((timestamp - start) / duration, 1);
                                const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                                const currentY = startY + (targetY - startY) * easing;
                                window.scrollTo(0, currentY);
                                if (progress < 1) requestAnimationFrame(step);
                            }}
                            requestAnimationFrame(step);
                        }}
                        smoothScroll({back_amount});
                    """)
                    time.sleep(random.uniform(0.8, 1.5))

                    # 다시 앞으로 진행
                    forward_again = random.randint(150, 350)
                    driver.execute_script(f"""
                        function smoothScroll(distance) {{
                            const startY = window.pageYOffset;
                            const targetY = startY + distance;
                            const duration = 800 + Math.random() * 400;
                            let start = null;
                            function step(timestamp) {{
                                if (!start) start = timestamp;
                                const progress = Math.min((timestamp - start) / duration, 1);
                                const easing = progress < 0.5 ? 2 * progress * progress : -1 + (4 - 2 * progress) * progress;
                                const currentY = startY + (targetY - startY) * easing;
                                window.scrollTo(0, currentY);
                                if (progress < 1) requestAnimationFrame(step);
                            }}
                            requestAnimationFrame(step);
                        }}
                        smoothScroll({forward_again});
                    """)
                    time.sleep(random.uniform(1.2, 2.5))
        pass
    except Exception as e:
        print(f"  ⚠️ 향상된 스크롤 오류: {e}")

def smart_scroll_selector(driver):
    """🎯 스마트 스크롤 선택기 - 두 함수 중 랜덤 선택 (로그 간소화)"""
    scroll_functions = [
        ("기본", human_like_scroll_patterns),
        ("향상", enhanced_scroll_patterns)
    ]

    _, selected_function = random.choice(scroll_functions)
    selected_function(driver)

print("✅ 그룹 5 완료: 브라우저 제어 및 유틸리티 함수들 정의 완료!")
print("   - setup_driver(): 크롬 드라이버 설정")
print("   - go_to_main_page(): 메인 페이지 이동")
print("   - find_and_fill_search(): 검색창 입력")
print("   - click_search_button(): 검색 버튼 클릭")
print("   - handle_popup(): 팝업 처리")
print("   - click_view_all(): 전체 상품 보기")
print("   - safe_browser_restart(): 안전한 브라우저 재시작")
print("   - human_like_scroll_patterns(): 자연스러운 스크롤")  # 🆕 추가
print("   - smart_scroll_selector(): 스마트 스크롤 선택")     # 🆕 추가

# 🆕 페이지 최적화 함수들 추가
print("   ⚡ smart_wait_for_page_load(): 동적 대기시간")
print("   ⚡ wait_for_page_ready(): 페이지 준비 대기")
print("   ⚡ adaptive_wait(): 적응형 대기 시간")
print("   ⚡ safe_tab_operation(): 안전한 탭 작업")
# 기존 유틸리티 함수들
print("   - print_progress(): 진행률 표시")
print("   - retry_operation(): 재시도 로직")
print("   - make_safe_filename(): 안전한 파일명 생성")

In [ ]:
# =============================================================================
# 🚀 그룹 6: 드라이버 초기화 및 기본 설정
# - 드라이버 시작, 이미지 폴더 설정, 기본 환경 구축
# =============================================================================

print("🚀 KLOOK 크롤링 시스템 시작!")
print("=" * 80)

# 결과 저장소 초기화
all_results = []
print("🔄 결과 저장소 초기화 완료")

# 드라이버 초기화
try:
    # 기존 드라이버가 있다면 상태 확인
    try:
        current_url = driver.current_url
        print("✅ 기존 드라이버 감지됨 - 재사용 가능한지 확인 중...")
        
        # 간단한 테스트로 드라이버 작동 확인
        driver.execute_script("return document.readyState;")
        print("✅ 기존 드라이버 정상 작동 중! 재사용합니다.")
        
    except (NameError, WebDriverException):
        print("🆕 새로운 드라이버 초기화 중...")
        driver = setup_driver()
        print("✅ 드라이버 초기화 완료!")
        
except Exception as e:
    print(f"❌ 드라이버 초기화 실패: {e}")
    print("🔄 드라이버 재생성 시도...")
    try:
        driver = setup_driver()
        print("✅ 드라이버 재생성 성공!")
    except Exception as retry_error:
        print(f"❌ 드라이버 재생성도 실패: {retry_error}")
        raise

# ✅ 수정: 이미지 폴더 연속성 확보 - 기존 이미지 보존
if CONFIG["SAVE_IMAGES"]:
    img_folder_path = os.path.join(os.path.abspath(""), "klook_thumb_img")
    # ✅ 핵심 수정: 이미지 폴더 삭제 코드 완전 제거 (데이터 연속성 확보)
    # shutil.rmtree(img_folder_path)  # 이 줄을 제거하여 기존 이미지 보존
    os.makedirs(img_folder_path, exist_ok=True)
    print(f"📁 이미지 폴더 확인 완료 (기존 이미지 연속성 보장): {img_folder_path}")

# 🆕 그룹 1 설정에서 도시 가져오기
if not CITIES_TO_SEARCH:
    print("❌ CITIES_TO_SEARCH가 비어있습니다!")
    print("💡 그룹 1에서 CITIES_TO_SEARCH = ['도시명']을 설정하세요!")
    raise ValueError("검색할 도시가 설정되지 않았습니다")

city_name = CITIES_TO_SEARCH[0]  # 🆕 첫 번째 도시 사용
continent, country = get_city_info(city_name)

print("=" * 60)
print(f"🌏 설정된 검색 도시: {city_name}")
print(f"  🌍 대륙: {continent}")
print(f"  🏛️ 국가: {country}")  
print(f"  ✈️ 공항 코드: {get_city_code(city_name)}")
print(f"⚙️ 크롤링 설정:")
print(f"  📊 최대 상품 수: {CONFIG['MAX_PRODUCTS_PER_CITY']}개")
print(f"  ⏱️ 재시도 횟수: {CONFIG['RETRY_COUNT']}회")
print(f"  🔄 대기 시간: {CONFIG['MEDIUM_MIN_DELAY']}-{CONFIG['MEDIUM_MAX_DELAY']}초")
print(f"  🖼️ 이미지 저장: {'✅ 활성화' if CONFIG['SAVE_IMAGES'] else '❌ 비활성화'}")
print("=" * 60)

# 도시 유효성 검증
is_valid, message = validate_city(city_name)
if is_valid:
    print(f"✅ 도시 유효성 검증: {message}")
else:
    print(f"⚠️ 도시 유효성 경고: {message}")
    print("💡 계속 진행하지만 예상치 못한 결과가 발생할 수 있습니다.")

# 데이터 저장 경로 미리 생성
try:
    if city_name in ["마카오", "홍콩", "싱가포르"]:
        # 도시국가: 대륙 폴더만 생성
        data_dir = os.path.join("data", continent)
        os.makedirs(data_dir, exist_ok=True)
        print(f"📁 도시국가 데이터 경로 생성: {data_dir}")
    else:
        # 일반 도시: 기존 구조
        data_dir = os.path.join("data", continent, country, city_name)
        os.makedirs(data_dir, exist_ok=True)
        print(f"📁 데이터 저장 경로 생성 완료: {data_dir}")
        
        # 국가별 통합 폴더도 생성
        country_dir = os.path.join("data", continent, country)
        os.makedirs(country_dir, exist_ok=True)
        print(f"📁 국가별 통합 경로 생성 완료: {country_dir}")
        
except Exception as e:
    print(f"⚠️ 데이터 폴더 생성 실패: {e}")
    print("💡 크롤링은 계속 진행되지만 저장 시 문제가 발생할 수 있습니다.")

# 상태 관리 시스템 초기화
print(f"\n🔄 상태 관리 시스템 초기화 중...")
try:
    crawler_state, completed_urls = load_crawler_state()
    print(f"✅ 상태 관리 시스템 로드 완료")
    print(f"  📊 이전 수집 완료: {crawler_state.get('total_collected_count', 0)}개")
    print(f"  📝 완료된 URL: {len(completed_urls)}개")
    
    # 🆕 번호 연속성 확보: 기존 CSV 마지막 번호 확인 (1부터 시작 보장)
    last_product_number = get_last_product_number(city_name)
    start_number = max(1, last_product_number + 1)  # 다음 번호부터, 최소 1 보장

    print(f"🔢 번호 연속성 설정 (1부터 시작):")
    print(f"  📊 기존 마지막 번호: {last_product_number}")
    print(f"  🆕 시작 번호: {start_number}")
    
except Exception as e:
    print(f"⚠️ 상태 관리 시스템 초기화 실패: {e}")
    print("💡 기본 상태로 진행합니다.")
    crawler_state = {
        "total_collected_count": 0,
        "last_crawled_page": 1,
        "current_session_start": datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        "last_updated": datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    completed_urls = set()
    start_number = 0

if city_name in ["마카오", "홍콩", "싱가포르"]:
    print(f"📁 이미지 저장 기본 경로: klook_thumb_img/{continent}/{city_name}/")
    print(f"📁 데이터 저장 경로: data/{continent}/")
else:
    print(f"📁 이미지 저장 기본 경로: klook_thumb_img/{continent}/{country}/{city_name}/")
    print(f"📁 데이터 저장 경로: data/{continent}/{country}/{city_name}/")
print(f"🔢 번호 시작점: {start_number}")
print("🚀 다음: 그룹 7을 실행하여 웹사이트 검색을 시작하세요!")
print("=" * 80)

In [ ]:
# 그룹7 🚀 통합 KLOOK 탭 셀렉터 & 전략 시스템 (그룹 7 + Enhanced)============
# - 각 탭별 상위 순위 크롤링 + 향상된 안정성
# - sitemap과 브라우저 크롤링 최적 조합
# - 강화된 오류 처리 및 폴백 메커니즘
# =============================================================================

print("🎯 통합 KLOOK 탭 셀렉터 & 전략 시스템 시작!")
print("=" * 80)

# =============================================================================
# 📊 KLOOK 탭 구조 정의 및 크롤링 전략 (Enhanced)
# =============================================================================

KLOOK_TAB_STRUCTURE = {
    "전체": {
        "index": 1,
        "xpath": "/html/body/div[1]/div/div/main/div/div/div[2]/div/div[1]/div/div[1]",
        "css_selector": ".klk-tabs-tab:nth-child(1)",
        "text_patterns": ["전체", "All", "전체보기", "모든"],
        "ranking_limit": 100,
        "description": "모든 카테고리 상품",
        "priority": 1,
        "backup_selectors": [
            "[data-testid*='all']",
            "button[aria-label*='all']",
            "a[href*='all']"
        ]
    },
    "투어&액티비티": {
        "index": 2,
        "xpath": "/html/body/div[1]/div/div/main/div/div/div[2]/div/div[1]/div/div[2]",
        "css_selector": ".klk-tabs-tab:nth-child(2)",
        "text_patterns": ["투어", "액티비티", "Tour", "Activity", "투어&액티비티"],
        "ranking_limit": 50,
        "description": "투어, 액티비티, 체험 상품",
        "priority": 2,
        "backup_selectors": [
            "[data-testid*='tour']",
            "[data-testid*='activity']",
            "button[aria-label*='tour']"
        ]
    },
    "티켓&입장권": {
        "index": 3,
        "xpath": "/html/body/div[1]/div/div/main/div/div/div[2]/div/div[1]/div/div[3]",
        "css_selector": ".klk-tabs-tab:nth-child(3)",
        "text_patterns": ["티켓", "입장권", "Ticket", "Admission", "티켓&입장권"],
        "ranking_limit": 50,
        "description": "입장권, 티켓 상품",
        "priority": 3,
        "backup_selectors": [
            "[data-testid*='ticket']",
            "[data-testid*='admission']",
            "button[aria-label*='ticket']"
        ]
    },
    "교통": {
        "index": 4,
        "xpath": "/html/body/div[1]/div/div/main/div/div/div[2]/div/div[1]/div/div[4]",
        "css_selector": ".klk-tabs-tab:nth-child(4)",
        "text_patterns": ["교통", "Transport", "Transportation", "이동"],
        "ranking_limit": 30,
        "description": "교통, 이동 관련 상품",
        "priority": 4,
        "backup_selectors": [
            "[data-testid*='transport']",
            "[data-testid*='transfer']",
            "button[aria-label*='transport']"
        ]
    },
    "기타": {
        "index": 6,
        "xpath": "/html/body/div[1]/div/div/main/div/div/div[2]/div/div[1]/div/div[6]",
        "css_selector": ".klk-tabs-tab:nth-child(6)",
        "text_patterns": ["기타", "Others", "기타서비스", "Miscellaneous"],
        "ranking_limit": 20,
        "description": "기타 카테고리 상품 (호텔 제외)",
        "priority": 5,
        "backup_selectors": [
            "[data-testid*='other']",
            "[data-testid*='misc']",
            "button[aria-label*='other']"
        ]
    }
}

CRAWLING_STRATEGIES = {
    "전체_sitemap": {
        "name": "전체 크롤링 (Sitemap 기반)",
        "description": "sitemap에서 모든 URL을 빠르게 수집",
        "method": "sitemap_only",
        "ranking_collection": False,
        "speed": "매우 빠름",
        "ranking_info": "없음",
        "use_tabs": False
    },
    "전체_hybrid": {
        "name": "전체 크롤링 (하이브리드)",
        "description": "상위 100개는 순위 정보, 나머지는 sitemap",
        "method": "hybrid",
        "ranking_collection": True,
        "ranking_limit": 100,
        "speed": "보통",
        "ranking_info": "상위 100개만",
        "use_tabs": True
    },
    "탭별_선택": {
        "name": "탭별 선택 크롤링",
        "description": "선택한 탭의 상위 N개 + sitemap 보완",
        "method": "tab_specific",
        "ranking_collection": True,
        "speed": "빠름",
        "ranking_info": "선택 탭만",
        "use_tabs": True
    },
    "순위만_수집": {
        "name": "순위 정보만 수집",
        "description": "각 탭별 상위 순위 정보만 빠르게 수집",
        "method": "ranking_only",
        "ranking_collection": True,
        "speed": "빠름",
        "ranking_info": "모든 탭",
        "use_tabs": True
    },
    "enhanced_full": {
        "name": "향상된 전체 크롤링",
        "description": "Enhanced 시스템으로 모든 탭에서 순위 정보 수집",
        "method": "enhanced_full",
        "ranking_collection": True,
        "speed": "보통",
        "ranking_info": "모든 탭 전체",
        "use_tabs": True
    }
}

# =============================================================================
# 🎛️ 통합 탭 셀렉터 함수들 (Enhanced + Original)
# =============================================================================

def show_crawling_strategies():
    """크롤링 전략 옵션 표시 (향상된 버전)"""
    print("\n🎯 사용 가능한 크롤링 전략:")
    print("=" * 70)

    for i, (key, strategy) in enumerate(CRAWLING_STRATEGIES.items(), 1):
        print(f"{i}. {strategy['name']}")
        print(f"   📝 설명: {strategy['description']}")
        print(f"   ⚡ 속도: {strategy['speed']}")
        print(f"   📊 순위 정보: {strategy['ranking_info']}")
        print(f"   🎯 탭 사용: {'예' if strategy['use_tabs'] else '아니오'}")
        print()

def show_available_tabs():
    """사용 가능한 탭 목록 표시 (향상된 버전)"""
    print("\n📋 KLOOK 카테고리 탭:")
    print("=" * 50)

    for i, (tab_name, tab_info) in enumerate(KLOOK_TAB_STRUCTURE.items(), 1):
        print(f"{i}. {tab_name}")
        print(f"   📝 {tab_info['description']}")
        print(f"   🎯 순위 수집: 상위 {tab_info['ranking_limit']}개")
        print(f"   🔍 우선순위: {tab_info['priority']}")
        print()

def select_crawling_strategy_interactive():
    """대화형 크롤링 전략 선택"""
    print("\n🤖 크롤링 전략을 선택하세요:")
    
    strategies = list(CRAWLING_STRATEGIES.keys())
    for i, strategy_key in enumerate(strategies, 1):
        strategy = CRAWLING_STRATEGIES[strategy_key]
        print(f"{i}. {strategy['name']}")
    
    try:
        choice = input("\n선택 (1-5, 기본값=2): ").strip()
        if not choice:
            choice = "2"
        
        choice_idx = int(choice) - 1
        if 0 <= choice_idx < len(strategies):
            selected_strategy = strategies[choice_idx]
        else:
            print("잘못된 선택, 기본값 사용")
            selected_strategy = "전체_hybrid"
    except:
        print("입력 오류, 기본값 사용")
        selected_strategy = "전체_hybrid"
    
    # 탭 선택 (탭 사용 전략인 경우)
    if CRAWLING_STRATEGIES[selected_strategy]["use_tabs"]:
        print("\n📋 크롤링할 탭을 선택하세요:")
        tabs = list(KLOOK_TAB_STRUCTURE.keys())
        for i, tab in enumerate(tabs, 1):
            print(f"{i}. {tab}")
        print("6. 모든 탭")
        
        try:
            tab_choice = input("\n선택 (1-6, 기본값=1): ").strip()
            if not tab_choice:
                tab_choice = "1"
            
            tab_idx = int(tab_choice) - 1
            if tab_idx == 5:  # 모든 탭
                selected_tabs = list(KLOOK_TAB_STRUCTURE.keys())
            elif 0 <= tab_idx < len(tabs):
                selected_tabs = [tabs[tab_idx]]
            else:
                selected_tabs = ["전체"]
        except:
            selected_tabs = ["전체"]
    else:
        selected_tabs = ["전체"]
    
    return selected_strategy, selected_tabs

def select_crawling_strategy_auto():
    """자동 크롤링 전략 선택"""
    print("🤖 자동 실행 모드: 기본 전략을 사용합니다")
    
    selected_strategy = "전체_hybrid"
    selected_tabs = ["전체"]
    
    return selected_strategy, selected_tabs

def detect_tabs_with_enhanced_fallback(driver):
    """향상된 탭 감지 (Enhanced + Original 결합)"""
    print("🔍 Enhanced 탭 구조 감지 시작...")
    
    detected_tabs = []
    wait = WebDriverWait(driver, 10)
    
    # 1. Enhanced 방식으로 탭 감지
    for tab_name, tab_info in KLOOK_TAB_STRUCTURE.items():
        tab_detected = False
        element = None
        detection_method = ""
        
        # XPath 시도
        try:
            element = driver.find_element(By.XPATH, tab_info["xpath"])
            if element.is_displayed():
                tab_detected = True
                detection_method = "XPath"
        except:
            pass
        
        # CSS 셀렉터 시도
        if not tab_detected:
            try:
                element = driver.find_element(By.CSS_SELECTOR, tab_info["css_selector"])
                if element.is_displayed():
                    tab_detected = True
                    detection_method = "CSS"
            except:
                pass
        
        # 백업 셀렉터 시도
        if not tab_detected:
            for backup_selector in tab_info.get("backup_selectors", []):
                try:
                    backup_elements = driver.find_elements(By.CSS_SELECTOR, backup_selector)
                    for backup_element in backup_elements:
                        if backup_element.is_displayed():
                            backup_text = backup_element.text.strip().lower()
                            for pattern in tab_info["text_patterns"]:
                                if pattern.lower() in backup_text:
                                    element = backup_element
                                    tab_detected = True
                                    detection_method = f"Backup: {backup_selector}"
                                    break
                        if tab_detected:
                            break
                    if tab_detected:
                        break
                except:
                    continue
        
        if tab_detected and element:
            detected_tabs.append({
                "name": tab_name,
                "element": element,
                "text": element.text.strip(),
                "tab_info": tab_info,
                "detection_method": detection_method
            })
            print(f"    ✅ {tab_name}: '{element.text.strip()}' ({detection_method})")
        else:
            print(f"    ❌ {tab_name}: 감지 실패")
    
    return detected_tabs

def click_tab_enhanced(driver, tab_name, detected_tabs=None):
    """향상된 탭 클릭 함수"""
    print(f"  🎯 '{tab_name}' 탭 처리 시작...")
    
    if tab_name == "전체":
        print(f"    ℹ️ 전체 탭은 별도 클릭 없이 현재 페이지 사용")
        return True
    
    # 감지된 탭에서 찾기
    target_tab = None
    if detected_tabs:
        for detected_tab in detected_tabs:
            if detected_tab["name"] == tab_name:
                target_tab = detected_tab
                break
    
    if not target_tab:
        print(f"    ❌ '{tab_name}' 탭을 찾을 수 없습니다")
        return False
    
    try:
        element = target_tab["element"]
        
        # 화면 중앙으로 스크롤
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
        time.sleep(1)
        
        # 클릭 가능 상태까지 대기
        try:
            wait = WebDriverWait(driver, 10)
            wait.until(EC.element_to_be_clickable(element))
        except:
            print(f"    ⚠️ 클릭 대기 타임아웃, 강제 진행")
        
        # 다중 클릭 전략
        click_methods = [
            ("JavaScript 클릭", lambda: driver.execute_script("arguments[0].click();", element)),
            ("일반 클릭", lambda: element.click()),
            ("이벤트 디스패치", lambda: driver.execute_script("arguments[0].dispatchEvent(new MouseEvent('click', {bubbles: true}));", element))
        ]
        
        for method_name, click_method in click_methods:
            try:
                click_method()
                print(f"    ✅ '{tab_name}' 탭 클릭 성공 ({method_name})")
                
                # 페이지 로딩 대기
                time.sleep(random.uniform(3, 5))
                return True
                
            except Exception as e:
                print(f"    ❌ {method_name} 실패: {e}")
                continue
        
        print(f"    ❌ '{tab_name}' 탭 클릭 완전 실패")
        return False
        
    except Exception as e:
        print(f"    ❌ '{tab_name}' 탭 처리 실패: {e}")
        return False

def collect_ranking_urls_enhanced(driver, limit=50, tab_name=""):
    """향상된 순위 URL 수집"""
    try:
        print(f"    🔍 '{tab_name}' 탭에서 상위 {limit}개 URL 수집 중...")
        
        # 페이지 안정화를 위한 스크롤
        try:
            # 점진적 스크롤로 모든 컨텐츠 로드
            for i in range(3):
                driver.execute_script("window.scrollBy(0, 1000);")
                time.sleep(1)
            
            # 맨 위로 돌아가기
            driver.execute_script("window.scrollTo(0, 0);")
            time.sleep(1)
        except:
            pass
        
        # 다양한 KLOOK 상품 링크 셀렉터들
        product_selectors = [
            "a[href*='/activity/']",
            ".product-item a",
            ".experience-card a",
            "[data-testid*='product'] a",
            ".product-card a",
            ".activity-card a",
            ".klk-product-card a"
        ]
        
        all_urls = []
        
        for selector in product_selectors:
            try:
                elements = driver.find_elements(By.CSS_SELECTOR, selector)
                print(f"      🔎 {selector}: {len(elements)}개 요소 발견")
                
                for element in elements:
                    try:
                        href = element.get_attribute('href')
                        if href and '/activity/' in href and href.startswith('http'):
                            # URL 정제
                            clean_url = href.split('?')[0]  # 쿼리 파라미터 제거
                            all_urls.append(clean_url)
                    except:
                        continue
                        
            except Exception as e:
                print(f"      ❌ {selector} 처리 실패: {e}")
                continue
        
        # 중복 제거하면서 순서 유지
        unique_urls = []
        seen = set()
        for url in all_urls:
            if url not in seen and len(unique_urls) < limit:
                seen.add(url)
                unique_urls.append(url)
        
        print(f"    ✅ '{tab_name}' 탭에서 {len(unique_urls)}개 순위 URL 수집 완료")
        
        # URL 품질 검증
        valid_urls = []
        for url in unique_urls:
            if 'klook.com' in url and '/activity/' in url:
                valid_urls.append(url)
        
        print(f"    📊 유효한 URL: {len(valid_urls)}개")
        return valid_urls
        
    except Exception as e:
        print(f"    ❌ '{tab_name}' 탭 URL 수집 실패: {e}")
        return []

def save_ranking_urls_enhanced(city_name, tab_name, urls, strategy, detection_info=None):
    """향상된 순위 URL 저장 (메타데이터 포함)"""
    try:
        os.makedirs("ranking_urls", exist_ok=True)
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"ranking_urls/{city_name}_{tab_name}_{strategy}_{timestamp}.json"
        
        ranking_data = {
            "metadata": {
                "city": city_name,
                "tab": tab_name,
                "strategy": strategy,
                "collected_time": datetime.now().isoformat(),
                "total_count": len(urls),
                "detection_method": detection_info.get("detection_method", "") if detection_info else "",
                "system_version": "integrated_enhanced"
            },
            "ranking_urls": urls,
            "url_analysis": {
                "unique_domains": list(set([url.split('/')[2] for url in urls if len(url.split('/')) > 2])),
                "avg_url_length": sum(len(url) for url in urls) / len(urls) if urls else 0,
                "contains_klook": sum(1 for url in urls if 'klook.com' in url)
            }
        }
        
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(ranking_data, f, ensure_ascii=False, indent=2)
        
        print(f"    💾 순위 URL 저장 완료: {filename}")
        print(f"    📊 저장된 URL: {len(urls)}개 (Klook: {ranking_data['url_analysis']['contains_klook']}개)")
        
        return filename
        
    except Exception as e:
        print(f"    ❌ 순위 URL 저장 실패: {e}")
        return None

# =============================================================================
# 🚀 통합 메인 실행 함수
# =============================================================================

def execute_integrated_tab_selector_system(city_name, driver, interactive_mode=False):
    """
    🎯 통합 탭 셀렉터 시스템 실행 (Enhanced + Original)
    """
    print(f"🎯 '{city_name}' 도시의 통합 탭 셀렉터 시스템 시작!")
    print("=" * 80)
    
    try:
        # 1. 크롤링 전략 선택
        show_crawling_strategies()
        show_available_tabs()
        
        if interactive_mode:
            selected_strategy, selected_tabs = select_crawling_strategy_interactive()
        else:
            selected_strategy, selected_tabs = select_crawling_strategy_auto()
        
        print(f"\n🚀 선택된 전략 실행: {selected_strategy}")
        print(f"🎯 선택된 탭: {', '.join(selected_tabs)}")
        
        # 2. 전략 정보 가져오기
        strategy_info = CRAWLING_STRATEGIES[selected_strategy]
        collected_ranking_urls = {}
        
        # 3. Sitemap 전용 모드
        if strategy_info["method"] == "sitemap_only":
            print("📊 Sitemap 전용 모드: 그룹 8에서 sitemap URL만 사용됩니다")
            return {
                "strategy": selected_strategy,
                "tabs": selected_tabs,
                "ranking_urls": {},
                "use_sitemap": True,
                "use_ranking": False,
                "execution_time": datetime.now().isoformat(),
                "success": True
            }
        
        # 4. 탭 기반 전략 실행
        if strategy_info["use_tabs"]:
            print("🔍 탭 구조 감지 및 순위 정보 수집 시작...")
            
            # Enhanced 탭 감지
            detected_tabs = detect_tabs_with_enhanced_fallback(driver)
            
            if not detected_tabs:
                print("❌ 탭을 감지할 수 없습니다. Sitemap 모드로 폴백합니다.")
                return {
                    "strategy": "fallback_sitemap",
                    "tabs": ["전체"],
                    "ranking_urls": {},
                    "use_sitemap": True,
                    "use_ranking": False,
                    "error": "Tab detection failed",
                    "execution_time": datetime.now().isoformat(),
                    "success": False
                }
            
            # Enhanced 전체 크롤링 모드
            if strategy_info["method"] == "enhanced_full":
                print("🎯 Enhanced 모드: 모든 탭에서 순위 수집")
                selected_tabs = list(KLOOK_TAB_STRUCTURE.keys())
            
            # 각 선택된 탭에서 순위 URL 수집
            for tab_name in selected_tabs:
                if tab_name in KLOOK_TAB_STRUCTURE:
                    tab_info = KLOOK_TAB_STRUCTURE[tab_name]
                    ranking_limit = tab_info["ranking_limit"]
                    
                    # 순위만 수집 모드에서는 제한
                    if strategy_info["method"] == "ranking_only":
                        ranking_limit = min(ranking_limit, 20)
                    
                    print(f"\n🔄 '{tab_name}' 탭 처리 중...")
                    
                    # 탭 클릭 (향상된 방식)
                    click_success = click_tab_enhanced(driver, tab_name, detected_tabs)
                    
                    if not click_success:
                        print(f"❌ '{tab_name}' 탭 클릭 실패, 건너뜀")
                        continue
                    
                    # 순위 URL 수집 (향상된 방식)
                    ranking_urls = collect_ranking_urls_enhanced(driver, ranking_limit, tab_name)
                    
                    if ranking_urls:
                        collected_ranking_urls[tab_name] = ranking_urls
                        
                        # 탭 감지 정보 찾기
                        detection_info = None
                        for detected_tab in detected_tabs:
                            if detected_tab["name"] == tab_name:
                                detection_info = detected_tab
                                break
                        
                        # 순위 URL 저장 (향상된 방식)
                        save_ranking_urls_enhanced(
                            city_name, tab_name, ranking_urls, 
                            selected_strategy, detection_info
                        )
                    else:
                        print(f"⚠️ '{tab_name}' 탭에서 URL을 수집하지 못했습니다")
        
        # 5. 결과 구성
        result = {
            "strategy": selected_strategy,
            "strategy_info": strategy_info,
            "tabs": selected_tabs,
            "ranking_urls": collected_ranking_urls,
            "use_sitemap": strategy_info["method"] in ["hybrid", "tab_specific"],
            "use_ranking": strategy_info["ranking_collection"],
            "execution_time": datetime.now().isoformat(),
            "success": True,
            "detected_tabs_count": len(detected_tabs) if 'detected_tabs' in locals() else 0
        }
        
        # 6. 결과 요약
        total_ranking_urls = sum(len(urls) for urls in collected_ranking_urls.values())
        successful_tabs = len(collected_ranking_urls)
        
        print(f"\n📊 통합 탭 셀렉터 시스템 완료 요약:")
        print(f"   🎯 전략: {strategy_info['name']}")
        print(f"   📋 선택 탭: {', '.join(selected_tabs)}")
        print(f"   ✅ 성공 탭: {successful_tabs}/{len(selected_tabs)}")
        print(f"   📈 총 순위 URL: {total_ranking_urls}개")
        print(f"   🗺️ Sitemap 사용: {'예' if result['use_sitemap'] else '아니오'}")
        print(f"   🏆 순위 정보: {'예' if result['use_ranking'] else '아니오'}")
        
        # 탭별 상세 정보
        if collected_ranking_urls:
            print(f"\n📈 탭별 수집 결과:")
            for tab_name, urls in collected_ranking_urls.items():
                print(f"   - {tab_name}: {len(urls)}개")
        
        return result
        
    except Exception as e:
        print(f"❌ 통합 탭 셀렉터 시스템 실행 실패: {e}")
        return {
            "strategy": "error_fallback",
            "tabs": ["전체"],
            "ranking_urls": {},
            "use_sitemap": True,
            "use_ranking": False,
            "error": str(e),
            "execution_time": datetime.now().isoformat(),
            "success": False
        }

# =============================================================================
# 🎯 자동 실행 래퍼 함수 (기존 그룹 7 호환)
# =============================================================================

def execute_tab_selector_system(city_name, driver):
    """기존 그룹 7 함수와의 호환성을 위한 래퍼"""
    return execute_integrated_tab_selector_system(city_name, driver, interactive_mode=False)

# =============================================================================
# 🎯 통합 시스템 테스트 및 검증 함수
# =============================================================================

def validate_system_integration(driver, test_city="서울"):
    """통합 시스템 검증 함수"""
    print("🧪 통합 시스템 검증 시작...")
    
    validation_results = {
        "tab_detection": False,
        "strategy_selection": False,
        "url_collection": False,
        "file_saving": False,
        "overall_success": False
    }
    
    try:
        # 1. 탭 감지 테스트
        print("1️⃣ 탭 감지 테스트...")
        detected_tabs = detect_tabs_with_enhanced_fallback(driver)
        validation_results["tab_detection"] = len(detected_tabs) > 0
        print(f"   탭 감지 결과: {len(detected_tabs)}개 탭 발견")
        
        # 2. 전략 선택 테스트
        print("2️⃣ 전략 선택 테스트...")
        try:
            strategy, tabs = select_crawling_strategy_auto()
            validation_results["strategy_selection"] = strategy in CRAWLING_STRATEGIES
            print(f"   전략 선택 결과: {strategy}, 탭: {tabs}")
        except Exception as e:
            print(f"   전략 선택 실패: {e}")
        
        # 3. URL 수집 테스트 (첫 번째 탭만)
        print("3️⃣ URL 수집 테스트...")
        if detected_tabs:
            try:
                first_tab = detected_tabs[0]["name"]
                test_urls = collect_ranking_urls_enhanced(driver, 5, first_tab)
                validation_results["url_collection"] = len(test_urls) > 0
                print(f"   URL 수집 결과: {len(test_urls)}개")
            except Exception as e:
                print(f"   URL 수집 실패: {e}")
        
        # 4. 파일 저장 테스트
        print("4️⃣ 파일 저장 테스트...")
        try:
            test_urls = ["https://www.klook.com/activity/12345-test/"]
            filename = save_ranking_urls_enhanced(test_city, "테스트", test_urls, "test_strategy")
            validation_results["file_saving"] = filename is not None
            print(f"   파일 저장 결과: {'성공' if filename else '실패'}")
            
            # 테스트 파일 정리
            if filename and os.path.exists(filename):
                os.remove(filename)
        except Exception as e:
            print(f"   파일 저장 실패: {e}")
        
        # 전체 검증 결과
        validation_results["overall_success"] = all([
            validation_results["tab_detection"],
            validation_results["strategy_selection"],
            validation_results["url_collection"] or validation_results["file_saving"]  # 최소 하나는 성공
        ])
        
        print(f"\n🧪 검증 완료:")
        for test_name, result in validation_results.items():
            status = "✅" if result else "❌"
            print(f"   {status} {test_name}: {'통과' if result else '실패'}")
        
        return validation_results
        
    except Exception as e:
        print(f"❌ 검증 중 오류 발생: {e}")
        return validation_results

def generate_system_report(city_name, execution_result):
    """시스템 실행 보고서 생성"""
    try:
        os.makedirs("reports", exist_ok=True)
        
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        report_filename = f"reports/{city_name}_system_report_{timestamp}.json"
        
        report_data = {
            "report_metadata": {
                "city": city_name,
                "generated_time": datetime.now().isoformat(),
                "system_version": "integrated_enhanced_v1.0",
                "report_type": "tab_selector_execution"
            },
            "execution_summary": execution_result,
            "performance_metrics": {
                "total_tabs_attempted": len(execution_result.get("tabs", [])),
                "successful_tabs": len(execution_result.get("ranking_urls", {})),
                "total_urls_collected": sum(len(urls) for urls in execution_result.get("ranking_urls", {}).values()),
                "success_rate": len(execution_result.get("ranking_urls", {})) / max(len(execution_result.get("tabs", [])), 1) * 100
            },
            "recommendations": []
        }
        
        # 성능 기반 추천사항 생성
        success_rate = report_data["performance_metrics"]["success_rate"]
        total_urls = report_data["performance_metrics"]["total_urls_collected"]
        
        if success_rate < 50:
            report_data["recommendations"].append("탭 감지 성공률이 낮습니다. 페이지 구조 변경 확인 필요")
        
        if total_urls < 10:
            report_data["recommendations"].append("수집된 URL이 적습니다. 페이지 로딩 시간 증가 고려")
        
        if execution_result.get("success", False):
            report_data["recommendations"].append("시스템이 정상 작동했습니다. 정기적인 모니터링 권장")
        
        with open(report_filename, 'w', encoding='utf-8') as f:
            json.dump(report_data, f, ensure_ascii=False, indent=2)
        
        print(f"📊 시스템 보고서 생성 완료: {report_filename}")
        return report_filename
        
    except Exception as e:
        print(f"❌ 보고서 생성 실패: {e}")
        return None

# =============================================================================
# 🎯 통합 시스템 메인 실행 함수 (모든 기능 포함)
# =============================================================================

def run_complete_klook_system(city_name, driver, config=None):
    """
    완전한 KLOOK 시스템 실행 (검증 + 실행 + 보고서)
    
    Args:
        city_name: 도시명
        driver: WebDriver 인스턴스
        config: 설정 딕셔너리
    
    Returns:
        dict: 완전한 실행 결과
    """
    print(f"🚀 완전한 KLOOK 시스템 실행 시작: '{city_name}'")
    print("=" * 90)
    
    # 기본 설정
    default_config = {
        "run_validation": True,
        "interactive_mode": False,
        "generate_report": True,
        "cleanup_test_files": True,
        "max_retries": 3
    }
    
    if config:
        default_config.update(config)
    
    complete_result = {
        "city_name": city_name,
        "config": default_config,
        "start_time": datetime.now().isoformat(),
        "validation_results": None,
        "execution_results": None,
        "report_filename": None,
        "overall_success": False,
        "errors": []
    }
    
    try:
        # 1. 시스템 검증 (선택적)
        if default_config["run_validation"]:
            print("\n🧪 === 1단계: 시스템 검증 ===")
            validation_results = validate_system_integration(driver, city_name)
            complete_result["validation_results"] = validation_results
            
            if not validation_results["overall_success"]:
                print("⚠️ 검증에서 일부 문제가 발견되었지만 실행을 계속합니다...")
        
        # 2. 메인 시스템 실행
        print("\n🎯 === 2단계: 탭 셀렉터 시스템 실행 ===")
        execution_results = execute_integrated_tab_selector_system(
            city_name, driver, default_config["interactive_mode"]
        )
        complete_result["execution_results"] = execution_results
        
        # 3. 보고서 생성 (선택적)
        if default_config["generate_report"] and execution_results:
            print("\n📊 === 3단계: 보고서 생성 ===")
            report_filename = generate_system_report(city_name, execution_results)
            complete_result["report_filename"] = report_filename
        
        # 4. 전체 성공 여부 판단
        complete_result["overall_success"] = (
            execution_results and 
            execution_results.get("success", False) and
            len(execution_results.get("ranking_urls", {})) > 0
        )
        
        complete_result["end_time"] = datetime.now().isoformat()
        
        # 5. 최종 요약
        print(f"\n🎉 === 완전한 KLOOK 시스템 실행 완료 ===")
        print(f"🏙️ 도시: {city_name}")
        print(f"✅ 전체 성공: {'예' if complete_result['overall_success'] else '아니오'}")
        
        if execution_results:
            print(f"🎯 사용된 전략: {execution_results.get('strategy', 'N/A')}")
            print(f"📈 수집된 URL: {sum(len(urls) for urls in execution_results.get('ranking_urls', {}).values())}개")
            print(f"🎪 성공한 탭: {len(execution_results.get('ranking_urls', {}))}개")
        
        if complete_result["report_filename"]:
            print(f"📊 보고서: {complete_result['report_filename']}")
        
        return complete_result
        
    except Exception as e:
        error_msg = f"완전한 시스템 실행 중 오류: {e}"
        print(f"❌ {error_msg}")
        complete_result["errors"].append(error_msg)
        complete_result["end_time"] = datetime.now().isoformat()
        return complete_result

# =============================================================================
# 🎯 편의 함수들 (Quick Start)
# =============================================================================

def quick_start_ranking_collection(city_name, driver, strategy="전체_hybrid"):
    """빠른 시작: 순위 수집만"""
    print(f"⚡ 빠른 시작: '{city_name}' 순위 수집")
    
    config = {
        "run_validation": False,
        "interactive_mode": False,
        "generate_report": False
    }
    
    result = execute_integrated_tab_selector_system(city_name, driver, False)
    
    if result.get("success"):
        total_urls = sum(len(urls) for urls in result.get("ranking_urls", {}).values())
        print(f"✅ 빠른 수집 완료: {total_urls}개 URL")
    else:
        print("❌ 빠른 수집 실패")
    
    return result

def quick_start_full_system(city_name, driver):
    """빠른 시작: 전체 시스템"""
    print(f"🚀 빠른 시작: '{city_name}' 전체 시스템")
    
    return run_complete_klook_system(city_name, driver)

# =============================================================================
# 🔧 시스템 상태 확인 및 디버깅
# =============================================================================

def check_system_health():
    """시스템 상태 확인"""
    print("🩺 시스템 상태 확인...")
    
    health_status = {
        "directories": {},
        "configurations": {},
        "dependencies": {},
        "overall_health": "unknown"
    }
    
    # 디렉토리 확인
    required_dirs = ["ranking_urls", "reports"]
    for dir_name in required_dirs:
        try:
            os.makedirs(dir_name, exist_ok=True)
            health_status["directories"][dir_name] = "ok"
        except Exception as e:
            health_status["directories"][dir_name] = f"error: {e}"
    
    # 설정 확인
    health_status["configurations"]["tab_structure"] = "ok" if KLOOK_TAB_STRUCTURE else "missing"
    health_status["configurations"]["strategies"] = "ok" if CRAWLING_STRATEGIES else "missing"
    
    # 의존성 확인
    try:
        import selenium
        health_status["dependencies"]["selenium"] = "ok"
    except ImportError:
        health_status["dependencies"]["selenium"] = "missing"
    
    try:
        import json
        health_status["dependencies"]["json"] = "ok"
    except ImportError:
        health_status["dependencies"]["json"] = "missing"
    
    # 전체 상태 결정
    all_dirs_ok = all(status == "ok" for status in health_status["directories"].values())
    all_configs_ok = all(status == "ok" for status in health_status["configurations"].values())
    all_deps_ok = all(status == "ok" for status in health_status["dependencies"].values())
    
    if all_dirs_ok and all_configs_ok and all_deps_ok:
        health_status["overall_health"] = "healthy"
    elif all_configs_ok and all_deps_ok:
        health_status["overall_health"] = "minor_issues"
    else:
        health_status["overall_health"] = "unhealthy"
    
    # 결과 출력
    print(f"📊 전체 상태: {health_status['overall_health']}")
    
    for category, items in health_status.items():
        if category != "overall_health":
            print(f"🔍 {category}:")
            for item, status in items.items():
                status_icon = "✅" if status == "ok" else "❌"
                print(f"   {status_icon} {item}: {status}")
    
    return health_status

# =============================================================================
# 🎯 시스템 초기화 및 완료 메시지
# =============================================================================

print("\n" + "=" * 90)
print("✅ 통합 KLOOK 탭 셀렉터 & 전략 시스템 완료!")
print("=" * 90)

print("\n🔧 주요 기능:")
print("   🎯 detect_tabs_with_enhanced_fallback() - 강화된 탭 감지")
print("   🔄 click_tab_enhanced() - 다중 방식 탭 클릭")
print("   📊 collect_ranking_urls_enhanced() - 향상된 URL 수집")
print("   💾 save_ranking_urls_enhanced() - 메타데이터 포함 저장")
print("   🚀 execute_integrated_tab_selector_system() - 통합 실행")
print("   📊 run_complete_klook_system() - 완전한 시스템 실행")

print("\n🎯 지원 크롤링 전략:")
print("   📋 전체_sitemap - Sitemap 전용")
print("   🔀 전체_hybrid - 순위 + Sitemap 조합")
print("   🎪 탭별_선택 - 특정 탭 선택")
print("   🏆 순위만_수집 - 순위 정보만")
print("   ⚡ enhanced_full - 모든 탭 Enhanced 모드")

print("\n🚀 빠른 시작:")
print("   # 기본 사용")
print("   result = execute_tab_selector_system(city_name, driver)")
print("")
print("   # 전체 시스템 실행")
print("   result = run_complete_klook_system(city_name, driver)")
print("")
print("   # 빠른 순위 수집")
print("   result = quick_start_ranking_collection(city_name, driver)")

print("\n🔧 시스템 상태 확인:")
print("   health = check_system_health()")

print("\n🎉 통합 시스템이 준비되었습니다!")
print("🔗 그룹 8 sitemap 시스템과의 연동 준비 완료")

In [ ]:
# =============================================================================
# 🚀 그룹 8: 최적화된 URL 수집 및 페이지네이션 분석 (중복 제거 버전 · 드롭인 완성본)
# - 기존 그룹 1~7 함수 100% 재활용
# - 전역 의존 최소화 / 안전 폴백 / 깔끔 로그
# =============================================================================

def _safe_get_config(local_config, key, default=None):
    try:
        if local_config and key in local_config:
            return local_config[key]
    except Exception:
        pass
    try:
        return CONFIG.get(key, default)  # 글로벌 CONFIG 폴백
    except Exception:
        return default

def _safe_callable(name):
    """globals()에서 name을 찾아 callable이면 반환, 아니면 None"""
    fn = globals().get(name)
    return fn if callable(fn) else None

def execute_optimized_url_collection(driver, city_name, start_number, completed_urls, config=None):
    """
    🎯 최적화된 URL 수집 및 분석 실행
    - 그룹 4: analyze_pagination / generate_crawling_plan / report_reconnaissance_results
    - 그룹 3: collect_urls_with_csv_safety / save_collected_urls
    """
    print("🔍 그룹 8: 최적화된 URL 수집 및 페이지네이션 분석 시작!")
    print("=" * 60)

    # ========== 1단계: 페이지네이션 분석 (그룹 4 재활용) ==========
    print("🔍 === 1단계: 페이지네이션 정보 분석 ===")

    analyze_pagination = _safe_callable("analyze_pagination")
    generate_crawling_plan = _safe_callable("generate_crawling_plan")
    report_reconnaissance_results = _safe_callable("report_reconnaissance_results")
    check_next_button = _safe_callable("check_next_button")

    pagination_info = {'total_products': 0, 'total_pages': 1, 'products_per_page': 24}
    strategy = "기본"
    try:
        if not (analyze_pagination and generate_crawling_plan and report_reconnaissance_results):
            raise RuntimeError("필수 분석 함수가 누락되었습니다 (Group 4)")

        pg_info = analyze_pagination(driver)
        pagination_info.update(pg_info or {})
        plan = generate_crawling_plan(pagination_info, city_name)
        can_proceed = report_reconnaissance_results(plan)
        button_ok = check_next_button(driver) if check_next_button else True

        strategy = "다중 페이지" if (can_proceed and button_ok) else "단일 페이지"
        print(f"📊 선택된 전략: {strategy}")

    except Exception as e:
        print(f"⚠️ 페이지네이션 분석 실패: {e}")
        strategy = "기본"

    # ========== 2단계: URL 수집 (그룹 3 재활용) ==========
    print("🔍 === 2단계: URL 수집 ===")

    collect_urls_with_csv_safety = _safe_callable("collect_urls_with_csv_safety")
    save_collected_urls = _safe_callable("save_collected_urls")

    if not collect_urls_with_csv_safety:
        print("❌ URL 수집 함수가 없습니다: collect_urls_with_csv_safety")
        return {
            'success': False,
            'strategy': strategy,
            'pagination_info': pagination_info,
            'urls_collected': 0,
            'urls_to_process': [],
            'city_name': city_name,
        }

    try:
        collected_urls = collect_urls_with_csv_safety(driver, city_name) or []
        if collected_urls:
            print(f"🎉 새로운 URL {len(collected_urls)}개 수집 성공!")

            max_products = int(_safe_get_config(config, 'MAX_PRODUCTS_PER_CITY', 100))
            urls_to_process = collected_urls[:max_products]

            if save_collected_urls:
                try:
                    save_collected_urls(city_name, urls_to_process)
                except Exception as se:
                    print(f"⚠️ URL 캐시 저장 실패: {type(se).__name__}: {se}")

            return {
                'success': True,
                'strategy': strategy,
                'pagination_info': pagination_info,
                'urls_collected': len(collected_urls),
                'urls_to_process': urls_to_process,
                'city_name': city_name,
                'start_number': start_number,
            }
        else:
            print("❌ 새로운 URL을 찾을 수 없습니다")
            return {
                'success': False,
                'strategy': strategy,
                'pagination_info': pagination_info,
                'urls_collected': 0,
                'urls_to_process': [],
                'city_name': city_name,
                'start_number': start_number,
            }

    except Exception as e:
        print(f"❌ URL 수집 실패: {e}")
        return {
            'success': False,
            'strategy': strategy,
            'pagination_info': pagination_info,
            'urls_collected': 0,
            'urls_to_process': [],
            'city_name': city_name,
            'start_number': start_number,
            'error': str(e),
        }

def display_collection_summary(result, completed_urls, config=None):
    """🔍 수집 결과 요약 표시 (핵심 정보만)"""
    print("🔍 === 3단계: 수집 결과 요약 ===")

    get_city_info = _safe_callable("get_city_info")

    city_name = result.get('city_name', 'Unknown')
    success = result.get('success', False)
    strategy = result.get('strategy', 'Unknown')
    start_number = result.get('start_number', 1)
    urls_to_process = result.get('urls_to_process', [])

    print("📊 수집 결과 요약:")
    print(f"  🏙️ 대상 도시: {city_name}")
    print(f"  📄 페이지네이션 전략: {strategy}")
    print(f"  🔢 수집된 신규 URL: {result.get('urls_collected', 0)}개")
    print(f"  🎯 처리할 URL: {len(urls_to_process)}개")

    if success and urls_to_process:
        print(f"  📈 예상 번호 범위: {start_number} ~ {start_number + len(urls_to_process) - 1}")

        continent, country = ("", "")
        try:
            if get_city_info:
                continent, country = get_city_info(city_name)
        except Exception:
            pass

        special_cities = {"마카오", "홍콩", "싱가포르"}
        if city_name in special_cities:
            print(f"  📁 이미지 저장: klook_thumb_img/{continent}/{city_name}/")
            print(f"  📁 데이터 저장: data/{continent}/")
        else:
            print(f"  📁 이미지 저장: klook_thumb_img/{continent}/{country}/{city_name}/")
            print(f"  📁 데이터 저장: data/{continent}/{country}/{city_name}/")

        print("✅ 그룹 8 완료: URL 수집 및 분석 성공!")
        print("🚀 다음: 그룹 9를 실행하여 실제 크롤링을 시작하세요!")
    else:
        print("⚠️ 그룹 8 완료: 크롤링할 새로운 URL이 없음")
        print("💡 다음 단계: 다른 도시로 변경하거나 페이지네이션을 통한 다음 페이지 이동")

# =============================================================================
# 🎯 메인 실행 (기존 인터페이스 호환)
# - 전역 변수 존재 시 그대로 재활용하며, 누락 시 안전 처리
# =============================================================================
def run_optimized_group8():
    """
    최적화된 그룹 8 실행 (전역 변수 재활용)
    필요 전역: driver, city_name, start_number, completed_urls, CONFIG(선택)
    """
    global driver, city_name, start_number, completed_urls
    try:
        print("📋 현재 상태 확인:")
        print(f"  🏙️ 대상 도시: {city_name}")
        print(f"  📊 완료된 URL: {len(completed_urls)}개")
        print(f"  🔢 시작 번호: {start_number}")
        print(f"  📱 드라이버 상태: 활성")
    except NameError as e:
        print(f"❌ 필수 변수가 설정되지 않았습니다: {e}")
        print("💡 그룹 6과 그룹 7을 먼저 실행하세요!")
        return {'success': False, 'error': 'Missing variables'}

    result = execute_optimized_url_collection(
        driver=driver,
        city_name=city_name,
        start_number=start_number,
        completed_urls=completed_urls,
        config=globals().get("CONFIG"),
    )

    display_collection_summary(result, completed_urls, config=globals().get("CONFIG"))

    # 기존 파이프라인 호환을 위해 결과를 일부 전역으로도 제공
    if result.get('success'):
        globals()['urls_to_crawl'] = result['urls_to_process']
        globals()['total_products_to_crawl'] = len(result['urls_to_process'])
    else:
        globals()['urls_to_crawl'] = []
        globals()['total_products_to_crawl'] = 0

    return result

# =============================================================================
# 🎯 자동 실행 (원하면 그대로 사용)
# =============================================================================
if __name__ == "__main__":
    r = run_optimized_group8()
    print("\n" + "=" * 60)
    print("✅ 최적화된 그룹 8 완료!")
    print("🔧 개선사항:")
    print("   - ❌ 중복 함수 제거 (그룹 3, 4 함수 재활용)")
    print("   - ❌ 전역 의존 축소 (명시 인자 전달)")
    print("   - ❌ 불필요한 로직 제거")
    print("   - ✅ 핵심 기능 100% 보존")
    print("   - ✅ 기존 시스템과 100% 호환성")
    print("🛡️ 보안 기능:")
    print("   - CSV 기반 URL 중복 방지")
    print("   - 완료된 작업 자동 제외")
    print("   - 번호 연속성 보장")
    if r.get('success') and r.get('urls_to_process'):
        print(f"🎯 다음 단계: 그룹 9에서 {len(r['urls_to_process'])}개 상품 크롤링 시작")


In [ ]:
# =============================================================================
# 🚀 그룹 9-A: 페이지네이션 핵심 시스템
# - 페이지네이션 상태 관리, URL 저장/복귀, 페이지 이동 함수들
# - 기존 그룹 1-8의 모든 연속성 보장 시스템 활용
# =============================================================================

print("🔧 그룹 9-A: 페이지네이션 핵심 시스템 로딩...")

# =============================================================================
# 📊 페이지네이션 상태 관리 시스템
# =============================================================================

def save_pagination_state(city_name, current_page, current_list_url, total_crawled, target_products):
    """
    페이지네이션 상태를 저장하여 세션 간 연속성 보장
    """
    try:
        os.makedirs("pagination_state", exist_ok=True)
        state_file = os.path.join("pagination_state", f"{city_name}_pagination.json")
        
        pagination_state = {
            "city": city_name,
            "current_page": current_page,
            "current_list_url": current_list_url,
            "total_crawled": total_crawled,
            "target_products": target_products,
            "last_updated": datetime.now().isoformat(),
            "session_id": datetime.now().strftime('%Y%m%d_%H%M%S'),
            "status": "active"
        }
        
        with open(state_file, 'w', encoding='utf-8') as f:
            json.dump(pagination_state, f, ensure_ascii=False, indent=2)
        
        print(f"      ✅ 페이지네이션 상태 저장: {current_page}페이지, {total_crawled}개 완료")
        return True
        
    except Exception as e:
        print(f"      ❌ 페이지네이션 상태 저장 실패: {e}")
        return False

def load_pagination_state(city_name):
    """
    이전 세션의 페이지네이션 상태 로드
    """
    try:
        state_file = os.path.join("pagination_state", f"{city_name}_pagination.json")
        
        if not os.path.exists(state_file):
            print(f"      ℹ️ 페이지네이션 상태 파일 없음 - 새 세션 시작")
            return None
        
        with open(state_file, 'r', encoding='utf-8') as f:
            state = json.load(f)
        
        print(f"      ✅ 페이지네이션 상태 로드: {state.get('current_page', 1)}페이지부터 재개")
        print(f"      📊 이전 진행: {state.get('total_crawled', 0)}개 완료")
        
        return state
        
    except Exception as e:
        print(f"      ⚠️ 페이지네이션 상태 로드 실패: {e}")
        return None

def clear_pagination_state(city_name):
    """
    페이지네이션 상태 초기화 (크롤링 완료 시)
    """
    try:
        state_file = os.path.join("pagination_state", f"{city_name}_pagination.json")
        
        if os.path.exists(state_file):
            # 완료 상태로 마킹
            with open(state_file, 'r', encoding='utf-8') as f:
                state = json.load(f)
            
            state["status"] = "completed"
            state["completed_time"] = datetime.now().isoformat()
            
            with open(state_file, 'w', encoding='utf-8') as f:
                json.dump(state, f, ensure_ascii=False, indent=2)
            
            print(f"      ✅ 페이지네이션 상태 완료 처리")
        
        return True
        
    except Exception as e:
        print(f"      ⚠️ 페이지네이션 상태 정리 실패: {e}")
        return False

# =============================================================================
# 🔗 목록페이지 URL 저장 및 복귀 시스템
# =============================================================================

def save_list_page_url(driver, city_name, page_number):
    """
    현재 목록페이지 URL을 저장
    """
    try:
        current_url = driver.current_url
        
        # URL 검증
        if not is_valid_list_page_url(current_url, city_name):
            print(f"      ⚠️ 유효하지 않은 목록페이지 URL: {current_url}")
            return None
        
        print(f"      📝 {page_number}페이지 URL 저장: ...{current_url[-50:]}")
        return current_url
        
    except Exception as e:
        print(f"      ❌ 목록페이지 URL 저장 실패: {e}")
        return None

def return_to_list_page(driver, saved_url, city_name, max_attempts=3):
    """
    저장된 목록페이지 URL로 안전하게 복귀
    """
    print(f"      🔙 목록페이지로 복귀 중...")
    
    for attempt in range(max_attempts):
        try:
            current_url = driver.current_url
            
            # 이미 목록페이지에 있는지 확인
            if is_valid_list_page_url(current_url, city_name):
                # 상품 링크 개수로 확인
                product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
                if len(product_links) >= 10:
                    print(f"      ✅ 이미 올바른 목록페이지에 있음 ({len(product_links)}개 상품)")
                    return True, current_url
            
            print(f"      🔄 목록페이지 복귀 시도 {attempt + 1}/{max_attempts}")
            
            if saved_url:
                # 저장된 URL로 직접 이동
                driver.get(saved_url)
                print(f"      📍 저장된 URL로 이동: ...{saved_url[-50:]}")
            else:
                # 뒤로가기 시도
                driver.back()
                print(f"      ⬅️ 뒤로가기 시도")
            
            time.sleep(random.uniform(3, 5))
            
            # 복귀 확인
            new_url = driver.current_url
            product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
            
            if len(product_links) >= 10:
                print(f"      ✅ 목록페이지 복귀 성공! ({len(product_links)}개 상품 확인)")
                return True, new_url
            else:
                print(f"      ⚠️ 복귀했지만 상품 수 부족: {len(product_links)}개")
                
        except Exception as e:
            print(f"      ❌ 복귀 시도 {attempt + 1} 실패: {e}")
            
        if attempt < max_attempts - 1:
            print(f"      ⏰ 2초 후 재시도...")
            time.sleep(2)
    
    print(f"      🚨 목록페이지 복귀 최종 실패")
    return False, None

def is_valid_list_page_url(url, city_name):
    """
    유효한 목록페이지 URL인지 확인
    """
    if not url:
        return False
    
    # 상품 상세페이지가 아닌지 확인
    if "/products/" in url and url.count("/") > 5:
        return False
    if "/offers/" in url and url.count("/") > 5:
        return False
    
    # 기본 목록페이지 패턴 확인
    valid_patterns = [
        "/experiences",
        "/offers",
        "/search"
    ]
    
    return any(pattern in url for pattern in valid_patterns)

# =============================================================================
# 🔄 페이지 이동 및 복구 시스템
# =============================================================================

def click_next_page_enhanced(driver, current_page=None):
    """
    KLOOK 전용 다음 페이지로 안전하게 이동 (강화된 버전)
    """
    print(f"    🔍 다음 페이지 버튼 찾는 중...")

    # 🆕 마지막 페이지 확인 (맨 앞에 추가)
    if not check_next_button(driver):
        print("🏁 마지막 페이지에 도달했습니다. 페이지네이션 종료")
        return False, "마지막 페이지", driver.current_url

    try:
        # 현재 상태 저장
        current_url = driver.current_url
        current_products = len(collect_all_24_urls(driver))

        # 🔧 KLOOK 전용 다음 페이지 버튼 셀렉터들
        next_button_selectors = [
            (By.CSS_SELECTOR, ".klk-pagination-next-btn:not(.klk-pagination-next-btn-disabled)"),
            (By.CSS_SELECTOR, ".filter-pagination .klk-pagination-next-btn:not(.klk-pagination-next-btn-disabled)"),
            (By.XPATH, "//span[@class='klk-pagination-next-btn' and not(contains(@class, 'disabled'))]"),
        ]

        next_button = None
        used_selector = None

        # 버튼 찾기
        for selector_type, selector in next_button_selectors:
            try:
                buttons = driver.find_elements(selector_type, selector)
                for button in buttons:
                    if button.is_displayed() and button.is_enabled():
                        next_button = button
                        used_selector = selector
                        print(f"    ✅ 다음 버튼 찾기 성공: {used_selector}")
                        break
                if next_button:
                    break
            except Exception:
                continue

        if not next_button:
            print(f"    ❌ 다음 페이지 버튼을 찾을 수 없습니다")
            return False, "다음 페이지 버튼 없음", current_url

        print(f"    ✅ 다음 페이지 버튼 발견!")

        # 안전한 클릭 실행
        for click_attempt in range(3):
            try:
                print(f"    🖱️ 다음 페이지 클릭 시도 {click_attempt + 1}/3...")

                # 스크롤 후 클릭
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", next_button)
                time.sleep(1)

                # JavaScript 클릭
                driver.execute_script("arguments[0].click();", next_button)

                # 페이지 로딩 대기
                print(f"    ⏰ 페이지 로딩 대기 중...")
                time.sleep(random.uniform(4, 7))

                # 변화 검증
                new_url = driver.current_url
                new_products = len(collect_all_24_urls(driver))

                # 성공 조건 확인
                if new_url != current_url:
                    print(f"    ✅ URL 변화 감지: 페이지 이동 성공!")
                    print(f"    📍 새 URL: ...{new_url[-60:]}")
                    return True, "페이지 이동 성공", new_url

                elif new_products != current_products and new_products > 0:
                    print(f"    ✅ 상품 수 변화 감지: {current_products} → {new_products}")
                    return True, "상품 변화로 이동 확인", new_url

                else:
                    print(f"    ⚠️ 클릭 {click_attempt+1}: 페이지 변화 미감지")
                    if click_attempt < 2:
                        time.sleep(2)
                        continue

            except Exception as e:
                print(f"    ❌ 클릭 시도 {click_attempt+1} 실패: {e}")
                if click_attempt < 2:
                    time.sleep(2)
                    continue

        # 모든 시도 실패
        print(f"    🏁 마지막 페이지이거나 페이지 이동 실패")
        return False, "페이지 이동 실패 (3회 시도 후)", current_url

    except Exception as e:
        print(f"    ❌ 다음 페이지 이동 중 오류: {e}")
        return False, f"오류: {type(e).__name__}", driver.current_url


# =============================================================================
# 🧰 유틸리티 함수들
# =============================================================================

def validate_pagination_environment():
    """
    페이지네이션 실행 환경 검증
    """
    print("🔍 페이지네이션 환경 검증 중...")
    
    # 필수 변수들 확인
    required_vars = ['driver', 'city_name', 'start_number', 'completed_urls', 'continent', 'country']
    missing_vars = []
    
    for var_name in required_vars:
        if var_name not in globals():
            missing_vars.append(var_name)
    
    if missing_vars:
        print(f"❌ 필수 변수 누락: {', '.join(missing_vars)}")
        print("💡 그룹 6-8을 먼저 실행하세요!")
        return False, missing_vars
    
    # 드라이버 상태 확인
    try:
        current_url = driver.current_url
        print(f"✅ 드라이버 상태: 정상 ({current_url[:50]}...)")
    except:
        print(f"❌ 드라이버 상태: 비정상")
        return False, ["driver_inactive"]
    
    # 현재 페이지가 목록페이지인지 확인
    try:
        product_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']")
        if len(product_links) >= 5:
            print(f"✅ 목록페이지 확인: {len(product_links)}개 상품 링크")
        else:
            print(f"⚠️ 목록페이지 의심: {len(product_links)}개 상품 링크만 발견")
    except:
        print(f"⚠️ 페이지 상태 확인 실패")
    
    print(f"✅ 페이지네이션 환경 검증 완료!")
    return True, []

def get_pagination_progress_info(total_crawled, target_products, current_page):
    """
    페이지네이션 진행 상황 정보 생성
    """
    progress_percentage = (total_crawled / target_products * 100) if target_products > 0 else 0
    remaining = max(0, target_products - total_crawled)
    
    return {
        'total_crawled': total_crawled,
        'target_products': target_products,
        'remaining': remaining,
        'progress_percentage': progress_percentage,
        'current_page': current_page,
        'estimated_pages': (target_products // 24) + (1 if target_products % 24 > 0 else 0)
    }

def print_pagination_progress(progress_info):
    """
    페이지네이션 진행률 시각적 표시
    """
    percentage = progress_info['progress_percentage']
    bar_length = 30
    filled_length = int(bar_length * progress_info['total_crawled'] // progress_info['target_products'])
    bar = '█' * filled_length + '░' * (bar_length - filled_length)
    
    print(f"\n📊 페이지네이션 진행률: [{bar}] {percentage:.1f}%")
    print(f"   🎯 진행: {progress_info['total_crawled']}/{progress_info['target_products']}개")
    print(f"   📄 페이지: {progress_info['current_page']}페이지")
    print(f"   ⏱️ 남은 상품: {progress_info['remaining']}개")
    print("-" * 60)

# =============================================================================
# 🎯 시스템 초기화 완료
# =============================================================================

print("✅ 그룹 9-A: 페이지네이션 핵심 시스템 로드 완료!")
print("   📊 save_pagination_state() - 페이지네이션 상태 저장")
print("   📊 load_pagination_state() - 페이지네이션 상태 로드")
print("   🔗 save_list_page_url() - 목록페이지 URL 저장")
print("   🔙 return_to_list_page() - 목록페이지 안전 복귀")
print("   🔄 click_next_page_enhanced() - 강화된 다음페이지 이동")
print("   🧰 validate_pagination_environment() - 환경 검증")
print()
print("🚀 다음: 그룹 9-B를 실행하여 완전한 페이지네이션 크롤링 시작!")

In [ ]:
# =============================================================================
# 🚀 그룹 9-B: 완전한 페이지네이션 크롤링 실행
# - 메인 페이지네이션 크롤링 루프, 단일 상품 크롤링, 테스트 함수들
# - 그룹 9-A의 핵심 시스템 + 기존 그룹 1-8의 모든 함수들 활용
# =============================================================================

print("🔧 그룹 9-B: 완전한 페이지네이션 크롤링 실행 로딩...")

# =============================================================================
# 🎯 메인 페이지네이션 크롤링 시스템
# =============================================================================

def crawl_with_full_pagination(city_name, target_products=100, resume_session=True, pre_collected_urls=None):
    """
    🎯 [최종 완성본] URL 캐시를 완벽하게 사용하는 지능형 크롤링 엔진
    - pre_collected_urls 인자를 받아, 캐시된 URL 목록으로 직접 작업을 수행합니다.
    - 캐시가 없을 때만 기존의 페이지네이션 방식으로 작동합니다.
    """
    print(f"🚀 최종 크롤링 엔진 시작: '{city_name}' 도시의 상품 {target_products}개 목표")
    print("=" * 80)
    
    # 1. 환경 및 전역 변수 설정 (기존과 동일)
    env_valid, missing = validate_pagination_environment()
    if not env_valid: return False
    global start_number, completed_urls, continent, country, driver
    
    # 2. 크롤링 상태 초기화 (기존과 동일)
    total_crawled_this_session = 0
    current_product_number = start_number 
    all_results = []

    print(f"📊 크롤링 설정:")
    print(f"   🏙️ 도시: {city_name} ({continent}/{country})")
    print(f"   🎯 목표 수량: {target_products}개")
    print(f"   🔢 시작 번호: {current_product_number}")
    print(f"   🔗 기완료 URL: {len(completed_urls)}개")

    # =================================================================
    # 👑 3. (핵심 로직) 작업 방식 결정: 캐시 사용 vs. 신규 탐색
    # =================================================================
    
    urls_to_process = []
    is_cache_mode = False

    if pre_collected_urls:
        print("\n✅ URL 캐시 모드로 실행: 제공된 목록을 우선 처리합니다.")
        urls_to_process = pre_collected_urls
        is_cache_mode = True
    
    # --- 캐시 모드 또는 라이브(페이지네이션) 모드 실행 ---

    page_results = [] # 수집된 결과를 임시 저장할 리스트
    
    # is_cache_mode가 True이면 한 번만 실행, False이면 페이지네이션 루프 실행
    current_page = 1
    while True:
        if not is_cache_mode: # 라이브 모드일 때만 URL을 새로 수집
            print(f"\n📄 === {current_page}페이지 처리 시작 ===")
            page_urls = collect_all_24_urls(driver)
            if not page_urls:
                print(f"🏁 {current_page}페이지에서 더 이상 상품을 찾을 수 없습니다. 작업을 종료합니다.")
                break
            print(f"✅ {current_page}페이지에서 {len(page_urls)}개의 URL을 발견했습니다.")
            urls_to_process = page_urls
        
        # --- 공통 상품 처리 로직 ---
        for url_index, product_url in enumerate(urls_to_process):
            if total_crawled_this_session >= target_products:
                break

            if product_url in completed_urls:
                print(f"⏭️ [내부 검증] 이미 완료된 URL이므로 건너뜁니다: {product_url[:50]}...")
                continue

            print(f"   📦 상품 {current_product_number} 처리 시작...")
            
            result = crawl_single_product_optimized(
                driver, product_url, current_product_number, 
                city_name, continent, country, current_page
            )
            
            if result:
                save_url_to_log(city_name, product_url)
                completed_urls.add(product_url)
                page_results.append(result)
                all_results.append(result)
                total_crawled_this_session += 1
                current_product_number += 1
                print(f"   ✅ 상품 {current_product_number-1} 완료: {result.get('상품명', '')[:30]}...")
        
        # --- 루프 제어 ---
        if total_crawled_this_session >= target_products:
            print(f"   🎉 목표 {target_products}개 달성! 이번 세션의 크롤링을 완료합니다.")
            break
            
        if is_cache_mode: # 캐시 모드는 한 번만 실행하고 종료
            break
        
        # 라이브 모드일 때만 다음 페이지로 이동
        print("🔄 다음 페이지로 이동합니다...")
        next_success, _, _ = click_next_page_enhanced(driver, current_page)
        if next_success:
            current_page += 1
            if page_results: # 다음 페이지로 넘어가기 전, 현재 페이지 결과 저장
                save_batch_data(page_results, city_name)
                page_results = [] # 초기화
        else:
            print("🏁 더 이상 다음 페이지가 없습니다. 페이지네이션을 종료합니다.")
            break
            
    # 최종적으로 남은 데이터 저장
    if page_results:
        save_batch_data(page_results, city_name)

    print(f"\n🎉 페이지네이션 크롤링 엔진이 작업을 마쳤습니다.")
    print(f"📊 이번 세션에서 총 {total_crawled_this_session}개의 상품을 수집했습니다.")
    return all_results




# =============================================================================
# 🔧 단일 상품 크롤링 함수 (기존 그룹 1-2 함수들 활용)
# =============================================================================

def crawl_single_product_optimized(driver, product_url, product_number, city_name, continent,
                                   country, page_num):
    """
    단일 상품 크롤링 최적화 버전 - KLOOK 메인+썸네일 이미지 (도시ID 추가)
    """
    try:
        # 상품 페이지 이동
        driver.get(product_url)
        time.sleep(random.uniform(CONFIG["MEDIUM_MIN_DELAY"], CONFIG["MEDIUM_MAX_DELAY"]))

        # 🆕 상품 상세페이지 자연스러운 스크롤 (봇 회피 강화)
        print(f"      📜 상품 페이지 자연스러운 탐색 중...")
        smart_scroll_selector(driver)

        # URL 타입 판별
        url_type = "Activity" if "/activity/" in product_url else "Product"

        # 정보 수집 (기존 그룹 1-2 함수들 활용)
        product_name = get_product_name(driver, url_type)
        price_raw = get_price(driver)
        price_clean = clean_price(price_raw)
        rating_raw = get_rating(driver)
        rating_clean = clean_rating(rating_raw)
        review_count = get_review_count(driver)
        language = get_language(driver)
        category = get_categories(driver)
        highlights = get_highlights(driver)

        # 🆕 도시ID 생성 (1부터 시작)
        city_code = get_city_code(city_name)
        city_id = f"{city_code}_{product_number}"

        # 🖼️ 메인+썸네일 이미지 다운로드 (신규 2개 이미지 시스템)
        if CONFIG["SAVE_IMAGES"]:
            img_results = download_image(driver, product_name, city_name, product_number)
            main_img = img_results.get('main_image', {})
            thumb_img = img_results.get('thumbnail_image', {})
        else:
            main_img = {'filename': '', 'relative_path': '', 'path': '', 'status': 'skipped'}
            thumb_img = {'filename': '', 'relative_path': '', 'path': '', 'status': 'skipped'}

        # 결과 반환 (도시ID + 2개 이미지 필드 - 8개 컬럼)
        return {
            '번호': product_number,
            '도시ID': city_id,  # 🆕 추가
            '페이지': page_num,
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '공항코드': city_code,
            '상품타입': url_type,
            '상품명': product_name,
            '가격_원본': price_raw,
            '가격_정제': price_clean,
            '평점_원본': rating_raw,
            '평점_정제': rating_clean,
            '리뷰수': review_count,
            '언어': language,
            '카테고리': category,
            '하이라이트': highlights,

            # 🖼️ 메인 이미지 정보 (4개 컬럼)
            '메인이미지_파일명': main_img.get('filename', ''),
            '메인이미지_상대경로': main_img.get('relative_path', ''),
            '메인이미지_전체경로': main_img.get('path', ''),
            '메인이미지_상태': main_img.get('status', ''),

            # 🖼️ 썸네일 이미지 정보 (4개 컬럼)
            '썸네일이미지_파일명': thumb_img.get('filename', ''),
            '썸네일이미지_상대경로': thumb_img.get('relative_path', ''),
            '썸네일이미지_전체경로': thumb_img.get('path', ''),
            '썸네일이미지_상태': thumb_img.get('status', ''),

            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '완전수집'
        }

    except Exception as e:
        print(f"      ❌ 상품 크롤링 실패: {e}")
        return None

# =============================================================================
# 🎯 시스템 로드 완료
# =============================================================================

print("✅ 그룹 9-B: 크롤링 엔진 로드 완료!")
print("   ⚙️ 이 셀은 실제 크롤링을 수행하는 핵심 엔진을 정의합니다.")
print("\n🚀 다음: 그룹 10을 실행하여 제어판을 사용하세요.")
print("=" * 80)

In [ ]:
# =============================================================================
# 🚀 그룹 10: KLOOK 전용 적응형 카테고리 시스템 (정리·안정화 버전)
# - 마이리얼트립 코드 완전 제거
# - KLOOK 전용 셀렉터 및 로직
# - 견고한 대기/클릭/중복제거/스코어링


# ---- 공통 설정 ---------------------------------------------------------------
DEFAULT_WAIT_SEC = 8
POST_CLICK_SLEEP_RANGE = (3.0, 5.0)

# KLOOK 탭/목록에서 자주 등장하는 키워드 스코어
KLOOK_TAB_KEYWORDS: List[Tuple[re.Pattern, int]] = [
    (re.compile(r"\b(tour|tours|투어)\b", re.I), 40),
    (re.compile(r"\b(activity|activities|액티비티)\b", re.I), 35),
    (re.compile(r"\b(ticket|tickets|티켓|입장권|admission)\b", re.I), 30),
    (re.compile(r"\b(transport|transportation|교통)\b", re.I), 20),
    (re.compile(r"\b(all|show all|전체)\b", re.I), 15),
]

# 목록·상세 판단에 사용할 셀렉터 묶음
KLOOK_PRODUCT_LINK_SEL = "a[href*='/activity/'], a[href*='/en/activity/']"
KLOOK_PRODUCT_CARD_SEL = (
    ".klk-card, [class*='Card'], a[href*='/activity/'] img, "
    "div[data-testid*='product-card']"
)

# 탭 후보 셀렉터
KLOOK_TAB_SELECTORS = [
    ".klk-tabs-tab",
    "[class*='tab'][role='tab']",
    "button[class*='tab']",
    "a[class*='tab']",
    "[data-testid*='tab']",
    ".tab-button",
]

# -----------------------------------------------------------------------------
# 유틸
# -----------------------------------------------------------------------------
def _wait(driver, timeout: int = DEFAULT_WAIT_SEC) -> WebDriverWait:
    return WebDriverWait(driver, timeout)

def _sleep_after_click() -> None:
    time.sleep(random.uniform(*POST_CLICK_SLEEP_RANGE))

def _visible(e: WebElement) -> bool:
    try:
        return e.is_displayed()
    except Exception:
        return False

def _clickable(driver, e: WebElement, timeout: int = DEFAULT_WAIT_SEC) -> bool:
    try:
        _wait(driver, timeout).until(EC.element_to_be_clickable(e))
        return True
    except Exception:
        return False

def _uniq_by_text_href(elems: List[WebElement]) -> List[WebElement]:
    """텍스트+href 조합으로 중복 제거 (WebElement는 해시불가)"""
    seen = set()
    out: List[WebElement] = []
    for el in elems:
        try:
            t = (el.text or "").strip()
            h = el.get_attribute("href") or ""
            key = (t, h)
            if key not in seen:
                seen.add(key)
                out.append(el)
        except Exception:
            # 문제가 있어도 다른 요소는 진행
            continue
    return out

def _score_tab(elem: WebElement) -> Tuple[int, List[str]]:
    score = 0
    reasons: List[str] = []
    txt = (elem.text or "").strip()
    href = elem.get_attribute("href") or ""
    role = elem.get_attribute("role") or ""
    cls = elem.get_attribute("class") or ""

    # 키워드
    for pat, pt in KLOOK_TAB_KEYWORDS:
        if pat.search(txt):
            score += pt
            reasons.append(f"키워드 매칭(+{pt}): {pat.pattern}")
            break

    # URL 특성
    if "klook.com" in href:
        score += 20
        reasons.append("klook 도메인(+20)")
    if any(s in href for s in ("/activity/", "/things-to-do/", "/attractions/")):
        score += 15
        reasons.append("상품/탐색 URL 패턴(+15)")

    # 표시/클릭 가능
    if _visible(elem):
        score += 10
        reasons.append("화면 표시(+10)")
    if elem.is_enabled():
        score += 10
        reasons.append("클릭 가능(+10)")

    # 탭 역할/활성 가점
    if role.lower() == "tab":
        score += 15
        reasons.append("role=tab(+15)")
    if re.search(r"\b(active|selected)\b", cls, flags=re.I):
        score += 5
        reasons.append("활성 상태(+5)")

    return score, reasons

# -----------------------------------------------------------------------------
# 1) KLOOK 페이지 타입 탐지
# -----------------------------------------------------------------------------
def detect_klook_page_type(driver) -> str:
    """KLOOK 페이지 타입 탐지 (KLOOK 전용) -> product_detail / product_list / destination_page / tab_based / non_klook / unknown_klook / error"""
    print("  🔍 KLOOK 페이지 타입 탐지 중...")
    try:
        current_url = driver.current_url
        print(f"    📍 현재 URL: {current_url}")

        if "klook.com" not in current_url:
            print("  ❌ KLOOK 도메인이 아닙니다")
            return "non_klook"

        if "/activity/" in current_url:
            print("  ✅ 감지: KLOOK 상품 상세 페이지")
            return "product_detail"

        if any(p in current_url for p in ("/things-to-do/", "/city/", "/country/")):
            print("  ✅ 감지: KLOOK 도시/국가 탐색 페이지")
            return "destination_page"

        # 탭/목록 힌트
        time.sleep(2)
        tabs = []
        for sel in KLOOK_TAB_SELECTORS:
            try:
                tabs.extend(driver.find_elements(By.CSS_SELECTOR, sel))
            except Exception:
                pass
        tabs = _uniq_by_text_href(tabs)

        if tabs:
            print(f"  ✅ 감지: KLOOK 탭 기반 페이지 (탭 {len(tabs)}개)")
            return "tab_based"

        links = driver.find_elements(By.CSS_SELECTOR, KLOOK_PRODUCT_LINK_SEL)
        cards = driver.find_elements(By.CSS_SELECTOR, KLOOK_PRODUCT_CARD_SEL)
        if len(links) + len(cards) >= 10:
            print(f"  ✅ 감지: KLOOK 상품 목록 페이지 (요소 {len(links)+len(cards)}개)")
            return "product_list"

        print("  ⚠️ 알 수 없는 KLOOK 페이지 타입")
        return "unknown_klook"

    except Exception as e:
        print(f"  ❌ 페이지 타입 탐지 실패: {e}")
        return "error"

# -----------------------------------------------------------------------------
# 2) KLOOK 카테고리 탭 찾기/분석
# -----------------------------------------------------------------------------
def find_klook_category_tabs(driver) -> List[Dict]:
    """KLOOK 카테고리 탭 찾기 (KLOOK 전용)"""
    print("  🔍 KLOOK 카테고리 탭 찾기 시작...")

    all_tabs: List[WebElement] = []
    for selector in KLOOK_TAB_SELECTORS:
        try:
            elems = driver.find_elements(By.CSS_SELECTOR, selector)
            if elems:
                print(f"    📊 선택자 '{selector}': {len(elems)}개 탭")
                all_tabs.extend(elems)
        except Exception as e:
            print(f"    ⚠️ 선택자 '{selector}' 실패: {e}")

    all_tabs = _uniq_by_text_href(all_tabs)
    print(f"    📈 총 {len(all_tabs)}개 후보 탭(중복 제거)")

    analyzed: List[Dict] = []
    for i, el in enumerate(all_tabs):
        try:
            txt = (el.text or "").strip()
            href = el.get_attribute("href") or ""
            data_value = el.get_attribute("data-value") or ""
            score, reasons = _score_tab(el)

            print(f"    🔍 탭 {i}: '{txt}' -> {href}")
            print(f"        📊 점수: {score}점")
            if reasons:
                print(f"        📝 이유: {', '.join(reasons)}")

            analyzed.append({
                "element": el,
                "text": txt,
                "href": href,
                "data_value": data_value,
                "score": score,
                "index": i,
            })
        except Exception as e:
            print(f"    ⚠️ 탭 {i} 분석 실패: {e}")

    return analyzed

# -----------------------------------------------------------------------------
# 3) 탭 클릭 시도
# -----------------------------------------------------------------------------
def attempt_klook_tab_click(driver, tab_info: Dict) -> Tuple[bool, str]:
    """KLOOK 탭 클릭 시도 (KLOOK 전용): (성공여부, 결과URL/사유)"""
    elem: WebElement = tab_info["element"]
    text: str = tab_info.get("text", "")

    try:
        driver.execute_script("arguments[0].scrollIntoView({block:'center'});", elem)
        time.sleep(0.8)

        if _clickable(driver, elem):
            # JS 우선 → 실패 시 일반 클릭
            try:
                driver.execute_script("arguments[0].click();", elem)
                print(f"        ✅ JavaScript 클릭 성공: '{text}'")
            except Exception:
                elem.click()
                print(f"        ✅ 일반 클릭 성공: '{text}'")
        else:
            # 비클릭 요소는 JS로 강제 시도
            driver.execute_script("arguments[0].click();", elem)
            print(f"        ⚠️ 강제 JS 클릭: '{text}'")

        _sleep_after_click()

        new_url = driver.current_url
        links = driver.find_elements(By.CSS_SELECTOR, KLOOK_PRODUCT_LINK_SEL)
        cards = driver.find_elements(By.CSS_SELECTOR, KLOOK_PRODUCT_CARD_SEL)
        product_count = len(links) + len(cards)
        print(f"        📊 이동 후 상품 추정 요소 수: {product_count}")

        if product_count >= 3:
            print("        ✅ KLOOK 상품 목록/결과 노출 확인!")
            return True, new_url
        return False, "상품 수 부족"

    except Exception as e:
        print(f"        ❌ 클릭 시도 실패: {e}")
        return False, f"클릭 오류: {e}"

# -----------------------------------------------------------------------------
# 4) 목표 카테고리로 이동
# -----------------------------------------------------------------------------
def navigate_to_klook_category(driver, city_name: str, target_category: str = "투어") -> Tuple[bool, str]:
    """KLOOK 카테고리 이동 (KLOOK 전용) -> (성공여부, 결과URL/사유)"""
    print(f"  🎯 KLOOK '{target_category}' 카테고리 이동 시작...")

    try:
        time.sleep(2)
        print("      🔍 KLOOK 페이지 탐색 중...")
        # smart_scroll_selector 가 있다면 사용
        if callable(globals().get("smart_scroll_selector")):
            try:
                globals()["smart_scroll_selector"](driver)
            except Exception as e:
                print(f"      ⚠️ smart_scroll_selector 실패: {e}")

        tabs = find_klook_category_tabs(driver)
        if not tabs:
            print("    ❌ KLOOK 카테고리 탭을 찾을 수 없습니다")
            return False, "탭 없음"

        # 목표 카테고리 우선 필터
        target_tabs = [t for t in tabs if target_category.lower() in t["text"].lower()]
        if target_tabs:
            target_tabs.sort(key=lambda x: x["score"], reverse=True)
            print(f"    🎯 '{target_category}' 관련 탭 {len(target_tabs)}개 발견")
        else:
            tabs.sort(key=lambda x: x["score"], reverse=True)
            target_tabs = tabs[:3]
            print("    📊 목표 키워드 탭 없음 → 상위 점수 탭 시도")

        for attempt, tab in enumerate(target_tabs, 1):
            if tab["score"] < 15:
                print(f"    ⚠️ 탭 점수 낮음({tab['score']}점) → 시도 중단")
                break
            print(f"    🎯 시도 {attempt}: '{tab['text']}' ({tab['score']}점)")
            ok, res = attempt_klook_tab_click(driver, tab)
            if ok:
                print("    ✅ 탭 클릭 성공!")
                return True, res
            print(f"    ❌ 탭 클릭 실패: {res}")
            if attempt < len(target_tabs):
                print("    🔄 다음 후보 탭 시도...")
                time.sleep(1.5)

        print("    ❌ 모든 탭 시도 실패")
        return False, "모든 탭 실패"

    except Exception as e:
        print(f"  ❌ KLOOK 카테고리 이동 실패: {e}")
        return False, f"오류: {e}"

# -----------------------------------------------------------------------------
# 5) KLOOK TAB STRUCTURE 연동
# -----------------------------------------------------------------------------
def get_klook_tab_info(tab_name: str) -> Dict:
    """KLOOK_TAB_STRUCTURE에서 탭 정보 가져오기 (없으면 빈 dict)"""
    try:
        if "KLOOK_TAB_STRUCTURE" in globals():
            return globals()["KLOOK_TAB_STRUCTURE"].get(tab_name, {}) or {}
    except Exception:
        pass
    return {}

# =============================================================================
# 로딩 로그
# =============================================================================
print("✅ KLOOK 전용 그룹 10 시스템 로드 완료!\n")
print("🔧 KLOOK 전용 개선사항:")
print("   🎯 전용 셀렉터 강화 (.klk-tabs-tab, a[href*='/activity/'], product-card 등)")
print("   🌍 klook 도메인/URL 패턴 기반 탐지")
print("   📊 키워드 점수 시스템 + 활성/role 가점")
print("   🧼 탭 수집 중복 제거(텍스트+href)")
print("   ⏳ 견고한 대기/클릭 폴백(JS→일반)")
print("   🔄 KLOOK_TAB_STRUCTURE 연동")



In [ ]:
# =============================================================================
# 🎯 그룹 11: 정리된 크롤링 정지 시스템 (경량화 버전)
# - 크롤링 정지 기능만 유지
# - 시스템 진단 및 초기화 기능 제거
# - 핵심 크롤링 로직은 100% 보존
# =============================================================================

# 전역 정지 플래그
CRAWLING_STOP_FLAG = False
CRAWLING_ACTIVE = False

def set_stop_flag():
    """크롤링 정지 플래그 설정"""
    global CRAWLING_STOP_FLAG
    CRAWLING_STOP_FLAG = True
    print("🛑 크롤링 정지 신호가 전송되었습니다...")

def reset_stop_flag():
    """크롤링 정지 플래그 초기화"""
    global CRAWLING_STOP_FLAG
    CRAWLING_STOP_FLAG = False

def check_stop_flag():
    """크롤링 정지 플래그 확인"""
    return CRAWLING_STOP_FLAG

def set_crawling_active(status):
    """크롤링 활성 상태 설정"""
    global CRAWLING_ACTIVE
    CRAWLING_ACTIVE = status

def is_crawling_active():
    """크롤링 활성 상태 확인"""
    return CRAWLING_ACTIVE

def perform_new_search(driver, city):
    """새로운 검색을 수행하는 공통 함수"""
    # 필수 함수들 존재 여부 확인
    required_functions = [
        'go_to_main_page', 'find_and_fill_search', 'click_search_button',
        'handle_popup', 'click_view_all'
    ]
    
    missing_functions = []
    for func_name in required_functions:
        if func_name not in globals():
            missing_functions.append(func_name)
    
    if missing_functions:
        print(f"❌ 필수 함수들이 없습니다: {', '.join(missing_functions)}")
        print("💡 그룹 5 (브라우저 제어 함수들)을 먼저 실행하세요!")
        return False
    
    try:
        go_to_main_page(driver)
        find_and_fill_search(driver, city)
        click_search_button(driver)
        print("  ✅ 페이지 최적화 중 (팝업 처리 및 전체 보기)...")
        handle_popup(driver)
        click_view_all(driver)
        print("  ⏳ 상품 목록 페이지가 완전히 로드될 때까지 대기합니다...")
        
        # WebDriverWait 및 관련 모듈들 import 확인
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']"))
            )
            print("  ✅ 상품 목록 페이지 로드 확인 완료.")
        except TimeoutException:
            print("  ⚠️ 시간 초과: 상품 목록 페이지 로드에 실패했을 수 있습니다.")
        except NameError:
            print("  ⚠️ WebDriverWait 모듈을 찾을 수 없습니다 - 기본 대기 사용")
            import time
            time.sleep(5)
            print("  ✅ 기본 대기 완료")
        
        return True
        
    except Exception as e:
        print(f"❌ 새로운 검색 실패: {e}")
        return False

def perform_new_search_with_switch(driver, city, use_group10=False):
    """간소화된 스위치 기반 검색 함수"""
    # 필수 함수들 존재 여부 확인
    required_functions = [
        'go_to_main_page', 'find_and_fill_search', 'click_search_button',
        'handle_popup', 'click_view_all'
    ]
    
    missing_functions = []
    for func_name in required_functions:
        if func_name not in globals():
            missing_functions.append(func_name)
    
    if missing_functions:
        print(f"❌ 필수 함수들이 없습니다: {', '.join(missing_functions)}")
        print("💡 그룹 5 (브라우저 제어 함수들)을 먼저 실행하세요!")
        return False
    
    try:
        go_to_main_page(driver)
        find_and_fill_search(driver, city)
        click_search_button(driver)
        handle_popup(driver)
        print("✅ 검색 완료 → 상품 목록 페이지 도달")
        
        if use_group10:
            # 🚀 그룹10모드 (간소화됨)
            print("  🚀 스크롤 모드: 새로운 페이지 구조 대응")
            success, result = navigate_to_scroll_list(driver, city)
            
            if success:
                print("  ✅ 스크롤 모드 성공!")
                return True
            else:
                print(f"  ❌ 스크롤 모드 실패: {result} - 기본모드로 전환")
                # fallback 계속 진행
        
        # 🔄 기본모드 (그룹10 실패 시에도 실행)
        print("  🔄 기본모드: 안정적인 방식")
        click_view_all(driver)
        print("  ⏳ 상품 목록 페이지가 완전히 로드될 때까지 대기합니다...")
        
        # WebDriverWait 및 관련 모듈들 import 확인
        try:
            WebDriverWait(driver, 15).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/products/'], a[href*='/offers/']"))
            )
            print("  ✅ 상품 목록 페이지 로드 확인 완료.")
        except TimeoutException:
            print("  ⚠️ 시간 초과: 상품 목록 페이지 로드에 실패했을 수 있습니다.")
        except NameError:
            print("  ⚠️ WebDriverWait 모듈을 찾을 수 없습니다 - 기본 대기 사용")
            import time
            time.sleep(5)
            print("  ✅ 기본 대기 완료")
        
        return True
        
    except Exception as e:
        print(f"❌ 검색 실패: {e}")
        return False

def run_crawler_with_stop_support(city, num_products_to_crawl, use_group10=False, resume_session=True):
        
    """
    🛑 정지 기능이 통합된 크롤링 실행 함수
    """
    # 필수 함수 존재 여부 확인
    required_functions = [
        'load_session_state', 'save_url_to_log', 'get_city_info',
        'perform_new_search_with_switch', 'load_collected_urls', 
        'collect_urls_with_csv_safety', 'save_collected_urls'
    ]
    missing_functions = [f for f in required_functions if f not in globals()]
    if missing_functions:
        print(f"❌ 필수 함수들이 없습니다: {', '.join(missing_functions)}")
        return

    # 정지 시스템 초기화
    reset_stop_flag()
    set_crawling_active(True)
    
    try:
        # 상태 복원 및 설정 통합
        global CITIES_TO_SEARCH, city_name, start_number, completed_urls, driver, urls_to_crawl, continent, country
        
        city_name = city
        CITIES_TO_SEARCH = [city]
        continent, country = get_city_info(city_name)
        
        # 정지 신호 체크 포인트 1
        if check_stop_flag():
            print("🛑 시작 전 정지 신호 감지 - 크롤링 중단")
            return
        try:
            start_number, completed_urls = load_session_state(city_name)
            if len(completed_urls) == 0:
                print(f"📊 신규 도시 - {start_number}번부터 시작")
            else:
                print(f"📊 재개 세션 - {len(completed_urls)}개 완료, {start_number}번부터 계속")
        except Exception as e:
            print(f"❌ 상태 복원 중 오류: {e}")
            return

        # 정지 신호 체크 포인트 2
        if check_stop_flag():
            print("🛑 상태 복원 후 정지 신호 감지 - 크롤링 중단")
            return

        # URL 목록 결정 (기존 캐시 활용)
        urls_to_crawl = []
        try:
            potential_urls = load_collected_urls(city)
            if potential_urls:
                new_urls = [url for url in potential_urls if url not in completed_urls]
                if new_urls:
                    print(f"✅ 기존 캐시에서 {len(new_urls)}개의 미완료 작업을 재사용합니다.")
                    urls_to_crawl = new_urls
        except Exception:
            pass

        # 정지 신호 체크 포인트 3
        if check_stop_flag():
            print("🛑 URL 로드 후 정지 신호 감지 - 크롤링 중단")
            return

        # 새로운 URL 검색
        if not urls_to_crawl:
            print(f"🔄 '{city}' 도시의 신규 검색을 시작합니다.")
            try:
                search_success = perform_new_search_with_switch(driver, city, use_group10)
                if not search_success:
                    print("❌ 상품 목록 페이지 이동 실패")
                    return

                # 정지 신호 체크 포인트 4
                if check_stop_flag():
                    print("🛑 검색 완료 후 정지 신호 감지 - 크롤링 중단")
                    return
                
                newly_collected_urls = collect_urls_with_csv_safety(driver, city)
                urls_to_crawl = newly_collected_urls
                if urls_to_crawl:
                    save_collected_urls(city, urls_to_crawl)
                    print(f"📊 URL 수집: {len(urls_to_crawl)}개 발견")
                               
            except Exception as e:
                print(f"❌ 신규 URL 검색 실패: {e}")
                return

        # 정지 신호 체크 포인트 5
        if check_stop_flag():
            print("🛑 URL 수집 후 정지 신호 감지 - 크롤링 중단")
            return

        # 최종 크롤링 실행
        if not urls_to_crawl:
            print("🎉 수집할 새로운 상품이 없습니다.")
            return
        
        final_urls_to_crawl = urls_to_crawl[:num_products_to_crawl]
        print(f"🎯 {len(final_urls_to_crawl)}개 상품 크롤링을 시작합니다.")

        # 정지 기능이 통합된 크롤링 엔진 호출
        try:
            crawl_with_pagination_and_stop_check(
                city_name=city,
                target_products=len(final_urls_to_crawl),
                resume_session=resume_session,
                pre_collected_urls=final_urls_to_crawl
            )
            
            if not check_stop_flag():
                print(f"\n🎉 '{city}' 도시 크롤링 작업이 정상 완료되었습니다.")
            else:
                print(f"\n🛑 '{city}' 도시 크롤링이 사용자 요청으로 중단되었습니다.")
                
        except Exception as e:
            print(f"❌ 크롤링 실행 실패: {e}")
            
    finally:
        # 크롤링 종료 시 플래그 정리
        set_crawling_active(False)
        reset_stop_flag()
        print("🏁 크롤링 세션 종료")

def crawl_with_pagination_and_stop_check(city_name, target_products,
                                        resume_session=True, pre_collected_urls=None):
    """
    🛑 정지 체크가 통합된 메인 크롤링 엔진
    """
    try:
        # 환경 검증 (조용히 실행)
        try:
            env_valid, missing = validate_pagination_environment()
            if not env_valid:
                print(f"❌ 환경 검증 실패: {missing}")
                return False
        except:
            pass
        
        global start_number, completed_urls, continent, country, driver
        
        # 크롤링 상태 초기화
        current_product_number = start_number
        total_crawled_this_session = 0
        page_results = []
        print(f"🎯 {target_products}개 상품 크롤링 시작")
        
        # URL 목록 처리
        urls_to_process = pre_collected_urls or []
        
        # 메인 크롤링 루프 (정지 신호 통합)
        for i, product_url in enumerate(urls_to_process):
            # 각 상품 처리 전 정지 신호 체크
            if check_stop_flag():
                print(f"🛑 상품 {current_product_number} 처리 전 정지 신호 감지")
                print(f"💾 현재까지 수집된 {len(page_results)}개 상품을 저장합니다...")
                if page_results:
                    save_batch_data(page_results, city_name)
                print(f"✅ 정지 완료: 총 {total_crawled_this_session}개 상품 수집됨")
                return True
            
            if total_crawled_this_session >= target_products:
                break
            
            if product_url in completed_urls:
                print(f"⏭️ 이미 완료된 URL 건너뜀: {product_url[:50]}...")
                continue
            
            # 🆕 간소화된 상품 처리 시작 메시지
            print(f"📦 상품 {current_product_number} 처리 중...")
            
            # 정지 체크가 통합된 상품 크롤링
            result = crawl_single_product_with_stop_check(
                driver, product_url, current_product_number,
                city_name, continent, country, 1
            )
            
            # 상품 크롤링 후 정지 신호 재확인
            if check_stop_flag():
                print(f"🛑 상품 {current_product_number} 처리 후 정지 신호 감지")
                if result:  # 현재 상품이 성공적으로 완료된 경우
                    print(f"✅ 현재 상품 크롤링 완료 후 정지")
                    save_url_to_log(city_name, product_url)
                    completed_urls.add(product_url)
                    page_results.append(result)
                    total_crawled_this_session += 1
                
                print(f"💾 현재까지 수집된 {len(page_results)}개 상품을 저장합니다...")
                if page_results:
                    save_batch_data(page_results, city_name)
                print(f"✅ 정지 완료: 총 {total_crawled_this_session}개 상품 수집됨")
                return True
            
            if result:
                save_url_to_log(city_name, product_url)
                completed_urls.add(product_url)
                page_results.append(result)
                total_crawled_this_session += 1
                current_product_number += 1
                
                # 🆕 간소화된 완료 메시지
                product_info = f"{result.get('상품명', '')[:40]}... | {result.get('가격_정제', '')} | {result.get('리뷰수', '')}"
                print(f"✅ 상품 {current_product_number-1}: {product_info}")
        
        # 최종 데이터 저장
        if page_results:
            save_batch_data(page_results, city_name)
        
        print(f"\n🎉 크롤링 작업 완료!")
        print(f"📊 총 {total_crawled_this_session}개 상품 수집됨")
        return True
        
    except Exception as e:
        print(f"❌ 크롤링 중 오류 발생: {e}")
        if 'page_results' in locals() and page_results:
            save_batch_data(page_results, city_name)
        return False

def crawl_single_product_with_stop_check(driver, product_url, product_number, city_name, continent, country, page_num):
    """
    🛑 정지 신호 체크가 통합된 단일 상품 크롤링
    """
    try:
        # 페이지 이동 전 정지 신호 체크
        if check_stop_flag():
            print(f"      🛑 상품 페이지 이동 전 정지 신호 감지")
            return None
            
        # 상품 페이지 이동
        driver.get(product_url)
        time.sleep(random.uniform(CONFIG["MEDIUM_MIN_DELAY"], CONFIG["MEDIUM_MAX_DELAY"]))
        
        # 🆕 상품 상세페이지 자연스러운 스크롤 (봇 회피 강화)
        print(f"      📜 상품 페이지 자연스러운 탐색 중...")
        smart_scroll_selector(driver)

        # 페이지 로드 후 정지 신호 체크
        if check_stop_flag():
            print(f"      🛑 상품 페이지 로드 후 정지 신호 감지")
            return None
        
        # URL 타입 판별
        url_type = "Activity" if "/activity/" in product_url else "Product"
        
        # 정보 수집 (각 단계마다 정지 신호 체크)
        if check_stop_flag(): return None
        product_name = get_product_name(driver, url_type)
        
        if check_stop_flag(): return None
        price_raw = get_price(driver)
        price_clean = clean_price(price_raw)
        
        if check_stop_flag(): return None
        rating_raw = get_rating(driver)
        rating_clean = clean_rating(rating_raw)
        
        if check_stop_flag(): return None
        review_count = get_review_count(driver)
        
        if check_stop_flag(): return None
        language = get_language(driver)

        if check_stop_flag(): return None
        category = get_categories(driver)

        if check_stop_flag(): return None
        highlights = get_highlights(driver)
        
        # 도시ID 생성
        city_code = get_city_code(city_name)
        city_id = f"{city_code}_{product_number}"
        
        # 🖼️ 메인+썸네일 이미지 다운로드 (정지 신호 체크 포함)
        if check_stop_flag():
            return None

        if CONFIG["SAVE_IMAGES"]:
            img_results = download_image(driver, product_name, city_name, product_number)
            main_img = img_results.get('main_image', {})
            thumb_img = img_results.get('thumbnail_image', {})
        else:
            main_img = {'filename': '', 'relative_path': '', 'path': '', 'status': 'skipped'}
            thumb_img = {'filename': '', 'relative_path': '', 'path': '', 'status': 'skipped'}
                       
        
        # 최종 정지 신호 체크
        if check_stop_flag():
            print(f"      🛑 상품 데이터 수집 완료 후 정지 신호 감지")
            return None
        
        # 결과 반환
        return {
            '번호': product_number,
            '도시ID': city_id,
            '페이지': page_num,
            '대륙': continent,
            '국가': country,
            '도시': city_name,
            '공항코드': city_code,
            '상품타입': url_type,
            '상품명': product_name,
            '가격_원본': price_raw,
            '가격_정제': price_clean,
            '평점_원본': rating_raw,
            '평점_정제': rating_clean,
            '리뷰수': review_count,
            '언어': language,
            '카테고리': category,
            '하이라이트': highlights,
            '메인이미지_파일명': main_img.get('filename', ''),
            '메인이미지_상대경로': main_img.get('relative_path', ''),
            '메인이미지_전체경로': main_img.get('path', ''),
            '메인이미지_상태': main_img.get('status', ''),
            '썸네일이미지_파일명': thumb_img.get('filename', ''),
            '썸네일이미지_상대경로': thumb_img.get('relative_path', ''),
            '썸네일이미지_전체경로': thumb_img.get('path', ''),
            '썸네일이미지_상태': thumb_img.get('status', ''),
            'URL': product_url,
            '수집_시간': time.strftime('%Y-%m-%d %H:%M:%S'),
            '상태': '완전수집'
        }
        
    except Exception as e:
        print(f"      ❌ 상품 크롤링 실패: {e}")
        return None

print("✅ 그룹 11 완료: 정리된 크롤링 정지 시스템!")
print("   🛑 정지 플래그 시스템")
print("   🔧 정지 지원 크롤링 엔진")
print("   🚀 핵심 크롤링 로직 100% 보존")
print("   📦 코드 크기: 약 50% 감소")
print("🎯 다음: 그룹 12에서 UI 간소화")

In [ ]:
# =============================================================================
# 🎛️ 그룹 12: 간소화된 UI 컨트롤 패널 (핵심 기능만)
# - 크롤링 시작/정지 기능만 유지
# - 시스템 진단 및 초기화 기능 제거
# - UI 깔끔하게 정리
# =============================================================================

print("🔧 그룹 12: 간소화된 UI 컨트롤 패널 로딩")

import ipywidgets as widgets
from IPython.display import display

# =============================================================================
# 🎛️ 위젯 생성 (핵심 기능만)
# =============================================================================

# 입력 위젯들
city_input = widgets.Text(value='', description='도시:', layout={'width': '200px'})
product_count_input = widgets.IntText(value=None, description='상품 수:', layout={'width': '200px'})

# 핵심 버튼들 (시작/정지만)
run_button = widgets.Button(description="🚀 크롤링 시작", button_style='danger')
stop_button = widgets.Button(description="🛑 크롤링 정지", button_style='warning', disabled=True)

# 스크롤 모드 스위치
group10_switch = widgets.Checkbox(
    value=False,
    description='🚀 스크롤 모드',
    disabled=False,
    indent=False
)

# 상태 표시 라벨들
mode_label = widgets.Label(
    value="🔄 기본모드 (안정적)",
    layout=widgets.Layout(width='250px')
)

status_label = widgets.Label(
    value="🔵 대기 중",
    layout=widgets.Layout(width='300px')
)

def on_switch_change(change):
    """스위치 변경 시 라벨 업데이트"""
    if change['new']:
        mode_label.value = "🚀 스크롤 모드 (새로운 페이지 대응)"
    else:
        mode_label.value = "🔄 기본모드 (안정적)"

group10_switch.observe(on_switch_change, names='value')

output_area = widgets.Output()

# =============================================================================
# 🎛️ 버튼 이벤트 함수들 (핵심 기능만)
# =============================================================================

def on_run_button_clicked(b):
    """크롤링 시작 버튼 클릭 (정지 기능 통합)"""
    with output_area:
        output_area.clear_output()
        city = city_input.value.strip()
        count = product_count_input.value
        use_group10 = group10_switch.value
                
        # 입력 검증
        if not city:
            print("⚠️ 도시 이름을 입력해주세요!")
            return
        if count is None or count <= 0:
            print("⚠️ 수집할 상품 개수를 1 이상으로 입력해주세요!")
            return
        
        # 버튼 상태 변경
        run_button.disabled = True
        stop_button.disabled = False
        status_label.value = f"🟡 크롤링 진행 중... ({city})"
        
        mode_text = "스크롤 모드" if use_group10 else "기본모드"
        print(f"🚀 '{city}' 도시 상품 {count}개 크롤링을 시작합니다... ({mode_text})")
        print(f"🛑 크롤링을 중단하려면 '크롤링 정지' 버튼을 클릭하세요!")
        
        try:
            # 정지 지원 크롤링 함수 호출
            run_crawler_with_stop_support(city=city, num_products_to_crawl=count, use_group10=use_group10)
        except Exception as e:
            print(f"❌ 크롤링 실행 중 오류: {e}")
        finally:
            # 크롤링 완료 후 버튼 상태 복원
            run_button.disabled = False
            stop_button.disabled = True
            status_label.value = "🔵 대기 중"

def on_stop_button_clicked(b):
    """크롤링 정지 버튼 클릭"""
    with output_area:
        print("\n🛑 사용자가 크롤링 정지를 요청했습니다!")
        print("📋 현재 진행 중인 상품을 완료한 후 안전하게 정지합니다...")
        
        set_stop_flag()
        stop_button.disabled = True
        status_label.value = "🟠 정지 처리 중..."

# =============================================================================
# 🎛️ 버튼 이벤트 연결 및 위젯 배치
# =============================================================================

# 버튼과 함수 연결
run_button.on_click(on_run_button_clicked)
stop_button.on_click(on_stop_button_clicked)

# 간소화된 위젯 배치
controls = widgets.VBox([
    widgets.Label(value="🚀 KLOOK 크롤러 (간소화 버전)"),
    widgets.Label(value=""),
    city_input,
    product_count_input,
    widgets.Label(value=""),
    widgets.HBox([
        mode_label,
        widgets.HTML(value="&nbsp;&nbsp;&nbsp;"),
        group10_switch
    ], layout=widgets.Layout(align_items='center')),
    widgets.Label(value=""),
    status_label,
    widgets.Label(value=""),
    widgets.HBox([
        widgets.HBox([run_button], layout=widgets.Layout(width='250px')),  # 왼쪽 고정폭
        stop_button
    ], layout=widgets.Layout(align_items='center'))
])

display(controls, output_area)

print("✅ 그룹 12 완료: 간소화된 UI 컨트롤 패널!")
print("🎯 핵심 기능:")
print("   - 🚀 크롤링 시작 (기본모드/스크롤모드)")
print("   - 🛑 크롤링 정지 (안전한 중단)")
print("   - 📊 실시간 상태 표시")
print("🧹 제거된 기능:")
print("   - 🩺 시스템 진단 (필요시 코드 셀에서 직접 실행)")
print("   - 🔄 시스템 초기화 (필요시 코드 셀에서 직접 실행)")